In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import estnltk
from estnltk import Text

In [8]:
from estnltk.taggers.estner.refac.ner import NerTagger
ntager = NerTagger()


In [19]:
text = Text('''Eesti Vabariik on riik Põhja-Euroopas.
    Eesti piirneb põhjas üle Soome lahe Soome Vabariigiga.
    ''')

In [10]:
ntager.tag_document(text)

Token:
Eesti
Token:
Vabariik
Token:
on
Token:
riik
Token:
Põhja-Euroopas
Token:
.
Token:
Eesti
Token:
piirneb
Token:
põhjas
Token:
üle
Token:
Soome
Token:
lahe
Token:
Soome
Token:
Vabariigiga
Token:
.
[{'p1': 'e', 's1': 'i', 'F': [], 'wl': 'eesti', 'cl': 'y', 's2': 'ti', 'cu': 'y', 'pos': '_H_', 'shape': 'ULLLL', 's4': 'esti', 'lem': 'eesti', 'shaped': 'UL', 'p3': 'ees', 'p4': 'eest', 'iu': 'y', 'prop': 'y', 'w': 'Eesti', 'fsnt': 'y', 'aan': 'y', 'gaz': {'loc'}, 'p2': 'ee', 'ca': 'y', 'len': '5', 'case': 'g', 's3': 'sti'}, {'p1': 'e', 's1': 'i', 'F': [], 'wl': 'eesti', 'cl': 'y', 's2': 'ti', 'cu': 'y', 'pos': '_H_', 'shape': 'ULLLL', 's4': 'esti', 'lem': 'eesti', 'shaped': 'UL', 'p3': 'ees', 'p4': 'eest', 'iu': 'y', 'prop': 'y', 'w': 'Eesti', 'aan': 'y', 'gaz': {'loc'}, 'p2': 'ee', 'ca': 'y', 'len': '5', 'case': 'n', 's3': 'sti'}]
[{'p1': 'p', 's1': 's', 'F': [], 'wl': 'põhja-euroopas', 'cl': 'y', 's2': 'as', 'cu': 'y', 'post': 'roobas', 'pos': '_H_', 'shape': 'ULLLL-ULLLLLLL', 's4': '

AttributeError: 'Document' object has no attribute 'ner_features'

In [4]:
from estnltk.taggers.estner.ner_tagger import NerTagger as nt2

nertagger = nt2()

In [106]:
this = [['iu[0]=y', 'fsnt[0]=y', 'p3[0]=ees', 'p4[0]=eest', 's3[0]=sti', 's4[0]=esti', 'aan[0]=y', 'cu[0]=y', 'iu[1]=y', 'cu[1]=y', 'lem[0]=eesti', 'pos[0]=_H_', 'case[0]=g', 'lem[1]=vabariik', 'pos[1]=_S_', 'post[1]=riik', 'lem[2]=ole', 'pos[2]=_V_', 'iuoc[0]=y', 'ngaz[0]=loc', 'gaz[0]=loc', 'gaz[1]=loc', 'iu[0]|fsnt[0]=y|y', 'lem[0]|lem[1]=eesti|vabariik', 'pos[0]|pos[1]=_H_|_S_', 'iu[0]|iu[1]=y|y', 'gaz[0]|gaz[1]=loc|loc'], ['iu[0]=y', 'p3[0]=vab', 'p4[0]=vaba', 's3[0]=iik', 's4[0]=riik', 'aan[0]=y', 'cu[0]=y', 'iu[-1]=y', 'cu[-1]=y', 'fsnt[-1]=y', 'lem[0]=vabariik', 'pos[0]=_S_', 'pref[0]=vaba', 'post[0]=riik', 'case[0]=n', 'lem[-1]=eesti', 'prop[-1]=y', 'pos[-1]=_H_', 'lem[1]=ole', 'pos[1]=_V_', 'lem[2]=riik', 'pos[2]=_S_', 'iuoc[0]=y', 'pprop[0]=y', 'pgaz[0]=loc', 'gaz[0]=loc', 'gaz[-1]=loc', 'lem[0]|lem[-1]=vabariik|eesti', 'lem[0]|lem[1]=vabariik|ole', 'pos[0]|pos[-1]=_S_|_H_', 'pos[0]|pos[1]=_S_|_V_', 'iu[0]|iu[-1]=y|y', 'gaz[-1]|gaz[0]=loc|loc'], ['p3[0]=ole', 's3[0]=ole', 'aan[0]=y', 'iu[-1]=y', 'cu[-1]=y', 'iu[-2]=y', 'cu[-2]=y', 'fsnt[-2]=y', 'iu[2]=y', 'cu[2]=y', 'cs[2]=y', 'cds[2]=y', 'lem[0]=ole', 'pos[0]=_V_', 'lem[-1]=vabariik', 'pos[-1]=_S_', 'post[-1]=riik', 'lem[-2]=eesti', 'prop[-2]=y', 'pos[-2]=_H_', 'lem[1]=riik', 'pos[1]=_S_', 'lem[2]=põhjaeuroobas', 'prop[2]=y', 'pos[2]=_H_', 'post[2]=roobas', 'gaz[-1]=loc', 'gaz[-2]=loc', 'lem[0]|lem[-1]=ole|vabariik', 'lem[0]|lem[1]=ole|riik', 'pos[0]|pos[-1]=_V_|_S_', 'pos[0]|pos[1]=_V_|_S_'], ['p3[0]=rii', 'p4[0]=riik', 's3[0]=iik', 's4[0]=riik', 'aan[0]=y', 'iu[-2]=y', 'cu[-2]=y', 'iu[1]=y', 'cu[1]=y', 'cs[1]=y', 'cds[1]=y', 'cs[2]=y', 'cdt[2]=y', 'lem[0]=riik', 'pos[0]=_S_', 'case[0]=n', 'lem[-1]=ole', 'pos[-1]=_V_', 'lem[-2]=vabariik', 'pos[-2]=_S_', 'post[-2]=riik', 'lem[1]=põhjaeuroobas', 'prop[1]=y', 'pos[1]=_H_', 'post[1]=roobas', 'lem[2]=.', 'pos[2]=_Z_', 'pun[2]=y', 'gaz[-2]=loc', 'lem[0]|lem[-1]=riik|ole', 'lem[0]|lem[1]=riik|põhjaeuroobas', 'pos[0]|pos[-1]=_S_|_V_', 'pos[0]|pos[1]=_S_|_H_'], ['iu[0]=y', 'p3[0]=põh', 'p4[0]=põhj', 's3[0]=bas', 's4[0]=obas', 'cu[0]=y', 'cs[0]=y', 'cds[0]=y', 'cs[1]=y', 'cdt[1]=y', 'iu[2]=y', 'cu[2]=y', 'lem[0]=põhjaeuroobas', 'pos[0]=_H_', 'pref[0]=põhja', 'post[0]=roobas', 'case[0]=in', 'end[0]=s', 'lem[-1]=riik', 'pos[-1]=_S_', 'lem[-2]=ole', 'pos[-2]=_V_', 'lem[1]=.', 'pos[1]=_Z_', 'pun[1]=y', 'lem[2]=eesti', 'prop[2]=y', 'pos[2]=_H_', 'iuoc[0]=y', 'gaz[2]=loc', 'lem[0]|lem[-1]=põhjaeuroobas|riik', 'lem[0]|lem[1]=põhjaeuroobas|.', 'pos[0]|pos[-1]=_H_|_S_', 'pos[0]|pos[1]=_H_|_Z_'], ['ao[0]=y', 'cs[0]=y', 'cdt[0]=y', 'iu[-1]=y', 'cu[-1]=y', 'cs[-1]=y', 'cds[-1]=y', 'iu[1]=y', 'cu[1]=y', 'lem[0]=.', 'pos[0]=_Z_', 'pun[0]=y', 'lem[-1]=põhjaeuroobas', 'prop[-1]=y', 'pos[-1]=_H_', 'post[-1]=roobas', 'lem[-2]=riik', 'pos[-2]=_S_', 'lem[1]=eesti', 'prop[1]=y', 'pos[1]=_H_', 'lem[2]=piirne', 'pos[2]=_V_', 'gaz[1]=loc', 'lem[0]|lem[-1]=.|põhjaeuroobas', 'lem[0]|lem[1]=.|eesti', 'pos[0]|pos[-1]=_Z_|_H_', 'pos[0]|pos[1]=_Z_|_H_'], ['iu[0]=y', 'p3[0]=ees', 'p4[0]=eest', 's3[0]=sti', 's4[0]=esti', 'aan[0]=y', 'cu[0]=y', 'cs[-1]=y', 'cdt[-1]=y', 'iu[-2]=y', 'cu[-2]=y', 'cs[-2]=y', 'cds[-2]=y', 'lem[0]=eesti', 'pos[0]=_H_', 'case[0]=n', 'lem[-1]=.', 'pos[-1]=_Z_', 'pun[-1]=y', 'lem[-2]=põhjaeuroobas', 'prop[-2]=y', 'pos[-2]=_H_', 'post[-2]=roobas', 'lem[1]=piirne', 'pos[1]=_V_', 'lem[2]=põhi', 'pos[2]=_S_', 'iuoc[0]=y', 'ngaz[0]=loc', 'gaz[0]=loc', 'lem[0]|lem[-1]=eesti|.', 'lem[0]|lem[1]=eesti|piirne', 'pos[0]|pos[-1]=_H_|_Z_', 'pos[0]|pos[1]=_H_|_V_'], ['p3[0]=pii', 'p4[0]=piir', 's3[0]=rne', 's4[0]=irne', 'aan[0]=y', 'iu[-1]=y', 'cu[-1]=y', 'cs[-2]=y', 'cdt[-2]=y', 'lem[0]=piirne', 'pos[0]=_V_', 'end[0]=b', 'lem[-1]=eesti', 'prop[-1]=y', 'pos[-1]=_H_', 'lem[-2]=.', 'pos[-2]=_Z_', 'pun[-2]=y', 'lem[1]=põhi', 'pos[1]=_S_', 'lem[2]=üle', 'pos[2]=_D_', 'gaz[-1]=loc', 'lem[0]|lem[-1]=piirne|eesti', 'lem[0]|lem[1]=piirne|põhi', 'pos[0]|pos[-1]=_V_|_H_', 'pos[0]|pos[1]=_V_|_S_'], ['p3[0]=põh', 'p4[0]=põhi', 's3[0]=õhi', 's4[0]=põhi', 'aan[0]=y', 'iu[-2]=y', 'cu[-2]=y', 'iu[2]=y', 'cu[2]=y', 'lem[0]=põhi', 'pos[0]=_S_', 'case[0]=in', 'end[0]=s', 'lem[-1]=piirne', 'pos[-1]=_V_', 'lem[-2]=eesti', 'prop[-2]=y', 'pos[-2]=_H_', 'lem[1]=üle', 'pos[1]=_D_', 'lem[2]=soome', 'prop[2]=y', 'pos[2]=_H_', 'gaz[-2]=loc', 'gaz[2]=loc', 'lem[0]|lem[-1]=põhi|piirne', 'lem[0]|lem[1]=põhi|üle', 'pos[0]|pos[-1]=_S_|_V_', 'pos[0]|pos[1]=_S_|_D_'], ['p3[0]=üle', 's3[0]=üle', 'aan[0]=y', 'iu[1]=y', 'cu[1]=y', 'lem[0]=üle', 'pos[0]=_D_', 'lem[-1]=põhi', 'pos[-1]=_S_', 'lem[-2]=piirne', 'pos[-2]=_V_', 'lem[1]=soome', 'prop[1]=y', 'pos[1]=_H_', 'lem[2]=laht', 'pos[2]=_S_', 'gaz[1]=loc', 'gaz[2]=loc', 'lem[0]|lem[-1]=üle|põhi', 'lem[0]|lem[1]=üle|soome', 'pos[0]|pos[-1]=_D_|_S_', 'pos[0]|pos[1]=_D_|_H_'], ['iu[0]=y', 'p3[0]=soo', 'p4[0]=soom', 's3[0]=ome', 's4[0]=oome', 'aan[0]=y', 'cu[0]=y', 'iu[2]=y', 'cu[2]=y', 'lem[0]=soome', 'pos[0]=_H_', 'case[0]=g', 'lem[-1]=üle', 'pos[-1]=_D_', 'lem[-2]=põhi', 'pos[-2]=_S_', 'lem[1]=laht', 'pos[1]=_S_', 'lem[2]=soome', 'prop[2]=y', 'pos[2]=_H_', 'iuoc[0]=y', 'nprop[0]=y', 'ngaz[0]=loc', 'pgaz[0]=loc', 'gaz[0]=loc', 'gaz[1]=loc', 'gaz[2]=loc', 'lem[0]|lem[-1]=soome|üle', 'lem[0]|lem[1]=soome|laht', 'pos[0]|pos[-1]=_H_|_D_', 'pos[0]|pos[1]=_H_|_S_', 'gaz[0]|gaz[1]=loc|loc'], ['p3[0]=lah', 'p4[0]=laht', 's3[0]=aht', 's4[0]=laht', 'aan[0]=y', 'iu[-1]=y', 'cu[-1]=y', 'iu[1]=y', 'cu[1]=y', 'iu[2]=y', 'cu[2]=y', 'lem[0]=laht', 'pos[0]=_S_', 'case[0]=g', 'lem[-1]=soome', 'prop[-1]=y', 'pos[-1]=_H_', 'lem[-2]=üle', 'pos[-2]=_D_', 'lem[1]=soome', 'prop[1]=y', 'pos[1]=_H_', 'lem[2]=vabariik', 'prop[2]=y', 'pos[2]=_H_', 'post[2]=riik', 'gaz[0]=loc', 'gaz[-1]=loc', 'gaz[1]=loc', 'gaz[2]=loc', 'lem[0]|lem[-1]=laht|soome', 'lem[0]|lem[1]=laht|soome', 'pos[0]|pos[-1]=_S_|_H_', 'pos[0]|pos[1]=_S_|_H_', 'gaz[0]|gaz[1]=loc|loc', 'gaz[-1]|gaz[0]=loc|loc'], ['iu[0]=y', 'p3[0]=soo', 'p4[0]=soom', 's3[0]=ome', 's4[0]=oome', 'aan[0]=y', 'cu[0]=y', 'iu[-2]=y', 'cu[-2]=y', 'iu[1]=y', 'cu[1]=y', 'cs[2]=y', 'cdt[2]=y', 'lem[0]=soome', 'pos[0]=_H_', 'case[0]=n', 'lem[-1]=laht', 'pos[-1]=_S_', 'lem[-2]=soome', 'prop[-2]=y', 'pos[-2]=_H_', 'lem[1]=vabariik', 'prop[1]=y', 'pos[1]=_H_', 'post[1]=riik', 'lem[2]=.', 'pos[2]=_Z_', 'pun[2]=y', 'iuoc[0]=y', 'nprop[0]=y', 'ngaz[0]=loc', 'pgaz[0]=loc', 'gaz[0]=loc', 'gaz[-1]=loc', 'gaz[-2]=loc', 'gaz[1]=loc', 'lem[0]|lem[-1]=soome|laht', 'lem[0]|lem[1]=soome|vabariik', 'pos[0]|pos[-1]=_H_|_S_', 'pos[0]|pos[1]=_H_|_H_', 'iu[0]|iu[1]=y|y', 'gaz[0]|gaz[1]=loc|loc', 'gaz[-1]|gaz[0]=loc|loc'], ['iu[0]=y', 'p3[0]=vab', 'p4[0]=vaba', 's3[0]=iik', 's4[0]=riik', 'aan[0]=y', 'cu[0]=y', 'iu[-1]=y', 'cu[-1]=y', 'cs[1]=y', 'cdt[1]=y', 'lem[0]=vabariik', 'pos[0]=_H_', 'pref[0]=vaba', 'post[0]=riik', 'case[0]=kom', 'end[0]=ga', 'lem[-1]=soome', 'prop[-1]=y', 'pos[-1]=_H_', 'lem[-2]=laht', 'pos[-2]=_S_', 'lem[1]=.', 'pos[1]=_Z_', 'pun[1]=y', 'iuoc[0]=y', 'pprop[0]=y', 'pgaz[0]=loc', 'gaz[0]=loc', 'gaz[-1]=loc', 'gaz[-2]=loc', 'lem[0]|lem[-1]=vabariik|soome', 'lem[0]|lem[1]=vabariik|.', 'pos[0]|pos[-1]=_H_|_H_', 'pos[0]|pos[1]=_H_|_Z_', 'iu[0]|iu[-1]=y|y', 'gaz[-1]|gaz[0]=loc|loc'], ['ao[0]=y', 'cs[0]=y', 'cdt[0]=y', 'iu[-1]=y', 'cu[-1]=y', 'iu[-2]=y', 'cu[-2]=y', 'lem[0]=.', 'pos[0]=_Z_', 'pun[0]=y', 'lem[-1]=vabariik', 'prop[-1]=y', 'pos[-1]=_H_', 'post[-1]=riik', 'lem[-2]=soome', 'prop[-2]=y', 'pos[-2]=_H_', 'gaz[-1]=loc', 'gaz[-2]=loc', 'lem[0]|lem[-1]=.|vabariik', 'pos[0]|pos[-1]=_Z_|_H_']]

In [102]:
text.ner

Layer(name='ner', attributes=('nertag', 'name'), spans=SL[EnvelopingSpan(['Eesti', 'Vabariik'], [{'nertag': 'LOC', 'name': 'po'}]),
EnvelopingSpan(['Põhja-Euroopas'], [{'nertag': 'LOC', 'name': 'po'}]),
EnvelopingSpan(['Eesti'], [{'nertag': 'LOC', 'name': 'po'}]),
EnvelopingSpan(['Soome', 'lahe', 'Soome', 'Vabariigiga'], [{'nertag': 'LOC', 'name': 'po'}])])

In [20]:
nertagger.tag(text)

[['B-LOC'], ['I-LOC'], ['O'], ['O'], ['B-LOC'], ['O'], ['B-LOC'], ['O'], ['O'], ['O'], ['B-LOC'], ['I-LOC'], ['B-LOC'], ['B-LOC'], ['O']]


Text(text='Eesti Vabariik on riik Põhja-Euroopas.\n    Eesti piirneb põhjas üle Soome lahe Soome Vabariigiga.\n    ')

In [21]:
text.ner

Layer(name='ner', attributes=('nertag', 'name'), spans=SL[EnvelopingSpan(['Eesti', 'Vabariik'], [{'nertag': 'LOC', 'name': 'po'}]),
EnvelopingSpan(['Põhja-Euroopas'], [{'nertag': 'LOC', 'name': 'po'}]),
EnvelopingSpan(['Eesti'], [{'nertag': 'LOC', 'name': 'po'}]),
EnvelopingSpan(['Soome', 'lahe'], [{'nertag': 'LOC', 'name': 'po'}]),
EnvelopingSpan(['Soome'], [{'nertag': 'LOC', 'name': 'po'}]),
EnvelopingSpan(['Vabariigiga'], [{'nertag': 'LOC', 'name': 'po'}])])

In [33]:
text.ner_features.spans[0].annotations[0]

Annotation('Eesti', {'value': 'eesti', 'name': 'lem'})

In [34]:
annotation = text.ner_features.spans[0].annotations[0]

In [35]:
annotation['lem'] = 'eesti'

In [38]:
annotation.__dict__

{'lem': 'eesti', 'name': 'lem', 'value': 'eesti'}

tee kiht väga paljude atribuutidega

In [7]:
text.ner_features

Layer(name='ner_features', attributes=('value', 'name'), spans=SL[Span('Eesti', [{'value': 'eesti', 'name': 'lem'}, {'value': '_H_', 'name': 'pos'}, {'value': 'y', 'name': 'prop'}, {'value': None, 'name': 'pref'}, {'value': None, 'name': 'post'}, {'value': 'g', 'name': 'case'}, {'value': None, 'name': 'end'}, {'value': None, 'name': 'pun'}, {'value': 'Eesti', 'name': 'w'}, {'value': 'eesti', 'name': 'wl'}, {'value': 'ULLLL', 'name': 'shape'}, {'value': 'UL', 'name': 'shaped'}, {'value': 'e', 'name': 'p1'}, {'value': 'ee', 'name': 'p2'}, {'value': 'ees', 'name': 'p3'}, {'value': 'eest', 'name': 'p4'}, {'value': 'i', 'name': 's1'}, {'value': 'ti', 'name': 's2'}, {'value': 'sti', 'name': 's3'}, {'value': 'esti', 'name': 's4'}, {'value': None, 'name': '2d'}, {'value': None, 'name': '4d'}, {'value': None, 'name': 'd&-'}, {'value': None, 'name': 'd&/'}, {'value': None, 'name': 'd&,'}, {'value': None, 'name': 'd&.'}, {'value': None, 'name': 'up'}, {'value': 'y', 'name': 'iu'}, {'value': None, 'name': 'au'}, {'value': None, 'name': 'al'}, {'value': None, 'name': 'ad'}, {'value': None, 'name': 'ao'}, {'value': 'y', 'name': 'aan'}, {'value': 'y', 'name': 'cu'}, {'value': 'y', 'name': 'cl'}, {'value': 'y', 'name': 'ca'}, {'value': None, 'name': 'cd'}, {'value': None, 'name': 'cp'}, {'value': None, 'name': 'cds'}, {'value': None, 'name': 'cdt'}, {'value': None, 'name': 'cs'}, {'value': None, 'name': 'bdash'}, {'value': None, 'name': 'adash'}, {'value': None, 'name': 'bdot'}, {'value': None, 'name': 'adot'}, {'value': '5', 'name': 'len'}, {'value': 'y', 'name': 'fsnt'}, {'value': {'loc'}, 'name': 'gaz'}, {'value': Span('Vabariik', [{'value': 'vabariik', 'name': 'lem'}, {'value': '_S_', 'name': 'pos'}, {'value': None, 'name': 'prop'}, {'value': 'vaba', 'name': 'pref'}, {'value': 'riik', 'name': 'post'}, {'value': 'n', 'name': 'case'}, {'value': None, 'name': 'end'}, {'value': None, 'name': 'pun'}, {'value': 'Vabariik', 'name': 'w'}, {'value': 'vabariik', 'name': 'wl'}, {'value': 'ULLLLLLL', 'name': 'shape'}, {'value': 'UL', 'name': 'shaped'}, {'value': 'v', 'name': 'p1'}, {'value': 'va', 'name': 'p2'}, {'value': 'vab', 'name': 'p3'}, {'value': 'vaba', 'name': 'p4'}, {'value': 'k', 'name': 's1'}, {'value': 'ik', 'name': 's2'}, {'value': 'iik', 'name': 's3'}, {'value': 'riik', 'name': 's4'}, {'value': None, 'name': '2d'}, {'value': None, 'name': '4d'}, {'value': None, 'name': 'd&-'}, {'value': None, 'name': 'd&/'}, {'value': None, 'name': 'd&,'}, {'value': None, 'name': 'd&.'}, {'value': None, 'name': 'up'}, {'value': 'y', 'name': 'iu'}, {'value': None, 'name': 'au'}, {'value': None, 'name': 'al'}, {'value': None, 'name': 'ad'}, {'value': None, 'name': 'ao'}, {'value': 'y', 'name': 'aan'}, {'value': 'y', 'name': 'cu'}, {'value': 'y', 'name': 'cl'}, {'value': 'y', 'name': 'ca'}, {'value': None, 'name': 'cd'}, {'value': None, 'name': 'cp'}, {'value': None, 'name': 'cds'}, {'value': None, 'name': 'cdt'}, {'value': None, 'name': 'cs'}, {'value': None, 'name': 'bdash'}, {'value': None, 'name': 'adash'}, {'value': None, 'name': 'bdot'}, {'value': None, 'name': 'adot'}, {'value': '8', 'name': 'len'}, {'value': {'loc'}, 'name': 'gaz'}, {'value': ..., 'name': 'prew'}, {'value': Span('on', [{'value': 'ole', 'name': 'lem'}, {'value': '_V_', 'name': 'pos'}, {'value': None, 'name': 'prop'}, {'value': None, 'name': 'pref'}, {'value': None, 'name': 'post'}, {'value': None, 'name': 'case'}, {'value': None, 'name': 'end'}, {'value': None, 'name': 'pun'}, {'value': 'on', 'name': 'w'}, {'value': 'on', 'name': 'wl'}, {'value': 'LL', 'name': 'shape'}, {'value': 'L', 'name': 'shaped'}, {'value': 'o', 'name': 'p1'}, {'value': 'ol', 'name': 'p2'}, {'value': 'ole', 'name': 'p3'}, {'value': None, 'name': 'p4'}, {'value': 'e', 'name': 's1'}, {'value': 'le', 'name': 's2'}, {'value': 'ole', 'name': 's3'}, {'value': None, 'name': 's4'}, {'value': None, 'name': '2d'}, {'value': None, 'name': '4d'}, {'value': None, 'name': 'd&-'}, {'value': None, 'name': 'd&/'}, {'value': N

In [40]:
"suur kala"[2:-2]

'ur ka'

In [15]:
text.ner_features

Layer(name='ner_features', attributes=('value', 'name'), spans=SL[Span('Eesti', [{'value': 'eesti', 'name': 'lem'}, {'value': '_H_', 'name': 'pos'}, {'value': 'y', 'name': 'prop'}, {'value': None, 'name': 'pref'}, {'value': None, 'name': 'post'}, {'value': 'g', 'name': 'case'}, {'value': None, 'name': 'end'}, {'value': None, 'name': 'pun'}, {'value': 'Eesti', 'name': 'w'}, {'value': 'eesti', 'name': 'wl'}, {'value': 'ULLLL', 'name': 'shape'}, {'value': 'UL', 'name': 'shaped'}, {'value': 'e', 'name': 'p1'}, {'value': 'ee', 'name': 'p2'}, {'value': 'ees', 'name': 'p3'}, {'value': 'eest', 'name': 'p4'}, {'value': 'i', 'name': 's1'}, {'value': 'ti', 'name': 's2'}, {'value': 'sti', 'name': 's3'}, {'value': 'esti', 'name': 's4'}, {'value': None, 'name': '2d'}, {'value': None, 'name': '4d'}, {'value': None, 'name': 'd&-'}, {'value': None, 'name': 'd&/'}, {'value': None, 'name': 'd&,'}, {'value': None, 'name': 'd&.'}, {'value': None, 'name': 'up'}, {'value': 'y', 'name': 'iu'}, {'value': None, 'name': 'au'}, {'value': None, 'name': 'al'}, {'value': None, 'name': 'ad'}, {'value': None, 'name': 'ao'}, {'value': 'y', 'name': 'aan'}, {'value': 'y', 'name': 'cu'}, {'value': 'y', 'name': 'cl'}, {'value': 'y', 'name': 'ca'}, {'value': None, 'name': 'cd'}, {'value': None, 'name': 'cp'}, {'value': None, 'name': 'cds'}, {'value': None, 'name': 'cdt'}, {'value': None, 'name': 'cs'}, {'value': None, 'name': 'bdash'}, {'value': None, 'name': 'adash'}, {'value': None, 'name': 'bdot'}, {'value': None, 'name': 'adot'}, {'value': '5', 'name': 'len'}, {'value': 'y', 'name': 'fsnt'}, {'value': {'loc'}, 'name': 'gaz'}, {'value': Span('Vabariik', [{'value': 'vabariik', 'name': 'lem'}, {'value': '_S_', 'name': 'pos'}, {'value': None, 'name': 'prop'}, {'value': 'vaba', 'name': 'pref'}, {'value': 'riik', 'name': 'post'}, {'value': 'g', 'name': 'case'}, {'value': None, 'name': 'end'}, {'value': None, 'name': 'pun'}, {'value': 'Vabariik', 'name': 'w'}, {'value': 'vabariik', 'name': 'wl'}, {'value': 'ULLLLLLL', 'name': 'shape'}, {'value': 'UL', 'name': 'shaped'}, {'value': 'v', 'name': 'p1'}, {'value': 'va', 'name': 'p2'}, {'value': 'vab', 'name': 'p3'}, {'value': 'vaba', 'name': 'p4'}, {'value': 'k', 'name': 's1'}, {'value': 'ik', 'name': 's2'}, {'value': 'iik', 'name': 's3'}, {'value': 'riik', 'name': 's4'}, {'value': None, 'name': '2d'}, {'value': None, 'name': '4d'}, {'value': None, 'name': 'd&-'}, {'value': None, 'name': 'd&/'}, {'value': None, 'name': 'd&,'}, {'value': None, 'name': 'd&.'}, {'value': None, 'name': 'up'}, {'value': 'y', 'name': 'iu'}, {'value': None, 'name': 'au'}, {'value': None, 'name': 'al'}, {'value': None, 'name': 'ad'}, {'value': None, 'name': 'ao'}, {'value': 'y', 'name': 'aan'}, {'value': 'y', 'name': 'cu'}, {'value': 'y', 'name': 'cl'}, {'value': 'y', 'name': 'ca'}, {'value': None, 'name': 'cd'}, {'value': None, 'name': 'cp'}, {'value': None, 'name': 'cds'}, {'value': None, 'name': 'cdt'}, {'value': None, 'name': 'cs'}, {'value': None, 'name': 'bdash'}, {'value': None, 'name': 'adash'}, {'value': None, 'name': 'bdot'}, {'value': None, 'name': 'adot'}, {'value': '8', 'name': 'len'}, {'value': {'loc'}, 'name': 'gaz'}, {'value': ..., 'name': 'prew'}, {'value': Span('on', [{'value': 'ole', 'name': 'lem'}, {'value': '_V_', 'name': 'pos'}, {'value': None, 'name': 'prop'}, {'value': None, 'name': 'pref'}, {'value': None, 'name': 'post'}, {'value': None, 'name': 'case'}, {'value': None, 'name': 'end'}, {'value': None, 'name': 'pun'}, {'value': 'on', 'name': 'w'}, {'value': 'on', 'name': 'wl'}, {'value': 'LL', 'name': 'shape'}, {'value': 'L', 'name': 'shaped'}, {'value': 'o', 'name': 'p1'}, {'value': 'ol', 'name': 'p2'}, {'value': 'ole', 'name': 'p3'}, {'value': None, 'name': 'p4'}, {'value': 'e', 'name': 's1'}, {'value': 'le', 'name': 's2'}, {'value': 'ole', 'name': 's3'}, {'value': None, 'name': 's4'}, {'value': None, 'name': '2d'}, {'value': None, 'name': '4d'}, {'value': None, 'name': 'd&-'}, {'value': None, 'name': 'd&/'}, {'value': N

In [73]:
text.ner

Layer(name='ner', attributes=('nertag', 'name'), spans=SL[EnvelopingSpan(['Eesti', 'Vabariik', 'on'], [{'nertag': '', 'name': 'po'}])])

### Import tagger

In [24]:
from estnltk.taggers.estner import CrfsuiteTagger
from estnltk.core import DEFAULT_PY3_NER_MODEL_DIR
from estnltk.taggers.estner.refac.ner import ModelStorageUtil

model_dir = DEFAULT_PY3_NER_MODEL_DIR
modelUtil = ModelStorageUtil(model_dir)
nersettings = modelUtil.load_settings()
tagger = CrfsuiteTagger(settings=nersettings,
                                     model_filename=modelUtil.model_filename)

In [4]:
import pycrfsuite
taggija = pycrfsuite.Tagger()
taggija.open(modelUtil.model_filename)

In [7]:
inputs = [['iu[0]=y', 'fsnt[0]=y', 'p3[0]=ees', 'p4[0]=eest', 's3[0]=sti', 's4[0]=esti', 'aan[0]=y', 'cu[0]=y', 'iu[1]=y', 'cu[1]=y', 'lem[0]=eesti', 'pos[0]=_H_', 'case[0]=g', 'lem[1]=vabariik', 'pos[1]=_S_', 'post[1]=riik', 'lem[2]=ole', 'pos[2]=_V_', 'iuoc[0]=y', 'ngaz[0]=loc', 'gaz[0]=loc', 'gaz[1]=loc', 'iu[0]|fsnt[0]=y|y', 'lem[0]|lem[1]=eesti|vabariik', 'pos[0]|pos[1]=_H_|_S_', 'iu[0]|iu[1]=y|y', 'gaz[0]|gaz[1]=loc|loc'], ['iu[0]=y', 'p3[0]=vab', 'p4[0]=vaba', 's3[0]=iik', 's4[0]=riik', 'aan[0]=y', 'cu[0]=y', 'iu[-1]=y', 'cu[-1]=y', 'fsnt[-1]=y', 'lem[0]=vabariik', 'pos[0]=_S_', 'pref[0]=vaba', 'post[0]=riik', 'case[0]=n', 'lem[-1]=eesti', 'prop[-1]=y', 'pos[-1]=_H_', 'lem[1]=ole', 'pos[1]=_V_', 'lem[2]=riik', 'pos[2]=_S_', 'iuoc[0]=y', 'pprop[0]=y', 'pgaz[0]=loc', 'gaz[0]=loc', 'gaz[-1]=loc', 'lem[0]|lem[-1]=vabariik|eesti', 'lem[0]|lem[1]=vabariik|ole', 'pos[0]|pos[-1]=_S_|_H_', 'pos[0]|pos[1]=_S_|_V_', 'iu[0]|iu[-1]=y|y', 'gaz[-1]|gaz[0]=loc|loc'], ['p3[0]=ole', 's3[0]=ole', 'aan[0]=y', 'iu[-1]=y', 'cu[-1]=y', 'iu[-2]=y', 'cu[-2]=y', 'fsnt[-2]=y', 'iu[2]=y', 'cu[2]=y', 'cs[2]=y', 'cds[2]=y', 'lem[0]=ole', 'pos[0]=_V_', 'lem[-1]=vabariik', 'pos[-1]=_S_', 'post[-1]=riik', 'lem[-2]=eesti', 'prop[-2]=y', 'pos[-2]=_H_', 'lem[1]=riik', 'pos[1]=_S_', 'lem[2]=põhjaeuroobas', 'prop[2]=y', 'pos[2]=_H_', 'post[2]=roobas', 'gaz[-1]=loc', 'gaz[-2]=loc', 'lem[0]|lem[-1]=ole|vabariik', 'lem[0]|lem[1]=ole|riik', 'pos[0]|pos[-1]=_V_|_S_', 'pos[0]|pos[1]=_V_|_S_'], ['p3[0]=rii', 'p4[0]=riik', 's3[0]=iik', 's4[0]=riik', 'aan[0]=y', 'iu[-2]=y', 'cu[-2]=y', 'iu[1]=y', 'cu[1]=y', 'cs[1]=y', 'cds[1]=y', 'cs[2]=y', 'cdt[2]=y', 'lem[0]=riik', 'pos[0]=_S_', 'case[0]=n', 'lem[-1]=ole', 'pos[-1]=_V_', 'lem[-2]=vabariik', 'pos[-2]=_S_', 'post[-2]=riik', 'lem[1]=põhjaeuroobas', 'prop[1]=y', 'pos[1]=_H_', 'post[1]=roobas', 'lem[2]=.', 'pos[2]=_Z_', 'pun[2]=y', 'gaz[-2]=loc', 'lem[0]|lem[-1]=riik|ole', 'lem[0]|lem[1]=riik|põhjaeuroobas', 'pos[0]|pos[-1]=_S_|_V_', 'pos[0]|pos[1]=_S_|_H_'], ['iu[0]=y', 'p3[0]=põh', 'p4[0]=põhj', 's3[0]=bas', 's4[0]=obas', 'cu[0]=y', 'cs[0]=y', 'cds[0]=y', 'cs[1]=y', 'cdt[1]=y', 'iu[2]=y', 'cu[2]=y', 'lem[0]=põhjaeuroobas', 'pos[0]=_H_', 'pref[0]=põhja', 'post[0]=roobas', 'case[0]=in', 'end[0]=s', 'lem[-1]=riik', 'pos[-1]=_S_', 'lem[-2]=ole', 'pos[-2]=_V_', 'lem[1]=.', 'pos[1]=_Z_', 'pun[1]=y', 'lem[2]=eesti', 'prop[2]=y', 'pos[2]=_H_', 'iuoc[0]=y', 'gaz[2]=loc', 'lem[0]|lem[-1]=põhjaeuroobas|riik', 'lem[0]|lem[1]=põhjaeuroobas|.', 'pos[0]|pos[-1]=_H_|_S_', 'pos[0]|pos[1]=_H_|_Z_'], ['ao[0]=y', 'cs[0]=y', 'cdt[0]=y', 'iu[-1]=y', 'cu[-1]=y', 'cs[-1]=y', 'cds[-1]=y', 'iu[1]=y', 'cu[1]=y', 'lem[0]=.', 'pos[0]=_Z_', 'pun[0]=y', 'lem[-1]=põhjaeuroobas', 'prop[-1]=y', 'pos[-1]=_H_', 'post[-1]=roobas', 'lem[-2]=riik', 'pos[-2]=_S_', 'lem[1]=eesti', 'prop[1]=y', 'pos[1]=_H_', 'lem[2]=piirne', 'pos[2]=_V_', 'gaz[1]=loc', 'lem[0]|lem[-1]=.|põhjaeuroobas', 'lem[0]|lem[1]=.|eesti', 'pos[0]|pos[-1]=_Z_|_H_', 'pos[0]|pos[1]=_Z_|_H_'], ['iu[0]=y', 'p3[0]=ees', 'p4[0]=eest', 's3[0]=sti', 's4[0]=esti', 'aan[0]=y', 'cu[0]=y', 'cs[-1]=y', 'cdt[-1]=y', 'iu[-2]=y', 'cu[-2]=y', 'cs[-2]=y', 'cds[-2]=y', 'lem[0]=eesti', 'pos[0]=_H_', 'case[0]=n', 'lem[-1]=.', 'pos[-1]=_Z_', 'pun[-1]=y', 'lem[-2]=põhjaeuroobas', 'prop[-2]=y', 'pos[-2]=_H_', 'post[-2]=roobas', 'lem[1]=piirne', 'pos[1]=_V_', 'lem[2]=põhi', 'pos[2]=_S_', 'iuoc[0]=y', 'ngaz[0]=loc', 'gaz[0]=loc', 'lem[0]|lem[-1]=eesti|.', 'lem[0]|lem[1]=eesti|piirne', 'pos[0]|pos[-1]=_H_|_Z_', 'pos[0]|pos[1]=_H_|_V_'], ['p3[0]=pii', 'p4[0]=piir', 's3[0]=rne', 's4[0]=irne', 'aan[0]=y', 'iu[-1]=y', 'cu[-1]=y', 'cs[-2]=y', 'cdt[-2]=y', 'lem[0]=piirne', 'pos[0]=_V_', 'end[0]=b', 'lem[-1]=eesti', 'prop[-1]=y', 'pos[-1]=_H_', 'lem[-2]=.', 'pos[-2]=_Z_', 'pun[-2]=y', 'lem[1]=põhi', 'pos[1]=_S_', 'lem[2]=üle', 'pos[2]=_D_', 'gaz[-1]=loc', 'lem[0]|lem[-1]=piirne|eesti', 'lem[0]|lem[1]=piirne|põhi', 'pos[0]|pos[-1]=_V_|_H_', 'pos[0]|pos[1]=_V_|_S_'], ['p3[0]=põh', 'p4[0]=põhi', 's3[0]=õhi', 's4[0]=põhi', 'aan[0]=y', 'iu[-2]=y', 'cu[-2]=y', 'iu[2]=y', 'cu[2]=y', 'lem[0]=põhi', 'pos[0]=_S_', 'case[0]=in', 'end[0]=s', 'lem[-1]=piirne', 'pos[-1]=_V_', 'lem[-2]=eesti', 'prop[-2]=y', 'pos[-2]=_H_', 'lem[1]=üle', 'pos[1]=_D_', 'lem[2]=soome', 'prop[2]=y', 'pos[2]=_H_', 'gaz[-2]=loc', 'gaz[2]=loc', 'lem[0]|lem[-1]=põhi|piirne', 'lem[0]|lem[1]=põhi|üle', 'pos[0]|pos[-1]=_S_|_V_', 'pos[0]|pos[1]=_S_|_D_'], ['p3[0]=üle', 's3[0]=üle', 'aan[0]=y', 'iu[1]=y', 'cu[1]=y', 'lem[0]=üle', 'pos[0]=_D_', 'lem[-1]=põhi', 'pos[-1]=_S_', 'lem[-2]=piirne', 'pos[-2]=_V_', 'lem[1]=soome', 'prop[1]=y', 'pos[1]=_H_', 'lem[2]=laht', 'pos[2]=_S_', 'gaz[1]=loc', 'gaz[2]=loc', 'lem[0]|lem[-1]=üle|põhi', 'lem[0]|lem[1]=üle|soome', 'pos[0]|pos[-1]=_D_|_S_', 'pos[0]|pos[1]=_D_|_H_'], ['iu[0]=y', 'p3[0]=soo', 'p4[0]=soom', 's3[0]=ome', 's4[0]=oome', 'aan[0]=y', 'cu[0]=y', 'iu[2]=y', 'cu[2]=y', 'lem[0]=soome', 'pos[0]=_H_', 'case[0]=g', 'lem[-1]=üle', 'pos[-1]=_D_', 'lem[-2]=põhi', 'pos[-2]=_S_', 'lem[1]=laht', 'pos[1]=_S_', 'lem[2]=soome', 'prop[2]=y', 'pos[2]=_H_', 'iuoc[0]=y', 'nprop[0]=y', 'ngaz[0]=loc', 'pgaz[0]=loc', 'gaz[0]=loc', 'gaz[1]=loc', 'gaz[2]=loc', 'lem[0]|lem[-1]=soome|üle', 'lem[0]|lem[1]=soome|laht', 'pos[0]|pos[-1]=_H_|_D_', 'pos[0]|pos[1]=_H_|_S_', 'gaz[0]|gaz[1]=loc|loc'], ['p3[0]=lah', 'p4[0]=laht', 's3[0]=aht', 's4[0]=laht', 'aan[0]=y', 'iu[-1]=y', 'cu[-1]=y', 'iu[1]=y', 'cu[1]=y', 'iu[2]=y', 'cu[2]=y', 'lem[0]=laht', 'pos[0]=_S_', 'case[0]=g', 'lem[-1]=soome', 'prop[-1]=y', 'pos[-1]=_H_', 'lem[-2]=üle', 'pos[-2]=_D_', 'lem[1]=soome', 'prop[1]=y', 'pos[1]=_H_', 'lem[2]=vabariik', 'prop[2]=y', 'pos[2]=_H_', 'post[2]=riik', 'gaz[0]=loc', 'gaz[-1]=loc', 'gaz[1]=loc', 'gaz[2]=loc', 'lem[0]|lem[-1]=laht|soome', 'lem[0]|lem[1]=laht|soome', 'pos[0]|pos[-1]=_S_|_H_', 'pos[0]|pos[1]=_S_|_H_', 'gaz[0]|gaz[1]=loc|loc', 'gaz[-1]|gaz[0]=loc|loc'], ['iu[0]=y', 'p3[0]=soo', 'p4[0]=soom', 's3[0]=ome', 's4[0]=oome', 'aan[0]=y', 'cu[0]=y', 'iu[-2]=y', 'cu[-2]=y', 'iu[1]=y', 'cu[1]=y', 'cs[2]=y', 'cdt[2]=y', 'lem[0]=soome', 'pos[0]=_H_', 'case[0]=n', 'lem[-1]=laht', 'pos[-1]=_S_', 'lem[-2]=soome', 'prop[-2]=y', 'pos[-2]=_H_', 'lem[1]=vabariik', 'prop[1]=y', 'pos[1]=_H_', 'post[1]=riik', 'lem[2]=.', 'pos[2]=_Z_', 'pun[2]=y', 'iuoc[0]=y', 'nprop[0]=y', 'ngaz[0]=loc', 'pgaz[0]=loc', 'gaz[0]=loc', 'gaz[-1]=loc', 'gaz[-2]=loc', 'gaz[1]=loc', 'lem[0]|lem[-1]=soome|laht', 'lem[0]|lem[1]=soome|vabariik', 'pos[0]|pos[-1]=_H_|_S_', 'pos[0]|pos[1]=_H_|_H_', 'iu[0]|iu[1]=y|y', 'gaz[0]|gaz[1]=loc|loc', 'gaz[-1]|gaz[0]=loc|loc'], ['iu[0]=y', 'p3[0]=vab', 'p4[0]=vaba', 's3[0]=iik', 's4[0]=riik', 'aan[0]=y', 'cu[0]=y', 'iu[-1]=y', 'cu[-1]=y', 'cs[1]=y', 'cdt[1]=y', 'lem[0]=vabariik', 'pos[0]=_H_', 'pref[0]=vaba', 'post[0]=riik', 'case[0]=kom', 'end[0]=ga', 'lem[-1]=soome', 'prop[-1]=y', 'pos[-1]=_H_', 'lem[-2]=laht', 'pos[-2]=_S_', 'lem[1]=.', 'pos[1]=_Z_', 'pun[1]=y', 'iuoc[0]=y', 'pprop[0]=y', 'pgaz[0]=loc', 'gaz[0]=loc', 'gaz[-1]=loc', 'gaz[-2]=loc', 'lem[0]|lem[-1]=vabariik|soome', 'lem[0]|lem[1]=vabariik|.', 'pos[0]|pos[-1]=_H_|_H_', 'pos[0]|pos[1]=_H_|_Z_', 'iu[0]|iu[-1]=y|y', 'gaz[-1]|gaz[0]=loc|loc'], ['ao[0]=y', 'cs[0]=y', 'cdt[0]=y', 'iu[-1]=y', 'cu[-1]=y', 'iu[-2]=y', 'cu[-2]=y', 'lem[0]=.', 'pos[0]=_Z_', 'pun[0]=y', 'lem[-1]=vabariik', 'prop[-1]=y', 'pos[-1]=_H_', 'post[-1]=riik', 'lem[-2]=soome', 'prop[-2]=y', 'pos[-2]=_H_', 'gaz[-1]=loc', 'gaz[-2]=loc', 'lem[0]|lem[-1]=.|vabariik', 'pos[0]|pos[-1]=_Z_|_H_']]

In [7]:
my_inputs = [['iu[0]=y', 'fsnt[0]=y', 'p3[0]=ees', 'p4[0]=eest', 's3[0]=sti', 's4[0]=esti', 'aan[0]=y', 'cu[0]=y', 'iu[1]=y', 'cu[1]=y', 'lem[0]=eesti', 'pos[0]=_H_', 'case[0]=g', 'lem[1]=vabariik', 'pos[1]=_S_', 'post[1]=riik', 'lem[2]=ole', 'pos[2]=_V_', 'gaz[0]=loc', 'gaz[1]=loc', 'iu[0]|fsnt[0]=y|y', 'lem[0]|lem[1]=eesti|vabariik', 'pos[0]|pos[1]=_H_|_S_', 'iu[0]|iu[1]=y|y', 'gaz[0]|gaz[1]=loc|loc'], ['iu[0]=y', 'p3[0]=vab', 'p4[0]=vaba', 's3[0]=iik', 's4[0]=riik', 'aan[0]=y', 'cu[0]=y', 'iu[-1]=y', 'cu[-1]=y', 'fsnt[-1]=y', 'lem[0]=vabariik', 'pos[0]=_S_', 'pref[0]=vaba', 'post[0]=riik', 'case[0]=g', 'lem[-1]=eesti', 'prop[-1]=y', 'pos[-1]=_H_', 'lem[1]=ole', 'pos[1]=_V_', 'lem[2]=riik', 'pos[2]=_S_', 'iuoc[0]=y', 'nprop[0]=y', 'pprop[0]=y', 'pgaz[0]=loc', 'gaz[0]=loc', 'gaz[-1]=loc', 'lem[0]|lem[-1]=vabariik|eesti', 'lem[0]|lem[1]=vabariik|ole', 'pos[0]|pos[-1]=_S_|_H_', 'pos[0]|pos[1]=_S_|_V_', 'iu[0]|iu[-1]=y|y', 'gaz[-1]|gaz[0]=loc|loc'], ['p3[0]=ole', 's3[0]=ole', 'aan[0]=y', 'iu[-1]=y', 'cu[-1]=y', 'iu[-2]=y', 'cu[-2]=y', 'fsnt[-2]=y', 'iu[2]=y', 'cu[2]=y', 'cs[2]=y', 'cds[2]=y', 'lem[0]=ole', 'pos[0]=_V_', 'lem[-1]=vabariik', 'pos[-1]=_S_', 'post[-1]=riik', 'lem[-2]=eesti', 'prop[-2]=y', 'pos[-2]=_H_', 'lem[1]=riik', 'pos[1]=_S_', 'lem[2]=põhjaeuroobas', 'prop[2]=y', 'pos[2]=_H_', 'post[2]=roobas', 'gaz[-1]=loc', 'gaz[-2]=loc', 'lem[0]|lem[-1]=ole|vabariik', 'lem[0]|lem[1]=ole|riik', 'pos[0]|pos[-1]=_V_|_S_', 'pos[0]|pos[1]=_V_|_S_'], ['p3[0]=rii', 'p4[0]=riik', 's3[0]=iik', 's4[0]=riik', 'aan[0]=y', 'iu[-2]=y', 'cu[-2]=y', 'iu[1]=y', 'cu[1]=y', 'cs[1]=y', 'cds[1]=y', 'cs[2]=y', 'cdt[2]=y', 'lem[0]=riik', 'pos[0]=_S_', 'case[0]=g', 'lem[-1]=ole', 'pos[-1]=_V_', 'lem[-2]=vabariik', 'pos[-2]=_S_', 'post[-2]=riik', 'lem[1]=põhjaeuroobas', 'prop[1]=y', 'pos[1]=_H_', 'post[1]=roobas', 'lem[2]=.', 'pos[2]=_Z_', 'pun[2]=y', 'gaz[-2]=loc', 'lem[0]|lem[-1]=riik|ole', 'lem[0]|lem[1]=riik|põhjaeuroobas', 'pos[0]|pos[-1]=_S_|_V_', 'pos[0]|pos[1]=_S_|_H_'], ['iu[0]=y', 'p3[0]=põh', 'p4[0]=põhj', 's3[0]=bas', 's4[0]=obas', 'cu[0]=y', 'cs[0]=y', 'cds[0]=y', 'cs[1]=y', 'cdt[1]=y', 'iu[2]=y', 'cu[2]=y', 'fsnt[2]=y', 'lem[0]=põhjaeuroobas', 'pos[0]=_H_', 'pref[0]=Põhja', 'post[0]=roobas', 'case[0]=g', 'end[0]=s', 'lem[-1]=riik', 'pos[-1]=_S_', 'lem[-2]=ole', 'pos[-2]=_V_', 'lem[1]=.', 'pos[1]=_Z_', 'pun[1]=y', 'lem[2]=eesti', 'prop[2]=y', 'pos[2]=_H_', 'iuoc[0]=y', 'nprop[0]=y', 'pprop[0]=y', 'gaz[2]=loc', 'lem[0]|lem[-1]=põhjaeuroobas|riik', 'lem[0]|lem[1]=põhjaeuroobas|.', 'pos[0]|pos[-1]=_H_|_S_', 'pos[0]|pos[1]=_H_|_Z_'], ['ao[0]=y', 'cs[0]=y', 'cdt[0]=y', 'iu[-1]=y', 'cu[-1]=y', 'cs[-1]=y', 'cds[-1]=y', 'iu[1]=y', 'cu[1]=y', 'fsnt[1]=y', 'lem[0]=.', 'pos[0]=_Z_', 'pun[0]=y', 'end[0]=', 'lem[-1]=põhjaeuroobas', 'prop[-1]=y', 'pos[-1]=_H_', 'post[-1]=roobas', 'lem[-2]=riik', 'pos[-2]=_S_', 'lem[1]=eesti', 'prop[1]=y', 'pos[1]=_H_', 'lem[2]=piirne', 'pos[2]=_V_', 'gaz[1]=loc', 'lem[0]|lem[-1]=.|põhjaeuroobas', 'lem[0]|lem[1]=.|eesti', 'pos[0]|pos[-1]=_Z_|_H_', 'pos[0]|pos[1]=_Z_|_H_'], ['iu[0]=y', 'fsnt[0]=y', 'p3[0]=ees', 'p4[0]=eest', 's3[0]=sti', 's4[0]=esti', 'aan[0]=y', 'cu[0]=y', 'cs[-1]=y', 'cdt[-1]=y', 'iu[-2]=y', 'cu[-2]=y', 'cs[-2]=y', 'cds[-2]=y', 'lem[0]=eesti', 'pos[0]=_H_', 'case[0]=g', 'lem[-1]=.', 'pos[-1]=_Z_', 'pun[-1]=y', 'lem[-2]=põhjaeuroobas', 'prop[-2]=y', 'pos[-2]=_H_', 'post[-2]=roobas', 'lem[1]=piirne', 'pos[1]=_V_', 'lem[2]=põhi', 'pos[2]=_S_', 'gaz[0]=loc', 'iu[0]|fsnt[0]=y|y', 'lem[0]|lem[-1]=eesti|.', 'lem[0]|lem[1]=eesti|piirne', 'pos[0]|pos[-1]=_H_|_Z_', 'pos[0]|pos[1]=_H_|_V_'], ['p3[0]=pii', 'p4[0]=piir', 's3[0]=rne', 's4[0]=irne', 'aan[0]=y', 'iu[-1]=y', 'cu[-1]=y', 'fsnt[-1]=y', 'cs[-2]=y', 'cdt[-2]=y', 'lem[0]=piirne', 'pos[0]=_V_', 'end[0]=b', 'lem[-1]=eesti', 'prop[-1]=y', 'pos[-1]=_H_', 'lem[-2]=.', 'pos[-2]=_Z_', 'pun[-2]=y', 'lem[1]=põhi', 'pos[1]=_S_', 'lem[2]=üle', 'pos[2]=_D_', 'gaz[-1]=loc', 'lem[0]|lem[-1]=piirne|eesti', 'lem[0]|lem[1]=piirne|põhi', 'pos[0]|pos[-1]=_V_|_H_', 'pos[0]|pos[1]=_V_|_S_'], ['p3[0]=põh', 'p4[0]=põhi', 's3[0]=õhi', 's4[0]=põhi', 'aan[0]=y', 'iu[-2]=y', 'cu[-2]=y', 'fsnt[-2]=y', 'iu[2]=y', 'cu[2]=y', 'lem[0]=põhi', 'pos[0]=_S_', 'case[0]=g', 'end[0]=s', 'lem[-1]=piirne', 'pos[-1]=_V_', 'lem[-2]=eesti', 'prop[-2]=y', 'pos[-2]=_H_', 'lem[1]=üle', 'pos[1]=_D_', 'lem[2]=soome', 'prop[2]=y', 'pos[2]=_H_', 'gaz[-2]=loc', 'gaz[2]=loc', 'lem[0]|lem[-1]=põhi|piirne', 'lem[0]|lem[1]=põhi|üle', 'pos[0]|pos[-1]=_S_|_V_', 'pos[0]|pos[1]=_S_|_D_'], ['p3[0]=üle', 's3[0]=üle', 'aan[0]=y', 'iu[1]=y', 'cu[1]=y', 'lem[0]=üle', 'pos[0]=_D_', 'lem[-1]=põhi', 'pos[-1]=_S_', 'lem[-2]=piirne', 'pos[-2]=_V_', 'lem[1]=soome', 'prop[1]=y', 'pos[1]=_H_', 'lem[2]=laht', 'pos[2]=_S_', 'gaz[1]=loc', 'gaz[2]=loc', 'lem[0]|lem[-1]=üle|põhi', 'lem[0]|lem[1]=üle|soome', 'pos[0]|pos[-1]=_D_|_S_', 'pos[0]|pos[1]=_D_|_H_'], ['iu[0]=y', 'p3[0]=soo', 'p4[0]=soom', 's3[0]=ome', 's4[0]=oome', 'aan[0]=y', 'cu[0]=y', 'iu[2]=y', 'cu[2]=y', 'lem[0]=soome', 'pos[0]=_H_', 'case[0]=g', 'lem[-1]=üle', 'pos[-1]=_D_', 'lem[-2]=põhi', 'pos[-2]=_S_', 'lem[1]=laht', 'pos[1]=_S_', 'lem[2]=soome', 'prop[2]=y', 'pos[2]=_H_', 'iuoc[0]=y', 'nprop[0]=y', 'pprop[0]=y', 'ngaz[0]=loc', 'pgaz[0]=loc', 'gaz[0]=loc', 'gaz[1]=loc', 'gaz[2]=loc', 'lem[0]|lem[-1]=soome|üle', 'lem[0]|lem[1]=soome|laht', 'pos[0]|pos[-1]=_H_|_D_', 'pos[0]|pos[1]=_H_|_S_', 'gaz[0]|gaz[1]=loc|loc'], ['p3[0]=lah', 'p4[0]=laht', 's3[0]=aht', 's4[0]=laht', 'aan[0]=y', 'iu[-1]=y', 'cu[-1]=y', 'iu[1]=y', 'cu[1]=y', 'iu[2]=y', 'cu[2]=y', 'lem[0]=laht', 'pos[0]=_S_', 'case[0]=g', 'lem[-1]=soome', 'prop[-1]=y', 'pos[-1]=_H_', 'lem[-2]=üle', 'pos[-2]=_D_', 'lem[1]=soome', 'prop[1]=y', 'pos[1]=_H_', 'lem[2]=vabariik', 'prop[2]=y', 'pos[2]=_H_', 'post[2]=riik', 'gaz[0]=loc', 'gaz[-1]=loc', 'gaz[1]=loc', 'gaz[2]=loc', 'lem[0]|lem[-1]=laht|soome', 'lem[0]|lem[1]=laht|soome', 'pos[0]|pos[-1]=_S_|_H_', 'pos[0]|pos[1]=_S_|_H_', 'gaz[0]|gaz[1]=loc|loc', 'gaz[-1]|gaz[0]=loc|loc'], ['iu[0]=y', 'p3[0]=soo', 'p4[0]=soom', 's3[0]=ome', 's4[0]=oome', 'aan[0]=y', 'cu[0]=y', 'iu[-2]=y', 'cu[-2]=y', 'iu[1]=y', 'cu[1]=y', 'cs[2]=y', 'cdt[2]=y', 'lem[0]=soome', 'pos[0]=_H_', 'case[0]=g', 'lem[-1]=laht', 'pos[-1]=_S_', 'lem[-2]=soome', 'prop[-2]=y', 'pos[-2]=_H_', 'lem[1]=vabariik', 'prop[1]=y', 'pos[1]=_H_', 'post[1]=riik', 'lem[2]=.', 'pos[2]=_Z_', 'pun[2]=y', 'iuoc[0]=y', 'nprop[0]=y', 'pprop[0]=y', 'ngaz[0]=loc', 'pgaz[0]=loc', 'gaz[0]=loc', 'gaz[-1]=loc', 'gaz[-2]=loc', 'gaz[1]=loc', 'lem[0]|lem[-1]=soome|laht', 'lem[0]|lem[1]=soome|vabariik', 'pos[0]|pos[-1]=_H_|_S_', 'pos[0]|pos[1]=_H_|_H_', 'iu[0]|iu[1]=y|y', 'gaz[0]|gaz[1]=loc|loc', 'gaz[-1]|gaz[0]=loc|loc'], ['iu[0]=y', 'p3[0]=vab', 'p4[0]=vaba', 's3[0]=iik', 's4[0]=riik', 'aan[0]=y', 'cu[0]=y', 'iu[-1]=y', 'cu[-1]=y', 'cs[1]=y', 'cdt[1]=y', 'lem[0]=vabariik', 'pos[0]=_H_', 'pref[0]=Vaba', 'post[0]=riik', 'case[0]=g', 'end[0]=ga', 'lem[-1]=soome', 'prop[-1]=y', 'pos[-1]=_H_', 'lem[-2]=laht', 'pos[-2]=_S_', 'lem[1]=.', 'pos[1]=_Z_', 'pun[1]=y', 'iuoc[0]=y', 'nprop[0]=y', 'pprop[0]=y', 'pgaz[0]=loc', 'gaz[0]=loc', 'gaz[-1]=loc', 'gaz[-2]=loc', 'lem[0]|lem[-1]=vabariik|soome', 'lem[0]|lem[1]=vabariik|.', 'pos[0]|pos[-1]=_H_|_H_', 'pos[0]|pos[1]=_H_|_Z_', 'iu[0]|iu[-1]=y|y', 'gaz[-1]|gaz[0]=loc|loc'], ['ao[0]=y', 'cs[0]=y', 'cdt[0]=y', 'iu[-1]=y', 'cu[-1]=y', 'iu[-2]=y', 'cu[-2]=y', 'lem[0]=.', 'pos[0]=_Z_', 'pun[0]=y', 'end[0]=', 'lem[-1]=vabariik', 'prop[-1]=y', 'pos[-1]=_H_', 'post[-1]=riik', 'lem[-2]=soome', 'prop[-2]=y', 'pos[-2]=_H_', 'gaz[-1]=loc', 'gaz[-2]=loc', 'lem[0]|lem[-1]=.|vabariik', 'pos[0]|pos[-1]=_Z_|_H_']]

In [13]:
my_inputs2 = [['iu[0]=y', 'fsnt[0]=y', 'p3[0]=ees', 'p4[0]=eest', 's3[0]=sti', 's4[0]=esti', 'aan[0]=y', 'cu[0]=y', 'iu[1]=y', 'cu[1]=y', 'lem[0]=eesti', 'pos[0]=_H_', 'case[0]=g', 'lem[1]=vabariik', 'pos[1]=_S_', 'post[1]=riik', 'lem[2]=ole', 'pos[2]=_V_', 'gaz[0]=loc', 'gaz[1]=loc', 'iu[0]|fsnt[0]=y|y', 'lem[0]|lem[1]=eesti|vabariik', 'pos[0]|pos[1]=_H_|_S_', 'iu[0]|iu[1]=y|y', 'gaz[0]|gaz[1]=loc|loc'], ['iu[0]=y', 'p3[0]=vab', 'p4[0]=vaba', 's3[0]=iik', 's4[0]=riik', 'aan[0]=y', 'cu[0]=y', 'iu[-1]=y', 'cu[-1]=y', 'fsnt[-1]=y', 'lem[0]=vabariik', 'pos[0]=_S_', 'pref[0]=vaba', 'post[0]=riik', 'case[0]=n', 'lem[-1]=eesti', 'prop[-1]=y', 'pos[-1]=_H_', 'lem[1]=ole', 'pos[1]=_V_', 'lem[2]=riik', 'pos[2]=_S_', 'iuoc[0]=y', 'nprop[0]=y', 'pprop[0]=y', 'pgaz[0]=loc', 'gaz[0]=loc', 'gaz[-1]=loc', 'lem[0]|lem[-1]=vabariik|eesti', 'lem[0]|lem[1]=vabariik|ole', 'pos[0]|pos[-1]=_S_|_H_', 'pos[0]|pos[1]=_S_|_V_', 'iu[0]|iu[-1]=y|y', 'gaz[-1]|gaz[0]=loc|loc'], ['p3[0]=ole', 's3[0]=ole', 'aan[0]=y', 'iu[-1]=y', 'cu[-1]=y', 'iu[-2]=y', 'cu[-2]=y', 'fsnt[-2]=y', 'iu[2]=y', 'cu[2]=y', 'cs[2]=y', 'cds[2]=y', 'lem[0]=ole', 'pos[0]=_V_', 'lem[-1]=vabariik', 'pos[-1]=_S_', 'post[-1]=riik', 'lem[-2]=eesti', 'prop[-2]=y', 'pos[-2]=_H_', 'lem[1]=riik', 'pos[1]=_S_', 'lem[2]=põhjaeuroobas', 'prop[2]=y', 'pos[2]=_H_', 'post[2]=roobas', 'gaz[-1]=loc', 'gaz[-2]=loc', 'lem[0]|lem[-1]=ole|vabariik', 'lem[0]|lem[1]=ole|riik', 'pos[0]|pos[-1]=_V_|_S_', 'pos[0]|pos[1]=_V_|_S_'], ['p3[0]=rii', 'p4[0]=riik', 's3[0]=iik', 's4[0]=riik', 'aan[0]=y', 'iu[-2]=y', 'cu[-2]=y', 'iu[1]=y', 'cu[1]=y', 'cs[1]=y', 'cds[1]=y', 'cs[2]=y', 'cdt[2]=y', 'lem[0]=riik', 'pos[0]=_S_', 'case[0]=n', 'lem[-1]=ole', 'pos[-1]=_V_', 'lem[-2]=vabariik', 'pos[-2]=_S_', 'post[-2]=riik', 'lem[1]=põhjaeuroobas', 'prop[1]=y', 'pos[1]=_H_', 'post[1]=roobas', 'lem[2]=.', 'pos[2]=_Z_', 'pun[2]=y', 'gaz[-2]=loc', 'lem[0]|lem[-1]=riik|ole', 'lem[0]|lem[1]=riik|põhjaeuroobas', 'pos[0]|pos[-1]=_S_|_V_', 'pos[0]|pos[1]=_S_|_H_'], ['iu[0]=y', 'p3[0]=põh', 'p4[0]=põhj', 's3[0]=bas', 's4[0]=obas', 'cu[0]=y', 'cs[0]=y', 'cds[0]=y', 'cs[1]=y', 'cdt[1]=y', 'iu[2]=y', 'cu[2]=y', 'fsnt[2]=y', 'lem[0]=põhjaeuroobas', 'pos[0]=_H_', 'pref[0]=Põhja', 'post[0]=roobas', 'case[0]=in', 'end[0]=s', 'lem[-1]=riik', 'pos[-1]=_S_', 'lem[-2]=ole', 'pos[-2]=_V_', 'lem[1]=.', 'pos[1]=_Z_', 'pun[1]=y', 'lem[2]=eesti', 'prop[2]=y', 'pos[2]=_H_', 'iuoc[0]=y', 'nprop[0]=y', 'pprop[0]=y', 'gaz[2]=loc', 'lem[0]|lem[-1]=põhjaeuroobas|riik', 'lem[0]|lem[1]=põhjaeuroobas|.', 'pos[0]|pos[-1]=_H_|_S_', 'pos[0]|pos[1]=_H_|_Z_'], ['ao[0]=y', 'cs[0]=y', 'cdt[0]=y', 'iu[-1]=y', 'cu[-1]=y', 'cs[-1]=y', 'cds[-1]=y', 'iu[1]=y', 'cu[1]=y', 'fsnt[1]=y', 'lem[0]=.', 'pos[0]=_Z_', 'pun[0]=y', 'end[0]=', 'lem[-1]=põhjaeuroobas', 'prop[-1]=y', 'pos[-1]=_H_', 'post[-1]=roobas', 'lem[-2]=riik', 'pos[-2]=_S_', 'lem[1]=eesti', 'prop[1]=y', 'pos[1]=_H_', 'lem[2]=piirne', 'pos[2]=_V_', 'gaz[1]=loc', 'lem[0]|lem[-1]=.|põhjaeuroobas', 'lem[0]|lem[1]=.|eesti', 'pos[0]|pos[-1]=_Z_|_H_', 'pos[0]|pos[1]=_Z_|_H_'], ['iu[0]=y', 'fsnt[0]=y', 'p3[0]=ees', 'p4[0]=eest', 's3[0]=sti', 's4[0]=esti', 'aan[0]=y', 'cu[0]=y', 'cs[-1]=y', 'cdt[-1]=y', 'iu[-2]=y', 'cu[-2]=y', 'cs[-2]=y', 'cds[-2]=y', 'lem[0]=eesti', 'pos[0]=_H_', 'case[0]=n', 'lem[-1]=.', 'pos[-1]=_Z_', 'pun[-1]=y', 'lem[-2]=põhjaeuroobas', 'prop[-2]=y', 'pos[-2]=_H_', 'post[-2]=roobas', 'lem[1]=piirne', 'pos[1]=_V_', 'lem[2]=põhi', 'pos[2]=_S_', 'gaz[0]=loc', 'iu[0]|fsnt[0]=y|y', 'lem[0]|lem[-1]=eesti|.', 'lem[0]|lem[1]=eesti|piirne', 'pos[0]|pos[-1]=_H_|_Z_', 'pos[0]|pos[1]=_H_|_V_'], ['p3[0]=pii', 'p4[0]=piir', 's3[0]=rne', 's4[0]=irne', 'aan[0]=y', 'iu[-1]=y', 'cu[-1]=y', 'fsnt[-1]=y', 'cs[-2]=y', 'cdt[-2]=y', 'lem[0]=piirne', 'pos[0]=_V_', 'end[0]=b', 'lem[-1]=eesti', 'prop[-1]=y', 'pos[-1]=_H_', 'lem[-2]=.', 'pos[-2]=_Z_', 'pun[-2]=y', 'lem[1]=põhi', 'pos[1]=_S_', 'lem[2]=üle', 'pos[2]=_D_', 'gaz[-1]=loc', 'lem[0]|lem[-1]=piirne|eesti', 'lem[0]|lem[1]=piirne|põhi', 'pos[0]|pos[-1]=_V_|_H_', 'pos[0]|pos[1]=_V_|_S_'], ['p3[0]=põh', 'p4[0]=põhi', 's3[0]=õhi', 's4[0]=põhi', 'aan[0]=y', 'iu[-2]=y', 'cu[-2]=y', 'fsnt[-2]=y', 'iu[2]=y', 'cu[2]=y', 'lem[0]=põhi', 'pos[0]=_S_', 'case[0]=in', 'end[0]=s', 'lem[-1]=piirne', 'pos[-1]=_V_', 'lem[-2]=eesti', 'prop[-2]=y', 'pos[-2]=_H_', 'lem[1]=üle', 'pos[1]=_D_', 'lem[2]=soome', 'prop[2]=y', 'pos[2]=_H_', 'gaz[-2]=loc', 'gaz[2]=loc', 'lem[0]|lem[-1]=põhi|piirne', 'lem[0]|lem[1]=põhi|üle', 'pos[0]|pos[-1]=_S_|_V_', 'pos[0]|pos[1]=_S_|_D_'], ['p3[0]=üle', 's3[0]=üle', 'aan[0]=y', 'iu[1]=y', 'cu[1]=y', 'lem[0]=üle', 'pos[0]=_D_', 'lem[-1]=põhi', 'pos[-1]=_S_', 'lem[-2]=piirne', 'pos[-2]=_V_', 'lem[1]=soome', 'prop[1]=y', 'pos[1]=_H_', 'lem[2]=laht', 'pos[2]=_S_', 'gaz[1]=loc', 'gaz[2]=loc', 'lem[0]|lem[-1]=üle|põhi', 'lem[0]|lem[1]=üle|soome', 'pos[0]|pos[-1]=_D_|_S_', 'pos[0]|pos[1]=_D_|_H_'], ['iu[0]=y', 'p3[0]=soo', 'p4[0]=soom', 's3[0]=ome', 's4[0]=oome', 'aan[0]=y', 'cu[0]=y', 'iu[2]=y', 'cu[2]=y', 'lem[0]=soome', 'pos[0]=_H_', 'case[0]=g', 'lem[-1]=üle', 'pos[-1]=_D_', 'lem[-2]=põhi', 'pos[-2]=_S_', 'lem[1]=laht', 'pos[1]=_S_', 'lem[2]=soome', 'prop[2]=y', 'pos[2]=_H_', 'iuoc[0]=y', 'nprop[0]=y', 'pprop[0]=y', 'ngaz[0]=loc', 'pgaz[0]=loc', 'gaz[0]=loc', 'gaz[1]=loc', 'gaz[2]=loc', 'lem[0]|lem[-1]=soome|üle', 'lem[0]|lem[1]=soome|laht', 'pos[0]|pos[-1]=_H_|_D_', 'pos[0]|pos[1]=_H_|_S_', 'gaz[0]|gaz[1]=loc|loc'], ['p3[0]=lah', 'p4[0]=laht', 's3[0]=aht', 's4[0]=laht', 'aan[0]=y', 'iu[-1]=y', 'cu[-1]=y', 'iu[1]=y', 'cu[1]=y', 'iu[2]=y', 'cu[2]=y', 'lem[0]=laht', 'pos[0]=_S_', 'case[0]=g', 'lem[-1]=soome', 'prop[-1]=y', 'pos[-1]=_H_', 'lem[-2]=üle', 'pos[-2]=_D_', 'lem[1]=soome', 'prop[1]=y', 'pos[1]=_H_', 'lem[2]=vabariik', 'prop[2]=y', 'pos[2]=_H_', 'post[2]=riik', 'gaz[0]=loc', 'gaz[-1]=loc', 'gaz[1]=loc', 'gaz[2]=loc', 'lem[0]|lem[-1]=laht|soome', 'lem[0]|lem[1]=laht|soome', 'pos[0]|pos[-1]=_S_|_H_', 'pos[0]|pos[1]=_S_|_H_', 'gaz[0]|gaz[1]=loc|loc', 'gaz[-1]|gaz[0]=loc|loc'], ['iu[0]=y', 'p3[0]=soo', 'p4[0]=soom', 's3[0]=ome', 's4[0]=oome', 'aan[0]=y', 'cu[0]=y', 'iu[-2]=y', 'cu[-2]=y', 'iu[1]=y', 'cu[1]=y', 'cs[2]=y', 'cdt[2]=y', 'lem[0]=soome', 'pos[0]=_H_', 'case[0]=n', 'lem[-1]=laht', 'pos[-1]=_S_', 'lem[-2]=soome', 'prop[-2]=y', 'pos[-2]=_H_', 'lem[1]=vabariik', 'prop[1]=y', 'pos[1]=_H_', 'post[1]=riik', 'lem[2]=.', 'pos[2]=_Z_', 'pun[2]=y', 'iuoc[0]=y', 'nprop[0]=y', 'pprop[0]=y', 'ngaz[0]=loc', 'pgaz[0]=loc', 'gaz[0]=loc', 'gaz[-1]=loc', 'gaz[-2]=loc', 'gaz[1]=loc', 'lem[0]|lem[-1]=soome|laht', 'lem[0]|lem[1]=soome|vabariik', 'pos[0]|pos[-1]=_H_|_S_', 'pos[0]|pos[1]=_H_|_H_', 'iu[0]|iu[1]=y|y', 'gaz[0]|gaz[1]=loc|loc', 'gaz[-1]|gaz[0]=loc|loc'], ['iu[0]=y', 'p3[0]=vab', 'p4[0]=vaba', 's3[0]=iik', 's4[0]=riik', 'aan[0]=y', 'cu[0]=y', 'iu[-1]=y', 'cu[-1]=y', 'cs[1]=y', 'cdt[1]=y', 'lem[0]=vabariik', 'pos[0]=_H_', 'pref[0]=Vaba', 'post[0]=riik', 'case[0]=kom', 'end[0]=ga', 'lem[-1]=soome', 'prop[-1]=y', 'pos[-1]=_H_', 'lem[-2]=laht', 'pos[-2]=_S_', 'lem[1]=.', 'pos[1]=_Z_', 'pun[1]=y', 'iuoc[0]=y', 'nprop[0]=y', 'pprop[0]=y', 'pgaz[0]=loc', 'gaz[0]=loc', 'gaz[-1]=loc', 'gaz[-2]=loc', 'lem[0]|lem[-1]=vabariik|soome', 'lem[0]|lem[1]=vabariik|.', 'pos[0]|pos[-1]=_H_|_H_', 'pos[0]|pos[1]=_H_|_Z_', 'iu[0]|iu[-1]=y|y', 'gaz[-1]|gaz[0]=loc|loc'], ['ao[0]=y', 'cs[0]=y', 'cdt[0]=y', 'iu[-1]=y', 'cu[-1]=y', 'iu[-2]=y', 'cu[-2]=y', 'lem[0]=.', 'pos[0]=_Z_', 'pun[0]=y', 'end[0]=', 'lem[-1]=vabariik', 'prop[-1]=y', 'pos[-1]=_H_', 'post[-1]=riik', 'lem[-2]=soome', 'prop[-2]=y', 'pos[-2]=_H_', 'gaz[-1]=loc', 'gaz[-2]=loc', 'lem[0]|lem[-1]=.|vabariik', 'pos[0]|pos[-1]=_Z_|_H_']]

In [16]:
inputs[1]

['iu[0]=y',
 'p3[0]=vab',
 'p4[0]=vaba',
 's3[0]=iik',
 's4[0]=riik',
 'aan[0]=y',
 'cu[0]=y',
 'iu[-1]=y',
 'cu[-1]=y',
 'fsnt[-1]=y',
 'lem[0]=vabariik',
 'pos[0]=_S_',
 'pref[0]=vaba',
 'post[0]=riik',
 'case[0]=n',
 'lem[-1]=eesti',
 'prop[-1]=y',
 'pos[-1]=_H_',
 'lem[1]=ole',
 'pos[1]=_V_',
 'lem[2]=riik',
 'pos[2]=_S_',
 'iuoc[0]=y',
 'pprop[0]=y',
 'pgaz[0]=loc',
 'gaz[0]=loc',
 'gaz[-1]=loc',
 'lem[0]|lem[-1]=vabariik|eesti',
 'lem[0]|lem[1]=vabariik|ole',
 'pos[0]|pos[-1]=_S_|_H_',
 'pos[0]|pos[1]=_S_|_V_',
 'iu[0]|iu[-1]=y|y',
 'gaz[-1]|gaz[0]=loc|loc']

In [15]:
my_inputs2[1]

['iu[0]=y',
 'p3[0]=vab',
 'p4[0]=vaba',
 's3[0]=iik',
 's4[0]=riik',
 'aan[0]=y',
 'cu[0]=y',
 'iu[-1]=y',
 'cu[-1]=y',
 'fsnt[-1]=y',
 'lem[0]=vabariik',
 'pos[0]=_S_',
 'pref[0]=vaba',
 'post[0]=riik',
 'case[0]=n',
 'lem[-1]=eesti',
 'prop[-1]=y',
 'pos[-1]=_H_',
 'lem[1]=ole',
 'pos[1]=_V_',
 'lem[2]=riik',
 'pos[2]=_S_',
 'iuoc[0]=y',
 'nprop[0]=y',
 'pprop[0]=y',
 'pgaz[0]=loc',
 'gaz[0]=loc',
 'gaz[-1]=loc',
 'lem[0]|lem[-1]=vabariik|eesti',
 'lem[0]|lem[1]=vabariik|ole',
 'pos[0]|pos[-1]=_S_|_H_',
 'pos[0]|pos[1]=_S_|_V_',
 'iu[0]|iu[-1]=y|y',
 'gaz[-1]|gaz[0]=loc|loc']

In [14]:
for i in range(len(inputs[1])):
    print(inputs[1][i] == my_inputs2[1][i])

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
False
False
False
False
False
False
False
False
False
False


In [35]:
inputs[10] == my_inputs[10]

False

In [45]:
text

Text(text='Eesti Vabariik on riik Põhja-Euroopas.\n    Eesti piirneb põhjas üle Soome lahe Soome Vabariigiga.\n    ')

In [43]:
my_inputs2[1]

['iu[0]=y',
 'p3[0]=vab',
 'p4[0]=vaba',
 's3[0]=iik',
 's4[0]=riik',
 'aan[0]=y',
 'cu[0]=y',
 'iu[-1]=y',
 'cu[-1]=y',
 'fsnt[-1]=y',
 'lem[0]=vabariik',
 'pos[0]=_S_',
 'pref[0]=vaba',
 'post[0]=riik',
 'case[0]=g',
 'lem[-1]=eesti',
 'prop[-1]=y',
 'pos[-1]=_H_',
 'lem[1]=ole',
 'pos[1]=_V_',
 'lem[2]=riik',
 'pos[2]=_S_',
 'iuoc[0]=y',
 'nprop[0]=y',
 'pprop[0]=y',
 'pgaz[0]=loc',
 'gaz[0]=loc',
 'gaz[-1]=loc',
 'lem[0]|lem[-1]=vabariik|eesti',
 'lem[0]|lem[1]=vabariik|ole',
 'pos[0]|pos[-1]=_S_|_H_',
 'pos[0]|pos[1]=_S_|_V_',
 'iu[0]|iu[-1]=y|y',
 'gaz[-1]|gaz[0]=loc|loc']

In [24]:
inputs[1]

['iu[0]=y',
 'p3[0]=vab',
 'p4[0]=vaba',
 's3[0]=iik',
 's4[0]=riik',
 'aan[0]=y',
 'cu[0]=y',
 'iu[-1]=y',
 'cu[-1]=y',
 'fsnt[-1]=y',
 'lem[0]=vabariik',
 'pos[0]=_S_',
 'pref[0]=vaba',
 'post[0]=riik',
 'case[0]=n',
 'lem[-1]=eesti',
 'prop[-1]=y',
 'pos[-1]=_H_',
 'lem[1]=ole',
 'pos[1]=_V_',
 'lem[2]=riik',
 'pos[2]=_S_',
 'iuoc[0]=y',
 'pprop[0]=y',
 'pgaz[0]=loc',
 'gaz[0]=loc',
 'gaz[-1]=loc',
 'lem[0]|lem[-1]=vabariik|eesti',
 'lem[0]|lem[1]=vabariik|ole',
 'pos[0]|pos[-1]=_S_|_H_',
 'pos[0]|pos[1]=_S_|_V_',
 'iu[0]|iu[-1]=y|y',
 'gaz[-1]|gaz[0]=loc|loc']

In [16]:
text.ner_features[2]

text,value,name
on,ole,lem
,_V_,pos
,None,prop
,None,pref
,None,post
,None,case
,None,end
,None,pun
,on,w
,on,wl


In [21]:
text.root_tokens

,root_tokens
0,['Eesti']
1,"['vaba', 'riik']"
2,['ole']
,['ole']
3,['riik']
4,"['Põhja', 'Eu', 'roobas']"
,"['Põhja', 'Euroopa']"
5,['.']
6,['Eesti']
7,['piirne']


In [112]:
inputs[0][0]

'iu[0]=y'

In [5]:
taggija.tag(inputs)

NameError: name 'inputs' is not defined

In [25]:
from estnltk.taggers.estner.fex import FeatureExtractor

In [6]:
nersettings

NameError: name 'nersettings' is not defined

In [26]:
fex = FeatureExtractor(nersettings)

In [30]:
fex.process([text])

AttributeError: tokens

In [30]:
inputs[0]

['iu[0]=y',
 'fsnt[0]=y',
 'p3[0]=ees',
 'p4[0]=eest',
 's3[0]=sti',
 's4[0]=esti',
 'aan[0]=y',
 'cu[0]=y',
 'iu[1]=y',
 'cu[1]=y',
 'lem[0]=eesti',
 'pos[0]=_H_',
 'case[0]=g',
 'lem[1]=vabariik',
 'pos[1]=_S_',
 'post[1]=riik',
 'lem[2]=ole',
 'pos[2]=_V_',
 'iuoc[0]=y',
 'ngaz[0]=loc',
 'gaz[0]=loc',
 'gaz[1]=loc',
 'iu[0]|fsnt[0]=y|y',
 'lem[0]|lem[1]=eesti|vabariik',
 'pos[0]|pos[1]=_H_|_S_',
 'iu[0]|iu[1]=y|y',
 'gaz[0]|gaz[1]=loc|loc']

In [121]:
inputs[0]

['iu[0]=y',
 'fsnt[0]=y',
 'p3[0]=ees',
 'p4[0]=eest',
 's3[0]=sti',
 's4[0]=esti',
 'aan[0]=y',
 'cu[0]=y',
 'iu[1]=y',
 'cu[1]=y',
 'lem[0]=eesti',
 'pos[0]=_H_',
 'case[0]=g',
 'lem[1]=vabariik',
 'pos[1]=_S_',
 'post[1]=riik',
 'lem[2]=ole',
 'pos[2]=_V_',
 'iuoc[0]=y',
 'ngaz[0]=loc',
 'gaz[0]=loc',
 'gaz[1]=loc',
 'iu[0]|fsnt[0]=y|y',
 'lem[0]|lem[1]=eesti|vabariik',
 'pos[0]|pos[1]=_H_|_S_',
 'iu[0]|iu[1]=y|y',
 'gaz[0]|gaz[1]=loc|loc']

In [13]:
text.tag_layer()

Text(text='Eesti Vabariik on riik Põhja-Euroopas.\n    Eesti piirneb põhjas üle Soome lahe Soome Vabariigiga.\n    ')

In [59]:
text.ner

Layer(name='ner', attributes=('nertag', 'name'), spans=SL[EnvelopingSpan(['Eesti', 'Vabariik'], [{'nertag': 'LOC', 'name': 'po'}]),
EnvelopingSpan(['Põhja-Euroopas'], [{'nertag': 'LOC', 'name': 'po'}]),
EnvelopingSpan(['Eesti'], [{'nertag': 'LOC', 'name': 'po'}]),
EnvelopingSpan(['Soome', 'lahe'], [{'nertag': 'LOC', 'name': 'po'}]),
EnvelopingSpan(['Soome', 'Vabariigiga'], [{'nertag': 'LOC', 'name': 'po'}])])

In [1]:
mfe = MorphFeatureExtractor()

NameError: name 'MorphFeatureExtractor' is not defined

In [12]:
from estnltk.taggers.estner.refac.ner import json_document_to_estner_document



In [56]:
nerdoc = json_document_to_estner_document(textobject2)

In [18]:
import re

In [34]:
from copy import deepcopy

In [59]:
textobject2.tag_layer()

Text(text='Eesti Vabariik on riik Põhja-Euroopas. Eesti piirneb põhjas üle Soome lahe Soome Vabariigiga.')

In [63]:
textobject2.morph_analysis[0]

text,normalized_text,lemma,root,root_tokens,ending,clitic,form,partofspeech
Eesti,Eesti,Eesti,Eesti,['Eesti'],0,,sg g,H


In [67]:
for token in nerdoc.tokens:
    token2 = deepcopy(token)
    mfe._process(token2)
    print(token2)
    print(token2.feature_list())

{'lem': 'eesti', 'pos': '_H_', 'prop': 'y', 'case': 'g'}


KeyError: 'F'

In [164]:
final_input = []
for span in text.ner_features:
    features = []
    for annotation in span.annotations:
        features.append(str(annotation.name)+"="+str(annotation.value))
    final_input.append(features)

In [166]:
taggija.tag(final_input)

['I-ORG',
 'I-ORG',
 'I-ORG',
 'I-ORG',
 'I-ORG',
 'I-ORG',
 'I-ORG',
 'I-ORG',
 'I-ORG',
 'I-ORG',
 'I-ORG',
 'I-ORG',
 'I-ORG',
 'I-ORG',
 'I-ORG']

### test

In [11]:
loppinput = [['iu[0]=y', 'fsnt[0]=y', 'p3[0]=Ees', 'p4[0]=Eest', 's3[0]=sti', 's4[0]=esti', '', 'aan[0]=y', 'cu[0]=y', 'iu[1]=y', 'cu[1]=y', 'lem[0]=Eesti', 'pos[0]=_H_', 'case[0]=g', 'lem[1]=vabariik', 'pos[1]=_S_', 'post[1]=riik', 'lem[2]=olema', 'pos[2]=_V_', 'gaz[0]=loc', 'gaz[1]=loc', 'iu[0]|fsnt[0]=y|y', 'lem[0]|lem[1]=Eesti|vabariik', 'pos[0]|pos[1]=_H_|_S_', 'iu[0]|iu[1]=y|y', 'gaz[0]|gaz[1]=loc|loc'], ['', '', '', 'loc', 'iu[0]=y', 'p3[0]=vab', 'p4[0]=vaba', 's3[0]=iik', 's4[0]=riik', '', 'aan[0]=y', 'cu[0]=y', 'iu[-1]=y', 'cu[-1]=y', 'fsnt[-1]=y', 'lem[0]=vabariik', 'pos[0]=_S_', 'pref[0]=vaba', 'post[0]=riik', 'case[0]=g', 'lem[-1]=Eesti', 'prop[-1]=y', 'pos[-1]=_H_', 'lem[1]=olema', 'pos[1]=_V_', 'lem[2]=riik', 'pos[2]=_S_', 'iuoc[0]=y', 'nprop[0]=y', 'pprop[0]=y', 'pgaz[0]=loc', 'gaz[0]=loc', 'gaz[-1]=loc', 'lem[0]|lem[-1]=vabariik|Eesti', 'lem[0]|lem[1]=vabariik|olema', 'pos[0]|pos[-1]=_S_|_H_', 'pos[0]|pos[1]=_S_|_V_', 'iu[0]|iu[-1]=y|y', 'gaz[-1]|gaz[0]=loc|loc'], ['', '', 'loc', '', 'p3[0]=ole', 'p4[0]=olem', 's3[0]=ema', 's4[0]=lema', 'aan[0]=y', 'iu[-1]=y', 'cu[-1]=y', 'iu[-2]=y', 'cu[-2]=y', 'fsnt[-2]=y', 'iu[2]=y', 'cu[2]=y', 'cs[2]=y', 'cds[2]=y', 'adash[2]=Euroobas', 'lem[0]=olema', 'pos[0]=_V_', 'lem[-1]=vabariik', 'pos[-1]=_S_', 'post[-1]=riik', 'lem[-2]=Eesti', 'prop[-2]=y', 'pos[-2]=_H_', 'lem[1]=riik', 'pos[1]=_S_', 'lem[2]=Põhja-Euroobas', 'prop[2]=y', 'pos[2]=_H_', 'post[2]=Eu', 'iuoc[0]=y', 'nprop[0]=y', 'pprop[0]=y', 'pgaz[0]=loc', 'gaz[-1]=loc', 'gaz[-2]=loc', 'lem[0]|lem[-1]=olema|vabariik', 'lem[0]|lem[1]=olema|riik', 'pos[0]|pos[-1]=_V_|_S_', 'pos[0]|pos[1]=_V_|_S_'], ['', '', '', 'p3[0]=rii', 'p4[0]=riik', 's3[0]=iik', 's4[0]=riik', 'aan[0]=y', 'iu[-2]=y', 'cu[-2]=y', 'iu[1]=y', 'cu[1]=y', 'cs[1]=y', 'cds[1]=y', 'adash[1]=Euroobas', 'cs[2]=y', 'cdt[2]=y', 'lem[0]=riik', 'pos[0]=_S_', 'case[0]=g', 'lem[-1]=olema', 'pos[-1]=_V_', 'lem[-2]=vabariik', 'pos[-2]=_S_', 'post[-2]=riik', 'lem[1]=Põhja-Euroobas', 'prop[1]=y', 'pos[1]=_H_', 'post[1]=Eu', 'lem[2]=.', 'pos[2]=_Z_', 'pun[2]=y', 'iuoc[0]=y', 'nprop[0]=y', 'pprop[0]=y', 'gaz[-2]=loc', 'lem[0]|lem[-1]=riik|olema', 'lem[0]|lem[1]=riik|Põhja-Euroobas', 'pos[0]|pos[-1]=_S_|_V_', 'pos[0]|pos[1]=_S_|_H_'], ['', '', 'iu[0]=y', 'p3[0]=Põh', 'p4[0]=Põhj', 's3[0]=bas', 's4[0]=obas', 'bdash[0]=Põhja', 'adash[0]=Euroobas', '', 'cu[0]=y', 'cs[0]=y', 'cds[0]=y', 'cs[1]=y', 'cdt[1]=y', 'iu[2]=y', 'cu[2]=y', 'fsnt[2]=y', 'lem[0]=Põhja-Euroobas', 'pos[0]=_H_', 'pref[0]=Põhja', 'post[0]=Eu', 'case[0]=g', 'end[0]=s', 'lem[-1]=riik', 'pos[-1]=_S_', 'lem[-2]=olema', 'pos[-2]=_V_', 'lem[1]=.', 'pos[1]=_Z_', 'pun[1]=y', 'lem[2]=Eesti', 'prop[2]=y', 'pos[2]=_H_', 'iuoc[0]=y', 'nprop[0]=y', 'pprop[0]=y', 'gaz[2]=loc', 'lem[0]|lem[-1]=Põhja-Euroobas|riik', 'lem[0]|lem[1]=Põhja-Euroobas|.', 'pos[0]|pos[-1]=_H_|_S_', 'pos[0]|pos[1]=_H_|_Z_'], ['', '', '', 'loc', 'loc', '', 'ao[0]=y', 'cs[0]=y', 'cdt[0]=y', 'iu[-1]=y', 'cu[-1]=y', 'cs[-1]=y', 'cds[-1]=y', 'adash[-1]=Euroobas', 'iu[1]=y', 'cu[1]=y', 'fsnt[1]=y', 'lem[0]=.', 'pos[0]=_Z_', 'pun[0]=y', 'end[0]=', 'lem[-1]=Põhja-Euroobas', 'prop[-1]=y', 'pos[-1]=_H_', 'post[-1]=Eu', 'lem[-2]=riik', 'pos[-2]=_S_', 'lem[1]=Eesti', 'prop[1]=y', 'pos[1]=_H_', 'lem[2]=piirnema', 'pos[2]=_V_', 'iuoc[0]=y', 'nprop[0]=y', 'pprop[0]=y', 'ngaz[0]=loc', 'pgaz[0]=loc', 'gaz[1]=loc', 'lem[0]|lem[-1]=.|Põhja-Euroobas', 'lem[0]|lem[1]=.|Eesti', 'pos[0]|pos[-1]=_Z_|_H_', 'pos[0]|pos[1]=_Z_|_H_'], ["an('piirneb', [{'value': 'piirnema', 'name': 'lem'}, {'value': '_V_', 'name': 'pos'}, {'value': None, 'name': 'prop'}, {'value': None, 'name': 'pref'}, {'value': None, 'name': 'post'}, {'value': None, 'name': 'case'}, {'value': 'b', 'name': 'end'}, {'value': None, 'name': 'pun'}, {'value': 'piirneb', 'name': 'w'}, {'value': 'piirneb', 'name': 'wl'}, {'value': 'LLLLLLL', 'name': 'shape'}, {'value': 'L', 'name': 'shaped'}, {'value': 'p', 'name': 'p1'}, {'value': 'pi', 'name': 'p2'}, {'value': 'pii', 'name': 'p3'}, {'value': 'piir', 'name': 'p4'}, {'value': 'a', 'name': 's1'}, {'value': 'ma', 'name': 's2'}, {'value': 'ema', 'name': 's3'}, {'value': 'nema', 'name': 's4'}, {'value': None, 'name': '2d'}, {'value': None, 'name': '4d'}, {'value': None, 'name': 'd&-'}, {'value': None, 'name': 'd&/'}, {'value': None, 'name': 'd&,'}, {'value': None, 'name': 'd&.'}, {'value': None, 'name': 'up'}, {'value': None, 'name': 'iu'}, {'value': None, 'name': 'au'}, {'value': 'y', 'name': 'al'}, {'value': None, 'name': 'ad'}, {'value': None, 'name': 'ao'}, {'value': 'y', 'name': 'aan'}, {'value': None, 'name': 'cu'}, {'value': 'y', 'name': 'cl'}, {'value': 'y', 'name': 'ca'}, {'value': None, 'name': 'cd'}, {'value': None, 'name': 'cp'}, {'value': None, 'name': 'cds'}, {'value': None, 'name': 'cdt'}, {'value': None, 'name': 'cs'}, {'value': None, 'name': 'bdash'}, {'value': None, 'name': 'adash'}, {'value': None, 'name': 'bdot'}, {'value': None, 'name': 'adot'}, {'value': '8', 'name': 'len'}, {'value': Span('Eesti', [{'value': 'Eesti', 'name': 'lem'}, {'value': '_H_', 'name': 'pos'}, {'value': 'y', 'name': 'prop'}, {'value': None, 'name': 'pref'}, {'value': None, 'name': 'post'}, {'value': 'g', 'name': 'case'}, {'value': None, 'name': 'end'}, {'value': None, 'name': 'pun'}, {'value': 'Eesti', 'name': 'w'}, {'value': 'eesti', 'name': 'wl'}, {'value': 'ULLLL', 'name': 'shape'}, {'value': 'UL', 'name': 'shaped'}, {'value': 'E', 'name': 'p1'}, {'value': 'Ee', 'name': 'p2'}, {'value': 'Ees', 'name': 'p3'}, {'value': 'Eest', 'name': 'p4'}, {'value': 'i', 'name': 's1'}, {'value': 'ti', 'name': 's2'}, {'value': 'sti', 'name': 's3'}, {'value': 'esti', 'name': 's4'}, {'value': None, 'name': '2d'}, {'value': None, 'name': '4d'}, {'value': None, 'name': 'd&-'}, {'value': None, 'name': 'd&/'}, {'value': None, 'name': 'd&,'}, {'value': None, 'name': 'd&.'}, {'value': None, 'name': 'up'}, {'value': 'y', 'name': 'iu'}, {'value': None, 'name': 'au'}, {'value': None, 'name': 'al'}, {'value': None, 'name': 'ad'}, {'value': None, 'name': 'ao'}, {'value': 'y', 'name': 'aan'}, {'value': 'y', 'name': 'cu'}, {'value': 'y', 'name': 'cl'}, {'value': 'y', 'name': 'ca'}, {'value': None, 'name': 'cd'}, {'value': None, 'name': 'cp'}, {'value': None, 'name': 'cds'}, {'value': None, 'name': 'cdt'}, {'value': None, 'name': 'cs'}, {'value': None, 'name': 'bdash'}, {'value': None, 'name': 'adash'}, {'value': None, 'name': 'bdot'}, {'value': None, 'name': 'adot'}, {'value': '5', 'name': 'len'}, {'value': 'y', 'name': 'fsnt'}, {'value': {'loc'}, 'name': 'gaz'}, {'value': Span('.', [{'value': '.', 'name': 'lem'}, {'value': '_Z_', 'name': 'pos'}, {'value': None, 'name': 'prop'}, {'value': None, 'name': 'pref'}, {'value': None, 'name': 'post'}, {'value': None, 'name': 'case'}, {'value': '', 'name': 'end'}, {'value': 'y', 'name': 'pun'}, {'value': '.', 'name': 'w'}, {'value': '.', 'name': 'wl'}, {'value': '.', 'name': 'shape'}, {'value': '.', 'name': 'shaped'}, {'value': '.', 'name': 'p1'}, {'value': None, 'name': 'p2'}, {'value': None, 'name': 'p3'}, {'value': None, 'name': 'p4'}, {'value': '.', 'name': 's1'}, {'value': None, 'name': 's2'}, {'value': None, 'name': 's3'}, {'value': None, 'name': 's4'}, {'value': None, 'name': '2d'}, {'value': None, 'name': '4d'}, {'value': None, 'name': 'd&-'}, {'value': None, 'name': 'd&/'}, {'value': None, 'name': 'd&,'}, {'value': None, 'name': 'd&.'}, {'value': None, 'name': 'up'}, {'value': None, 'name': 'iu'}, {'value': None, 'name': 'au'}, {'value': None, 'name': 'al'}, {'value': None, 'name': 'ad'}, {'value': 'y', 'name': 'ao'}, {'value': None, 'name': 'aan'}, {'value': None, 'name': 'cu'}, {'value': None, 'name': 'cl'}, {'value': None, 'name': 'ca'}, {'value': None, 'name': 'cd'}, {'value': None, 'name': 'cp'}, {'value': None, 'name': 'cds'}, {'value': 'y', 'name': 'cdt'}, {'value': 'y', 'name': 'cs'}, {'value': None, 'name': 'bdash'}, {'value': None, 'name': 'adash'}, {'value': '', 'name': 'bdot'}, {'value': '', 'name': 'adot'}, {'value': '1', 'name': 'len'}, {'value': 'y', 'name': 'lsnt'}, {'value': Span('Põhja-Euroopas', [{'value': 'Põhja-Euroobas', 'name': 'lem'}, {'value': '_H_', 'name': 'pos'}, {'value': 'y', 'name': 'prop'}, {'value': 'Põhja', 'name': 'pref'}, {'value': 'Eu', 'name': 'post'}, {'value': 'g', 'name': 'case'}, {'value': 's', 'name': 'end'}, {'value': None, 'name': 'pun'}, {'value': 'Põhja-Euroopas', 'name': 'w'}, {'value': 'põhja-euroopas', 'name': 'wl'}, {'value': 'ULLLL-ULLLLLLL', 'name': 'shape'}, {'value': 'UL-UL', 'name': 'shaped'}, {'value': 'P', 'name': 'p1'}, {'value': 'Põ', 'name': 'p2'}, {'value': 'Põh', 'name': 'p3'}, {'value': 'Põhj', 'name': 'p4'}, {'value': 's', 'name': 's1'}, {'value': 'as', 'name': 's2'}, {'value': 'bas', 'name': 's3'}, {'value': 'obas', 'name': 's4'}, {'value': None, 'name': '2d'}, {'value': None, 'name': '4d'}, {'value': None, 'name': 'd&-'}, {'value': None, 'name': 'd&/'}, {'value': None, 'name': 'd&,'}, {'value': None, 'name': 'd&.'}, {'value': None, 'name': 'up'}, {'value': 'y', 'name': 'iu'}, {'value': None, 'name': 'au'}, {'value': None, 'name': 'al'}, {'value': None, 'name': 'ad'}, {'value': None, 'name': 'ao'}, {'value': None, 'name': 'aan'}, {'value': 'y', 'name': 'cu'}, {'value': 'y', 'name': 'cl'}, {'value': 'y', 'name': 'ca'}, {'value': None, 'name': 'cd'}, {'value': None, 'name': 'cp'}, {'value': 'y', 'name': 'cds'}, {'value': None, 'name': 'cdt'}, {'value': 'y', 'name': 'cs'}, {'value': 'Põhja', 'name': 'bdash'}, {'value': 'Euroobas', 'name': 'adash'}, {'value': None, 'name': 'bdot'}, {'value': None, 'name': 'adot'}, {'value': '14', 'name': 'len'}, {'value': Span('riik', [{'value': 'riik', 'name': 'lem'}, {'value': '_S_', 'name': 'pos'}, {'value': None, 'name': 'prop'}, {'value': None, 'name': 'pref'}, {'value': None, 'name': 'post'}, {'value': 'g', 'name': 'case'}, {'value': None, 'name': 'end'}, {'value': None, 'name': 'pun'}, {'value': 'riik', 'name': 'w'}, {'value': 'riik', 'name': 'wl'}, {'value': 'LLLL', 'name': 'shape'}, {'value': 'L', 'name': 'shaped'}, {'value': 'r', 'name': 'p1'}, {'value': 'ri', 'name': 'p2'}, {'value': 'rii', 'name': 'p3'}, {'value': 'riik', 'name': 'p4'}, {'value': 'k', 'name': 's1'}, {'value': 'ik', 'name': 's2'}, {'value': 'iik', 'name': 's3'}, {'value': 'riik', 'name': 's4'}, {'value': None, 'name': '2d'}, {'value': None, 'name': '4d'}, {'value': None, 'name': 'd&-'}, {'value': None, 'name': 'd&/'}, {'value': None, 'name': 'd&,'}, {'value': None, 'name': 'd&.'}, {'value': None, 'name': 'up'}, {'value': None, 'name': 'iu'}, {'value': None, 'name': 'au'}, {'value': 'y', 'name': 'al'}, {'value': None, 'name': 'ad'}, {'value': None, 'name': 'ao'}, {'value': 'y', 'name': 'aan'}, {'value': None, 'name': 'cu'}, {'value': 'y', 'name': 'cl'}, {'value': 'y', 'name': 'ca'}, {'value': None, 'name': 'cd'}, {'value': None, 'name': 'cp'}, {'value': None, 'name': 'cds'}, {'value': None, 'name': 'cdt'}, {'value': None, 'name': 'cs'}, {'value': None, 'name': 'bdash'}, {'value': None, 'name': 'adash'}, {'value': None, 'name': 'bdot'}, {'value': None, 'name': 'adot'}, {'value': '4', 'name': 'len'}, {'value': Span('on', [{'value': 'olema', 'name': 'lem'}, {'value': '_V_', 'name': 'pos'}, {'value': None, 'name': 'prop'}, {'value': None, 'name': 'pref'}, {'value': None, 'name': 'post'}, {'value': None, 'name': 'case'}, {'value': None, 'name': 'end'}, {'value': None, 'name': 'pun'}, {'value': 'on', 'name': 'w'}, {'value': 'on', 'name': 'wl'}, {'value': 'LL', 'name': 'shape'}, {'value': 'L', 'name': 'shaped'}, {'value': 'o', 'name': 'p1'}, {'value': 'ol', 'name': 'p2'}, {'value': 'ole', 'name': 'p3'}, {'value': 'olem', 'name': 'p4'}, {'value': 'a', 'name': 's1'}, {'value': 'ma', 'name': 's2'}, {'value': 'ema', 'name': 's3'}, {'value': 'lema', 'name': 's4'}, {'value': None, 'name': '2d'}, {'value': None, 'name': '4d'}, {'value': None, 'name': 'd&-'}, {'value': None, 'name': 'd&/'}, {'value': None, 'name': 'd&,'}, {'value': None, 'name': 'd&.'}, {'value': None, 'name': 'up'}, {'value': None, 'name': 'iu'}, {'value': None, 'name': 'au'}, {'value': 'y', 'name': 'al'}, {'value': None, 'name': 'ad'}, {'value': None, 'name': 'ao'}, {'value': 'y', 'name': 'aan'}, {'value': None, 'name': 'cu'}, {'value': 'y', 'name': 'cl'}, {'value': 'y', 'name': 'ca'}, {'value': None, 'name': 'cd'}, {'value': None, 'name': 'cp'}, {'value': None, 'name': 'cds'}, {'value': None, 'name': 'cdt'}, {'value': None, 'name': 'cs'}, {'value': None, 'name': 'bdash'}, {'value': None, 'name': 'adash'}, {'value': None, 'name': 'bdot'}, {'value': None, 'name': 'adot'}, {'value': '5', 'name': 'len'}, {'value': Span('Vabariik', [{'value': 'vabariik', 'name': 'lem'}, {'value': '_S_', 'name': 'pos'}, {'value': None, 'name': 'prop'}, {'value': 'vaba', 'name': 'pref'}, {'value': 'riik', 'name': 'post'}, {'value': 'g', 'name': 'case'}, {'value': None, 'name': 'end'}, {'value': None, 'name': 'pun'}, {'value': 'Vabariik', 'name': 'w'}, {'value': 'vabariik', 'name': 'wl'}, {'value': 'ULLLLLLL', 'name': 'shape'}, {'value': 'UL', 'name': 'shaped'}, {'value': 'v', 'name': 'p1'}, {'value': 'va', 'name': 'p2'}, {'value': 'vab', 'name': 'p3'}, {'value': 'vaba', 'name': 'p4'}, {'value': 'k', 'name': 's1'}, {'value': 'ik', 'name': 's2'}, {'value': 'iik', 'name': 's3'}, {'value': 'riik', 'name': 's4'}, {'value': None, 'name': '2d'}, {'value': None, 'name': '4d'}, {'value': None, 'name': 'd&-'}, {'value': None, 'name': 'd&/'}, {'value': None, 'name': 'd&,'}, {'value': None, 'name': 'd&.'}, {'value': None, 'name': 'up'}, {'value': 'y', 'name': 'iu'}, {'value': None, 'name': 'au'}, {'value': None, 'name': 'al'}, {'value': None, 'name': 'ad'}, {'value': None, 'name': 'ao'}, {'value': 'y', 'name': 'aan'}, {'value': 'y', 'name': 'cu'}, {'value': 'y', 'name': 'cl'}, {'value': 'y', 'name': 'ca'}, {'value': None, 'name': 'cd'}, {'value': None, 'name': 'cp'}, {'value': None, 'name': 'cds'}, {'value': None, 'name': 'cdt'}, {'value': None, 'name': 'cs'}, {'value': None, 'name': 'bdash'}, {'value': None, 'name': 'adash'}, {'value': None, 'name': 'bdot'}, {'value': None, 'name': 'adot'}, {'value': '8', 'name': 'len'}, {'value': {'loc'}, 'name': 'gaz'}, {'value': Span('Eesti', [{'value': 'Eesti', 'name': 'lem'}, {'value': '_H_', 'name': 'pos'}, {'value': 'y', 'name': 'prop'}, {'value': None, 'name': 'pref'}, {'value': None, 'name': 'post'}, {'value': 'g', 'name': 'case'}, {'value': None, 'name': 'end'}, {'value': None, 'name': 'pun'}, {'value': 'Eesti', 'name': 'w'}, {'value': 'eesti', 'name': 'wl'}, {'value': 'ULLLL', 'name': 'shape'}, {'value': 'UL', 'name': 'shaped'}, {'value': 'E', 'name': 'p1'}, {'value': 'Ee', 'name': 'p2'}, {'value': 'Ees', 'name': 'p3'}, {'value': 'Eest', 'name': 'p4'}, {'value': 'i', 'name': 's1'}, {'value': 'ti', 'name': 's2'}, {'value': 'sti', 'name': 's3'}, {'value': 'esti', 'name': 's4'}, {'value': None, 'name': '2d'}, {'value': None, 'name': '4d'}, {'value': None, 'name': 'd&-'}, {'value': None, 'name': 'd&/'}, {'value': None, 'name': 'd&,'}, {'value': None, 'name': 'd&.'}, {'value': None, 'name': 'up'}, {'value': 'y', 'name': 'iu'}, {'value': None, 'name': 'au'}, {'value': None, 'name': 'al'}, {'value': None, 'name': 'ad'}, {'value': None, 'name': 'ao'}, {'value': 'y', 'name': 'aan'}, {'value': 'y', 'name': 'cu'}, {'value': 'y', 'name': 'cl'}, {'value': 'y', 'name': 'ca'}, {'value': None, 'name': 'cd'}, {'value': None, 'name': 'cp'}, {'value': None, 'name': 'cds'}, {'value': None, 'name': 'cdt'}, {'value': None, 'name': 'cs'}, {'value': None, 'name': 'bdash'}, {'value': None, 'name': 'adash'}, {'value': None, 'name': 'bdot'}, {'value': None, 'name': 'adot'}, {'value': '5', 'name': 'len'}, {'value': 'y', 'name': 'fsnt'}, {'value': {'loc'}, 'name': 'gaz'}, {'value': ..., 'name': 'next'}, {'value': ['iu[0]=y'], 'name': 'F'}, {'value': ['fsnt[0]=y'], 'name': 'F'}, {'value': ['p3[0]=Ees'], 'name': 'F'}, {'value': ['p4[0]=Eest'], 'name': 'F'}, {'value': ['s3[0]=sti'], 'name': 'F'}, {'value': ['s4[0]=esti'], 'name': 'F'}, {'value': [], 'name': 'F'}, {'value': ['aan[0]=y'], 'name': 'F'}, {'value': ['cu[0]=y'], 'name': 'F'}, {'value': ['iu[1]=y'], 'name': 'F'}, {'value': ['cu[1]=y'], 'name': 'F'}, {'value': ['lem[0]=Eesti'], 'name': 'F'}, {'value': ['pos[0]=_H_'], 'name': 'F'}, {'value': ['case[0]=g'], 'name': 'F'}, {'value': ['lem[1]=vabariik'], 'name': 'F'}, {'value': ['pos[1]=_S_'], 'name': 'F'}, {'value': ['post[1]=riik'], 'name': 'F'}, {'value': ['lem[2]=olema'], 'name': 'F'}, {'value': ['pos[2]=_V_'], 'name': 'F'}, {'value': ['gaz[0]=loc'], 'name': 'F'}, {'value': ['gaz[1]=loc'], 'name': 'F'}, {'value': ['iu[0]|fsnt[0]=y|y'], 'name': 'F'}, {'value': ['lem[0]|lem[1]=Eesti|vabariik'], 'name': 'F'}, {'value': ['pos[0]|pos[1]=_H_|_S_'], 'name': 'F'}, {'value': ['iu[0]|iu[1]=y|y'], 'name': 'F'}, {'value': ['gaz[0]|gaz[1]=loc|loc'], 'name': 'F'}]), 'name': 'prew'}, {'value': ..., 'name': 'next'}, {'value': 'y', 'name': 'iuoc'}, {'value': 'y', 'name': 'pprop'}, {'value': 'y', 'name': 'nprop'}, {'value': {'loc'}, 'name': 'pgaz'}, {'value': ['iu[0]=y'], 'name': 'F'}, {'value': ['p3[0]=vab'], 'name': 'F'}, {'value': ['p4[0]=vaba'], 'name': 'F'}, {'value': ['s3[0]=iik'], 'name': 'F'}, {'value': ['s4[0]=riik'], 'name': 'F'}, {'value': [], 'name': 'F'}, {'value': ['aan[0]=y'], 'name': 'F'}, {'value': ['cu[0]=y'], 'name': 'F'}, {'value': ['iu[-1]=y'], 'name': 'F'}, {'value': ['cu[-1]=y'], 'name': 'F'}, {'value': ['fsnt[-1]=y'], 'name': 'F'}, {'value': ['lem[0]=vabariik'], 'name': 'F'}, {'value': ['pos[0]=_S_'], 'name': 'F'}, {'value': ['pref[0]=vaba'], 'name': 'F'}, {'value': ['post[0]=riik'], 'name': 'F'}, {'value': ['case[0]=g'], 'name': 'F'}, {'value': ['lem[-1]=Eesti'], 'name': 'F'}, {'value': ['prop[-1]=y'], 'name': 'F'}, {'value': ['pos[-1]=_H_'], 'name': 'F'}, {'value': ['lem[1]=olema'], 'name': 'F'}, {'value': ['pos[1]=_V_'], 'name': 'F'}, {'value': ['lem[2]=riik'], 'name': 'F'}, {'value': ['pos[2]=_S_'], 'name': 'F'}, {'value': ['iuoc[0]=y'], 'name': 'F'}, {'value': ['nprop[0]=y'], 'name': 'F'}, {'value': ['pprop[0]=y'], 'name': 'F'}, {'value': ['pgaz[0]=loc'], 'name': 'F'}, {'value': ['gaz[0]=loc'], 'name': 'F'}, {'value': ['gaz[-1]=loc'], 'name': 'F'}, {'value': ['lem[0]|lem[-1]=vabariik|Eesti'], 'name': 'F'}, {'value': ['lem[0]|lem[1]=vabariik|olema'], 'name': 'F'}, {'value': ['pos[0]|pos[-1]=_S_|_H_'], 'name': 'F'}, {'value': ['pos[0]|pos[1]=_S_|_V_'], 'name': 'F'}, {'value': ['iu[0]|iu[-1]=y|y'], 'name': 'F'}, {'value': ['gaz[-1]|gaz[0]=loc|loc'], 'name': 'F'}]), 'name': 'prew'}, {'value': ..., 'name': 'next'}, {'value': 'y', 'name': 'iuoc'}, {'value': 'y', 'name': 'pprop'}, {'value': 'y', 'name': 'nprop'}, {'value': {'loc'}, 'name': 'pgaz'}, {'value': [], 'name': 'F'}, {'value': ['p3[0]=ole'], 'name': 'F'}, {'value': ['p4[0]=olem'], 'name': 'F'}, {'value': ['s3[0]=ema'], 'name': 'F'}, {'value': ['s4[0]=lema'], 'name': 'F'}, {'value': ['aan[0]=y'], 'name': 'F'}, {'value': ['iu[-1]=y'], 'name': 'F'}, {'value': ['cu[-1]=y'], 'name': 'F'}, {'value': ['iu[-2]=y'], 'name': 'F'}, {'value': ['cu[-2]=y'], 'name': 'F'}, {'value': ['fsnt[-2]=y'], 'name': 'F'}, {'value': ['iu[2]=y'], 'name': 'F'}, {'value': ['cu[2]=y'], 'name': 'F'}, {'value': ['cs[2]=y'], 'name': 'F'}, {'value': ['cds[2]=y'], 'name': 'F'}, {'value': ['adash[2]=Euroobas'], 'name': 'F'}, {'value': ['lem[0]=olema'], 'name': 'F'}, {'value': ['pos[0]=_V_'], 'name': 'F'}, {'value': ['lem[-1]=vabariik'], 'name': 'F'}, {'value': ['pos[-1]=_S_'], 'name': 'F'}, {'value': ['post[-1]=riik'], 'name': 'F'}, {'value': ['lem[-2]=Eesti'], 'name': 'F'}, {'value': ['prop[-2]=y'], 'name': 'F'}, {'value': ['pos[-2]=_H_'], 'name': 'F'}, {'value': ['lem[1]=riik'], 'name': 'F'}, {'value': ['pos[1]=_S_'], 'name': 'F'}, {'value': ['lem[2]=Põhja-Euroobas'], 'name': 'F'}, {'value': ['prop[2]=y'], 'name': 'F'}, {'value': ['pos[2]=_H_'], 'name': 'F'}, {'value': ['post[2]=Eu'], 'name': 'F'}, {'value': ['iuoc[0]=y'], 'name': 'F'}, {'value': ['nprop[0]=y'], 'name': 'F'}, {'value': ['pprop[0]=y'], 'name': 'F'}, {'value': ['pgaz[0]=loc'], 'name': 'F'}, {'value': ['gaz[-1]=loc'], 'name': 'F'}, {'value': ['gaz[-2]=loc'], 'name': 'F'}, {'value': ['lem[0]|lem[-1]=olema|vabariik'], 'name': 'F'}, {'value': ['lem[0]|lem[1]=olema|riik'], 'name': 'F'}, {'value': ['pos[0]|pos[-1]=_V_|_S_'], 'name': 'F'}, {'value': ['pos[0]|pos[1]=_V_|_S_'], 'name': 'F'}]), 'name': 'prew'}, {'value': ..., 'name': 'next'}, {'value': 'y', 'name': 'iuoc'}, {'value': 'y', 'name': 'pprop'}, {'value': 'y', 'name': 'nprop'}, {'value': [], 'name': 'F'}, {'value': ['p3[0]=rii'], 'name': 'F'}, {'value': ['p4[0]=riik'], 'name': 'F'}, {'value': ['s3[0]=iik'], 'name': 'F'}, {'value': ['s4[0]=riik'], 'name': 'F'}, {'value': ['aan[0]=y'], 'name': 'F'}, {'value': ['iu[-2]=y'], 'name': 'F'}, {'value': ['cu[-2]=y'], 'name': 'F'}, {'value': ['iu[1]=y'], 'name': 'F'}, {'value': ['cu[1]=y'], 'name': 'F'}, {'value': ['cs[1]=y'], 'name': 'F'}, {'value': ['cds[1]=y'], 'name': 'F'}, {'value': ['adash[1]=Euroobas'], 'name': 'F'}, {'value': ['cs[2]=y'], 'name': 'F'}, {'value': ['cdt[2]=y'], 'name': 'F'}, {'value': ['lem[0]=riik'], 'name': 'F'}, {'value': ['pos[0]=_S_'], 'name': 'F'}, {'value': ['case[0]=g'], 'name': 'F'}, {'value': ['lem[-1]=olema'], 'name': 'F'}, {'value': ['pos[-1]=_V_'], 'name': 'F'}, {'value': ['lem[-2]=vabariik'], 'name': 'F'}, {'value': ['pos[-2]=_S_'], 'name': 'F'}, {'value': ['post[-2]=riik'], 'name': 'F'}, {'value': ['lem[1]=Põhja-Euroobas'], 'name': 'F'}, {'value': ['prop[1]=y'], 'name': 'F'}, {'value': ['pos[1]=_H_'], 'name': 'F'}, {'value': ['post[1]=Eu'], 'name': 'F'}, {'value': ['lem[2]=.'], 'name': 'F'}, {'value': ['pos[2]=_Z_'], 'name': 'F'}, {'value': ['pun[2]=y'], 'name': 'F'}, {'value': ['iuoc[0]=y'], 'name': 'F'}, {'value': ['nprop[0]=y'], 'name': 'F'}, {'value': ['pprop[0]=y'], 'name': 'F'}, {'value': ['gaz[-2]=loc'], 'name': 'F'}, {'value': ['lem[0]|lem[-1]=riik|olema'], 'name': 'F'}, {'value': ['lem[0]|lem[1]=riik|Põhja-Euroobas'], 'name': 'F'}, {'value': ['pos[0]|pos[-1]=_S_|_V_'], 'name': 'F'}, {'value': ['pos[0]|pos[1]=_S_|_H_'], 'name': 'F'}]), 'name': 'prew'}, {'value': ..., 'name': 'next'}, {'value': 'y', 'name': 'iuoc'}, {'value': 'y', 'name': 'pprop'}, {'value': 'y', 'name': 'nprop'}, {'value': ['iu[0]=y'], 'name': 'F'}, {'value': ['p3[0]=Põh'], 'name': 'F'}, {'value': ['p4[0]=Põhj'], 'name': 'F'}, {'value': ['s3[0]=bas'], 'name': 'F'}, {'value': ['s4[0]=obas'], 'name': 'F'}, {'value': ['bdash[0]=Põhja'], 'name': 'F'}, {'value': ['adash[0]=Euroobas'], 'name': 'F'}, {'value': [], 'name': 'F'}, {'value': ['cu[0]=y'], 'name': 'F'}, {'value': ['cs[0]=y'], 'name': 'F'}, {'value': ['cds[0]=y'], 'name': 'F'}, {'value': ['cs[1]=y'], 'name': 'F'}, {'value': ['cdt[1]=y'], 'name': 'F'}, {'value': ['iu[2]=y'], 'name': 'F'}, {'value': ['cu[2]=y'], 'name': 'F'}, {'value': ['fsnt[2]=y'], 'name': 'F'}, {'value': ['lem[0]=Põhja-Euroobas'], 'name': 'F'}, {'value': ['pos[0]=_H_'], 'name': 'F'}, {'value': ['pref[0]=Põhja'], 'name': 'F'}, {'value': ['post[0]=Eu'], 'name': 'F'}, {'value': ['case[0]=g'], 'name': 'F'}, {'value': ['end[0]=s'], 'name': 'F'}, {'value': ['lem[-1]=riik'], 'name': 'F'}, {'value': ['pos[-1]=_S_'], 'name': 'F'}, {'value': ['lem[-2]=olema'], 'name': 'F'}, {'value': ['pos[-2]=_V_'], 'name': 'F'}, {'value': ['lem[1]=.'], 'name': 'F'}, {'value': ['pos[1]=_Z_'], 'name': 'F'}, {'value': ['pun[1]=y'], 'name': 'F'}, {'value': ['lem[2]=Eesti'], 'name': 'F'}, {'value': ['prop[2]=y'], 'name': 'F'}, {'value': ['pos[2]=_H_'], 'name': 'F'}, {'value': ['iuoc[0]=y'], 'name': 'F'}, {'value': ['nprop[0]=y'], 'name': 'F'}, {'value': ['pprop[0]=y'], 'name': 'F'}, {'value': ['gaz[2]=loc'], 'name': 'F'}, {'value': ['lem[0]|lem[-1]=Põhja-Euroobas|riik'], 'name': 'F'}, {'value': ['lem[0]|lem[1]=Põhja-Euroobas|.'], 'name': 'F'}, {'value': ['pos[0]|pos[-1]=_H_|_S_'], 'name': 'F'}, {'value': ['pos[0]|pos[1]=_H_|_Z_'], 'name': 'F'}]), 'name': 'prew'}, {'value': ..., 'name': 'next'}, {'value': 'y', 'name': 'iuoc'}, {'value': 'y', 'name': 'pprop'}, {'value': 'y', 'name': 'nprop'}, {'value': {'loc'}, 'name': 'pgaz'}, {'value': {'loc'}, 'name': 'ngaz'}, {'value': [], 'name': 'F'}, {'value': ['ao[0]=y'], 'name': 'F'}, {'value': ['cs[0]=y'], 'name': 'F'}, {'value': ['cdt[0]=y'], 'name': 'F'}, {'value': ['iu[-1]=y'], 'name': 'F'}, {'value': ['cu[-1]=y'], 'name': 'F'}, {'value': ['cs[-1]=y'], 'name': 'F'}, {'value': ['cds[-1]=y'], 'name': 'F'}, {'value': ['adash[-1]=Euroobas'], 'name': 'F'}, {'value': ['iu[1]=y'], 'name': 'F'}, {'value': ['cu[1]=y'], 'name': 'F'}, {'value': ['fsnt[1]=y'], 'name': 'F'}, {'value': ['lem[0]=.'], 'name': 'F'}, {'value': ['pos[0]=_Z_'], 'name': 'F'}, {'value': ['pun[0]=y'], 'name': 'F'}, {'value': ['end[0]='], 'name': 'F'}, {'value': ['lem[-1]=Põhja-Euroobas'], 'name': 'F'}, {'value': ['prop[-1]=y'], 'name': 'F'}, {'value': ['pos[-1]=_H_'], 'name': 'F'}, {'value': ['post[-1]=Eu'], 'name': 'F'}, {'value': ['lem[-2]=riik'], 'name': 'F'}, {'value': ['pos[-2]=_S_'], 'name': 'F'}, {'value': ['lem[1]=Eesti'], 'name': 'F'}, {'value': ['prop[1]=y'], 'name': 'F'}, {'value': ['pos[1]=_H_'], 'name': 'F'}, {'value': ['lem[2]=piirnema'], 'name': 'F'}, {'value': ['pos[2]=_V_'], 'name': 'F'}, {'value': ['iuoc[0]=y'], 'name': 'F'}, {'value': ['nprop[0]=y'], 'name': 'F'}, {'value': ['pprop[0]=y'], 'name': 'F'}, {'value': ['ngaz[0]=loc'], 'name': 'F'}, {'value': ['pgaz[0]=loc'], 'name': 'F'}, {'value': ['gaz[1]=loc'], 'name': 'F'}, {'value': ['lem[0]|lem[-1]=.|Põhja-Euroobas'], 'name': 'F'}, {'value': ['lem[0]|lem[1]=.|Eesti'], 'name': 'F'}, {'value': ['pos[0]|pos[-1]=_Z_|_H_'], 'name': 'F'}, {'value': ['pos[0]|pos[1]=_Z_|_H_'], 'name': 'F'}]), 'name': 'prew'}, {'value': ..., 'name': 'next'}, {'value': ['iu[0]=y'], 'name': 'F'}, {'value': ['fsnt[0]=y'], 'name': 'F'}, {'value': ['p3[0]=Ees'], 'name': 'F'}, {'value': ['p4[0]=Eest'], 'name': 'F'}, {'value': ['s3[0]=sti'], 'name': 'F'}, {'value': ['s4[0]=esti'], 'name': 'F'}, {'value': [], 'name': 'F'}, {'value': ['aan[0]=y'], 'name': 'F'}, {'value': ['cu[0]=y'], 'name': 'F'}, {'value': ['cs[-1]=y'], 'name': 'F'}, {'value': ['cdt[-1]=y'], 'name': 'F'}, {'value': ['iu[-2]=y'], 'name': 'F'}, {'value': ['cu[-2]=y'], 'name': 'F'}, {'value': ['cs[-2]=y'], 'name': 'F'}, {'value': ['cds[-2]=y'], 'name': 'F'}, {'value': ['adash[-2]=Euroobas'], 'name': 'F'}, {'value': ['lem[0]=Eesti'], 'name': 'F'}, {'value': ['pos[0]=_H_'], 'name': 'F'}, {'value': ['case[0]=g'], 'name': 'F'}, {'value': ['lem[-1]=.'], 'name': 'F'}, {'value': ['pos[-1]=_Z_'], 'name': 'F'}, {'value': ['pun[-1]=y'], 'name': 'F'}, {'value': ['lem[-2]=Põhja-Euroobas'], 'name': 'F'}, {'value': ['prop[-2]=y'], 'name': 'F'}, {'value': ['pos[-2]=_H_'], 'name': 'F'}, {'value': ['post[-2]=Eu'], 'name': 'F'}, {'value': ['lem[1]=piirnema'], 'name': 'F'}, {'value': ['pos[1]=_V_'], 'name': 'F'}, {'value': ['lem[2]=põhi'], 'name': 'F'}, {'value': ['pos[2]=_S_'], 'name': 'F'}, {'value': ['gaz[0]=loc'], 'name': 'F'}, {'value': ['iu[0]|fsnt[0]=y|y'], 'name': 'F'}, {'value': ['lem[0]|lem[-1]=Eesti|.'], 'name': 'F'}, {'value': ['lem[0]|lem[1]=Eesti|piirnema'], 'name': 'F'}, {'value': ['pos[0]|pos[-1]=_H_|_Z_'], 'name': 'F'}, {'value': ['pos[0]|pos[1]=_H_|_V_'], 'name': 'F'}]), 'name': 'prew'}, {'value': Span('põhjas', [{'value': 'põhi', 'name': 'lem'}, {'value': '_S_', 'name': 'pos'}, {'value': None, 'name': 'prop'}, {'value': None, 'name': 'pref'}, {'value': None, 'name': 'post'}, {'value': 'g', 'name': 'case'}, {'value': 's', 'name': 'end'}, {'value': None, 'name': 'pun'}, {'value': 'põhjas', 'name': 'w'}, {'value': 'põhjas', 'name': 'wl'}, {'value': 'LLLLLL', 'name': 'shape'}, {'value': 'L', 'name': 'shaped'}, {'value': 'p', 'name': 'p1'}, {'value': 'põ', 'name': 'p2'}, {'value': 'põh', 'name': 'p3'}, {'value': 'põhi', 'name': 'p4'}, {'value': 'i', 'name': 's1'}, {'value': 'hi', 'name': 's2'}, {'value': 'õhi', 'name': 's3'}, {'value': 'põhi', 'name': 's4'}, {'value': None, 'name': '2d'}, {'value': None, 'name': '4d'}, {'value': None, 'name': 'd&-'}, {'value': None, 'name': 'd&/'}, {'value': None, 'name': 'd&,'}, {'value': None, 'name': 'd&.'}, {'value': None, 'name': 'up'}, {'value': None, 'name': 'iu'}, {'value': None, 'name': 'au'}, {'value': 'y', 'name': 'al'}, {'value': None, 'name': 'ad'}, {'value': None, 'name': 'ao'}, {'value': 'y', 'name': 'aan'}, {'value': None, 'name': 'cu'}, {'value': 'y', 'name': 'cl'}, {'value': 'y', 'name': 'ca'}, {'value': None, 'name': 'cd'}, {'value': None, 'name': 'cp'}, {'value': None, 'name': 'cds'}, {'value': None, 'name': 'cdt'}, {'value': None, 'name': 'cs'}, {'value': None, 'name': 'bdash'}, {'value': None, 'name': 'adash'}, {'value': None, 'name': 'bdot'}, {'value': None, 'name': 'adot'}, {'value': '4', 'name': 'len'}, {'value': ..., 'name': 'prew'}, {'value': Span('üle', [{'value': 'üle', 'name': 'lem'}, {'value': '_D_', 'name': 'pos'}, {'value': None, 'name': 'prop'}, {'value': None, 'name': 'pref'}, {'value': None, 'name': 'post'}, {'value': None, 'name': 'case'}, {'value': None, 'name': 'end'}, {'value': None, 'name': 'pun'}, {'value': 'üle', 'name': 'w'}, {'value': 'üle', 'name': 'wl'}, {'value': 'LLL', 'name': 'shape'}, {'value': 'L', 'name': 'shaped'}, {'value': 'ü', 'name': 'p1'}, {'value': 'ül', 'name': 'p2'}, {'value': 'üle', 'name': 'p3'}, {'value': None, 'name': 'p4'}, {'value': 'e', 'name': 's1'}, {'value': 'le', 'name': 's2'}, {'value': 'üle', 'name': 's3'}, {'value': None, 'name': 's4'}, {'value': None, 'name': '2d'}, {'value': None, 'name': '4d'}, {'value': None, 'name': 'd&-'}, {'value': None, 'name': 'd&/'}, {'value': None, 'name': 'd&,'}, {'value': None, 'name': 'd&.'}, {'value': None, 'name': 'up'}, {'value': None, 'name': 'iu'}, {'value': None, 'name': 'au'}, {'value': 'y', 'name': 'al'}, {'value': None, 'name': 'ad'}, {'value': None, 'name': 'ao'}, {'value': 'y', 'name': 'aan'}, {'value': None, 'name': 'cu'}, {'value': 'y', 'name': 'cl'}, {'value': 'y', 'name': 'ca'}, {'value': None, 'name': 'cd'}, {'value': None, 'name': 'cp'}, {'value': None, 'name': 'cds'}, {'value': None, 'name': 'cdt'}, {'value': None, 'name': 'cs'}, {'value': None, 'name': 'bdash'}, {'value': None, 'name': 'adash'}, {'value': None, 'name': 'bdot'}, {'value': None, 'name': 'adot'}, {'value': '3', 'name': 'len'}, {'value': ..., 'name': 'prew'}, {'value': Span('Soome', [{'value': 'Soome', 'name': 'lem'}, {'value': '_H_', 'name': 'pos'}, {'value': 'y', 'name': 'prop'}, {'value': None, 'name': 'pref'}, {'value': None, 'name': 'post'}, {'value': 'g', 'name': 'case'}, {'value': None, 'name': 'end'}, {'value': None, 'name': 'pun'}, {'value': 'Soome', 'name': 'w'}, {'value': 'soome', 'name': 'wl'}, {'value': 'ULLLL', 'name': 'shape'}, {'value': 'UL', 'name': 'shaped'}, {'value': 'S', 'name': 'p1'}, {'value': 'So', 'name': 'p2'}, {'value': 'Soo', 'name': 'p3'}, {'value': 'Soom', 'name': 'p4'}, {'value': 'e', 'name': 's1'}, {'value': 'me', 'name': 's2'}, {'value': 'ome', 'name': 's3'}, {'value': 'oome', 'name': 's4'}, {'value': None, 'name': '2d'}, {'value': None, 'name': '4d'}, {'value': None, 'name': 'd&-'}, {'value': None, 'name': 'd&/'}, {'value': None, 'name': 'd&,'}, {'value': None, 'name': 'd&.'}, {'value': None, 'name': 'up'}, {'value': 'y', 'name': 'iu'}, {'value': None, 'name': 'au'}, {'value': None, 'name': 'al'}, {'value': None, 'name': 'ad'}, {'value': None, 'name': 'ao'}, {'value': 'y', 'name': 'aan'}, {'value': 'y', 'name': 'cu'}, {'value': 'y', 'name': 'cl'}, {'value': 'y', 'name': 'ca'}, {'value': None, 'name': 'cd'}, {'value': None, 'name': 'cp'}, {'value': None, 'name': 'cds'}, {'value': None, 'name': 'cdt'}, {'value': None, 'name': 'cs'}, {'value': None, 'name': 'bdash'}, {'value': None, 'name': 'adash'}, {'value': None, 'name': 'bdot'}, {'value': None, 'name': 'adot'}, {'value': '5', 'name': 'len'}, {'value': {'loc'}, 'name': 'gaz'}, {'value': ..., 'name': 'prew'}, {'value': Span('lahe', [{'value': 'laht', 'name': 'lem'}, {'value': '_S_', 'name': 'pos'}, {'value': None, 'name': 'prop'}, {'value': None, 'name': 'pref'}, {'value': None, 'name': 'post'}, {'value': 'g', 'name': 'case'}, {'value': None, 'name': 'end'}, {'value': None, 'name': 'pun'}, {'value': 'lahe', 'name': 'w'}, {'value': 'lahe', 'name': 'wl'}, {'value': 'LLLL', 'name': 'shape'}, {'value': 'L', 'name': 'shaped'}, {'value': 'l', 'name': 'p1'}, {'value': 'la', 'name': 'p2'}, {'value': 'lah', 'name': 'p3'}, {'value': 'laht', 'name': 'p4'}, {'value': 't', 'name': 's1'}, {'value': 'ht', 'name': 's2'}, {'value': 'aht', 'name': 's3'}, {'value': 'laht', 'name': 's4'}, {'value': None, 'name': '2d'}, {'value': None, 'name': '4d'}, {'value': None, 'name': 'd&-'}, {'value': None, 'name': 'd&/'}, {'value': None, 'name': 'd&,'}, {'value': None, 'name': 'd&.'}, {'value': None, 'name': 'up'}, {'value': None, 'name': 'iu'}, {'value': None, 'name': 'au'}, {'value': 'y', 'name': 'al'}, {'value': None, 'name': 'ad'}, {'value': None, 'name': 'ao'}, {'value': 'y', 'name': 'aan'}, {'value': None, 'name': 'cu'}, {'value': 'y', 'name': 'cl'}, {'value': 'y', 'name': 'ca'}, {'value': None, 'name': 'cd'}, {'value': None, 'name': 'cp'}, {'value': None, 'name': 'cds'}, {'value': None, 'name': 'cdt'}, {'value': None, 'name': 'cs'}, {'value': None, 'name': 'bdash'}, {'value': None, 'name': 'adash'}, {'value': None, 'name': 'bdot'}, {'value': None, 'name': 'adot'}, {'value': '4', 'name': 'len'}, {'value': {'loc'}, 'name': 'gaz'}, {'value': ..., 'name': 'prew'}, {'value': Span('Soome', [{'value': 'Soome', 'name': 'lem'}, {'value': '_H_', 'name': 'pos'}, {'value': 'y', 'name': 'prop'}, {'value': None, 'name': 'pref'}, {'value': None, 'name': 'post'}, {'value': 'g', 'name': 'case'}, {'value': None, 'name': 'end'}, {'value': None, 'name': 'pun'}, {'value': 'Soome', 'name': 'w'}, {'value': 'soome', 'name': 'wl'}, {'value': 'ULLLL', 'name': 'shape'}, {'value': 'UL', 'name': 'shaped'}, {'value': 'S', 'name': 'p1'}, {'value': 'So', 'name': 'p2'}, {'value': 'Soo', 'name': 'p3'}, {'value': 'Soom', 'name': 'p4'}, {'value': 'e', 'name': 's1'}, {'value': 'me', 'name': 's2'}, {'value': 'ome', 'name': 's3'}, {'value': 'oome', 'name': 's4'}, {'value': None, 'name': '2d'}, {'value': None, 'name': '4d'}, {'value': None, 'name': 'd&-'}, {'value': None, 'name': 'd&/'}, {'value': None, 'name': 'd&,'}, {'value': None, 'name': 'd&.'}, {'value': None, 'name': 'up'}, {'value': 'y', 'name': 'iu'}, {'value': None, 'name': 'au'}, {'value': None, 'name': 'al'}, {'value': None, 'name': 'ad'}, {'value': None, 'name': 'ao'}, {'value': 'y', 'name': 'aan'}, {'value': 'y', 'name': 'cu'}, {'value': 'y', 'name': 'cl'}, {'value': 'y', 'name': 'ca'}, {'value': None, 'name': 'cd'}, {'value': None, 'name': 'cp'}, {'value': None, 'name': 'cds'}, {'value': None, 'name': 'cdt'}, {'value': None, 'name': 'cs'}, {'value': None, 'name': 'bdash'}, {'value': None, 'name': 'adash'}, {'value': None, 'name': 'bdot'}, {'value': None, 'name': 'adot'}, {'value': '5', 'name': 'len'}, {'value': {'loc'}, 'name': 'gaz'}, {'value': ..., 'name': 'prew'}, {'value': Span('Vabariigiga', [{'value': 'Vabariik', 'name': 'lem'}, {'value': '_H_', 'name': 'pos'}, {'value': 'y', 'name': 'prop'}, {'value': 'Vaba', 'name': 'pref'}, {'value': 'riik', 'name': 'post'}, {'value': 'g', 'name': 'case'}, {'value': 'ga', 'name': 'end'}, {'value': None, 'name': 'pun'}, {'value': 'Vabariigiga', 'name': 'w'}, {'value': 'vabariigiga', 'name': 'wl'}, {'value': 'ULLLLLLLLLL', 'name': 'shape'}, {'value': 'UL', 'name': 'shaped'}, {'value': 'V', 'name': 'p1'}, {'value': 'Va', 'name': 'p2'}, {'value': 'Vab', 'name': 'p3'}, {'value': 'Vaba', 'name': 'p4'}, {'value': 'k', 'name': 's1'}, {'value': 'ik', 'name': 's2'}, {'value': 'iik', 'name': 's3'}, {'value': 'riik', 'name': 's4'}, {'value': None, 'name': '2d'}, {'value': None, 'name': '4d'}, {'value': None, 'name': 'd&-'}, {'value': None, 'name': 'd&/'}, {'value': None, 'name': 'd&,'}, {'value': None, 'name': 'd&.'}, {'value': None, 'name': 'up'}, {'value': 'y', 'name': 'iu'}, {'value': None, 'name': 'au'}, {'value': None, 'name': 'al'}, {'value': None, 'name': 'ad'}, {'value': None, 'name': 'ao'}, {'value': 'y', 'name': 'aan'}, {'value': 'y', 'name': 'cu'}, {'value': 'y', 'name': 'cl'}, {'value': 'y', 'name': 'ca'}, {'value': None, 'name': 'cd'}, {'value': None, 'name': 'cp'}, {'value': None, 'name': 'cds'}, {'value': None, 'name': 'cdt'}, {'value': None, 'name': 'cs'}, {'value': None, 'name': 'bdash'}, {'value': None, 'name': 'adash'}, {'value': None, 'name': 'bdot'}, {'value': None, 'name': 'adot'}, {'value': '8', 'name': 'len'}, {'value': {'loc'}, 'name': 'gaz'}, {'value': ..., 'name': 'prew'}, {'value': Span('.', [{'value': '.', 'name': 'lem'}, {'value': '_Z_', 'name': 'pos'}, {'value': None, 'name': 'prop'}, {'value': None, 'name': 'pref'}, {'value': None, 'name': 'post'}, {'value': None, 'name': 'case'}, {'value': '', 'name': 'end'}, {'value': 'y', 'name': 'pun'}, {'value': '.', 'name': 'w'}, {'value': '.', 'name': 'wl'}, {'value': '.', 'name': 'shape'}, {'value': '.', 'name': 'shaped'}, {'value': '.', 'name': 'p1'}, {'value': None, 'name': 'p2'}, {'value': None, 'name': 'p3'}, {'value': None, 'name': 'p4'}, {'value': '.', 'name': 's1'}, {'value': None, 'name': 's2'}, {'value': None, 'name': 's3'}, {'value': None, 'name': 's4'}, {'value': None, 'name': '2d'}, {'value': None, 'name': '4d'}, {'value': None, 'name': 'd&-'}, {'value': None, 'name': 'd&/'}, {'value': None, 'name': 'd&,'}, {'value': None, 'name': 'd&.'}, {'value': None, 'name': 'up'}, {'value': None, 'name': 'iu'}, {'value': None, 'name': 'au'}, {'value': None, 'name': 'al'}, {'value': None, 'name': 'ad'}, {'value': 'y', 'name': 'ao'}, {'value': None, 'name': 'aan'}, {'value': None, 'name': 'cu'}, {'value': None, 'name': 'cl'}, {'value': None, 'name': 'ca'}, {'value': None, 'name': 'cd'}, {'value': None, 'name': 'cp'}, {'value': None, 'name': 'cds'}, {'value': 'y', 'name': 'cdt'}, {'value': 'y', 'name': 'cs'}, {'value': None, 'name': 'bdash'}, {'value': None, 'name': 'adash'}, {'value': '', 'name': 'bdot'}, {'value': '', 'name': 'adot'}, {'value': '1', 'name': 'len'}, {'value': 'y', 'name': 'lsnt'}, {'value': ..., 'name': 'prew'}, {'value': 'y', 'name': 'iuoc'}, {'value': 'y', 'name': 'pprop'}, {'value': 'y', 'name': 'nprop'}, {'value': {'loc'}, 'name': 'pgaz'}, {'value': {'loc'}, 'name': 'ngaz'}, {'value': [], 'name': 'F'}, {'value': ['ao[0]=y'], 'name': 'F'}, {'value': ['cs[0]=y'], 'name': 'F'}, {'value': ['cdt[0]=y'], 'name': 'F'}, {'value': ['iu[-1]=y'], 'name': 'F'}, {'value': ['cu[-1]=y'], 'name': 'F'}, {'value': ['iu[-2]=y'], 'name': 'F'}, {'value': ['cu[-2]=y'], 'name': 'F'}, {'value': ['lem[0]=.'], 'name': 'F'}, {'value': ['pos[0]=_Z_'], 'name': 'F'}, {'value': ['pun[0]=y'], 'name': 'F'}, {'value': ['end[0]='], 'name': 'F'}, {'value': ['lem[-1]=Vabariik'], 'name': 'F'}, {'value': ['prop[-1]=y'], 'name': 'F'}, {'value': ['pos[-1]=_H_'], 'name': 'F'}, {'value': ['post[-1]=riik'], 'name': 'F'}, {'value': ['lem[-2]=Soome'], 'name': 'F'}, {'value': ['prop[-2]=y'], 'name': 'F'}, {'value': ['pos[-2]=_H_'], 'name': 'F'}, {'value': ['iuoc[0]=y'], 'name': 'F'}, {'value': ['nprop[0]=y'], 'name': 'F'}, {'value': ['pprop[0]=y'], 'name': 'F'}, {'value': ['ngaz[0]=loc'], 'name': 'F'}, {'value': ['pgaz[0]=loc'], 'name': 'F'}, {'value': ['gaz[-1]=loc'], 'name': 'F'}, {'value': ['gaz[-2]=loc'], 'name': 'F'}, {'value': ['lem[0]|lem[-1]=.|Vabariik'], 'name': 'F'}, {'value': ['pos[0]|pos[-1]=_Z_|_H_'], 'name': 'F'}]), 'name': 'next'}, {'value': 'y', 'name': 'iuoc'}, {'value': 'y', 'name': 'pprop'}, {'value': 'y', 'name': 'nprop'}, {'value': {'loc'}, 'name': 'pgaz'}, {'value': ['iu[0]=y'], 'name': 'F'}, {'value': ['p3[0]=Vab'], 'name': 'F'}, {'value': ['p4[0]=Vaba'], 'name': 'F'}, {'value': ['s3[0]=iik'], 'name': 'F'}, {'value': ['s4[0]=riik'], 'name': 'F'}, {'value': [], 'name': 'F'}, {'value': ['aan[0]=y'], 'name': 'F'}, {'value': ['cu[0]=y'], 'name': 'F'}, {'value': ['iu[-1]=y'], 'name': 'F'}, {'value': ['cu[-1]=y'], 'name': 'F'}, {'value': ['cs[1]=y'], 'name': 'F'}, {'value': ['cdt[1]=y'], 'name': 'F'}, {'value': ['lem[0]=Vabariik'], 'name': 'F'}, {'value': ['pos[0]=_H_'], 'name': 'F'}, {'value': ['pref[0]=Vaba'], 'name': 'F'}, {'value': ['post[0]=riik'], 'name': 'F'}, {'value': ['case[0]=g'], 'name': 'F'}, {'value': ['end[0]=ga'], 'name': 'F'}, {'value': ['lem[-1]=Soome'], 'name': 'F'}, {'value': ['prop[-1]=y'], 'name': 'F'}, {'value': ['pos[-1]=_H_'], 'name': 'F'}, {'value': ['lem[-2]=laht'], 'name': 'F'}, {'value': ['pos[-2]=_S_'], 'name': 'F'}, {'value': ['lem[1]=.'], 'name': 'F'}, {'value': ['pos[1]=_Z_'], 'name': 'F'}, {'value': ['pun[1]=y'], 'name': 'F'}, {'value': ['iuoc[0]=y'], 'name': 'F'}, {'value': ['nprop[0]=y'], 'name': 'F'}, {'value': ['pprop[0]=y'], 'name': 'F'}, {'value': ['pgaz[0]=loc'], 'name': 'F'}, {'value': ['gaz[0]=loc'], 'name': 'F'}, {'value': ['gaz[-1]=loc'], 'name': 'F'}, {'value': ['gaz[-2]=loc'], 'name': 'F'}, {'value': ['lem[0]|lem[-1]=Vabariik|Soome'], 'name': 'F'}, {'value': ['lem[0]|lem[1]=Vabariik|.'], 'name': 'F'}, {'value': ['pos[0]|pos[-1]=_H_|_H_'], 'name': 'F'}, {'value': ['pos[0]|pos[1]=_H_|_Z_'], 'name': 'F'}, {'value': ['iu[0]|iu[-1]=y|y'], 'name': 'F'}, {'value': ['gaz[-1]|gaz[0]=loc|loc'], 'name': 'F'}]), 'name': 'next'}, {'value': 'y', 'name': 'iuoc'}, {'value': 'y', 'name': 'pprop'}, {'value': 'y', 'name': 'nprop'}, {'value': {'loc'}, 'name': 'pgaz'}, {'value': {'loc'}, 'name': 'ngaz'}, {'value': ['iu[0]=y'], 'name': 'F'}, {'value': ['p3[0]=Soo'], 'name': 'F'}, {'value': ['p4[0]=Soom'], 'name': 'F'}, {'value': ['s3[0]=ome'], 'name': 'F'}, {'value': ['s4[0]=oome'], 'name': 'F'}, {'value': [], 'name': 'F'}, {'value': ['aan[0]=y'], 'name': 'F'}, {'value': ['cu[0]=y'], 'name': 'F'}, {'value': ['iu[-2]=y'], 'name': 'F'}, {'value': ['cu[-2]=y'], 'name': 'F'}, {'value': ['iu[1]=y'], 'name': 'F'}, {'value': ['cu[1]=y'], 'name': 'F'}, {'value': ['cs[2]=y'], 'name': 'F'}, {'value': ['cdt[2]=y'], 'name': 'F'}, {'value': ['lem[0]=Soome'], 'name': 'F'}, {'value': ['pos[0]=_H_'], 'name': 'F'}, {'value': ['case[0]=g'], 'name': 'F'}, {'value': ['lem[-1]=laht'], 'name': 'F'}, {'value': ['pos[-1]=_S_'], 'name': 'F'}, {'value': ['lem[-2]=Soome'], 'name': 'F'}, {'value': ['prop[-2]=y'], 'name': 'F'}, {'value': ['pos[-2]=_H_'], 'name': 'F'}, {'value': ['lem[1]=Vabariik'], 'name': 'F'}, {'value': ['prop[1]=y'], 'name': 'F'}, {'value': ['pos[1]=_H_'], 'name': 'F'}, {'value': ['post[1]=riik'], 'name': 'F'}, {'value': ['lem[2]=.'], 'name': 'F'}, {'value': ['pos[2]=_Z_'], 'name': 'F'}, {'value': ['pun[2]=y'], 'name': 'F'}, {'value': ['iuoc[0]=y'], 'name': 'F'}, {'value': ['nprop[0]=y'], 'name': 'F'}, {'value': ['pprop[0]=y'], 'name': 'F'}, {'value': ['ngaz[0]=loc'], 'name': 'F'}, {'value': ['pgaz[0]=loc'], 'name': 'F'}, {'value': ['gaz[0]=loc'], 'name': 'F'}, {'value': ['gaz[-1]=loc'], 'name': 'F'}, {'value': ['gaz[-2]=loc'], 'name': 'F'}, {'value': ['gaz[1]=loc'], 'name': 'F'}, {'value': ['lem[0]|lem[-1]=Soome|laht'], 'name': 'F'}, {'value': ['lem[0]|lem[1]=Soome|Vabariik'], 'name': 'F'}, {'value': ['pos[0]|pos[-1]=_H_|_S_'], 'name': 'F'}, {'value': ['pos[0]|pos[1]=_H_|_H_'], 'name': 'F'}, {'value': ['iu[0]|iu[1]=y|y'], 'name': 'F'}, {'value': ['gaz[0]|gaz[1]=loc|loc'], 'name': 'F'}, {'value': ['gaz[-1]|gaz[0]=loc|loc'], 'name': 'F'}]), 'name': 'next'}, {'value': 'y', 'name': 'iuoc'}, {'value': 'y', 'name': 'pprop'}, {'value': 'y', 'name': 'nprop'}, {'value': {'loc'}, 'name': 'pgaz'}, {'value': {'loc'}, 'name': 'ngaz'}, {'value': [], 'name': 'F'}, {'value': ['p3[0]=lah'], 'name': 'F'}, {'value': ['p4[0]=laht'], 'name': 'F'}, {'value': ['s3[0]=aht'], 'name': 'F'}, {'value': ['s4[0]=laht'], 'name': 'F'}, {'value': ['aan[0]=y'], 'name': 'F'}, {'value': ['iu[-1]=y'], 'name': 'F'}, {'value': ['cu[-1]=y'], 'name': 'F'}, {'value': ['iu[1]=y'], 'name': 'F'}, {'value': ['cu[1]=y'], 'name': 'F'}, {'value': ['iu[2]=y'], 'name': 'F'}, {'value': ['cu[2]=y'], 'name': 'F'}, {'value': ['lem[0]=laht'], 'name': 'F'}, {'value': ['pos[0]=_S_'], 'name': 'F'}, {'value': ['case[0]=g'], 'name': 'F'}, {'value': ['lem[-1]=Soome'], 'name': 'F'}, {'value': ['prop[-1]=y'], 'name': 'F'}, {'value': ['pos[-1]=_H_'], 'name': 'F'}, {'value': ['lem[-2]=üle'], 'name': 'F'}, {'value': ['pos[-2]=_D_'], 'name': 'F'}, {'value': ['lem[1]=Soome'], 'name': 'F'}, {'value': ['prop[1]=y'], 'name': 'F'}, {'value': ['pos[1]=_H_'], 'name': 'F'}, {'value': ['lem[2]=Vabariik'], 'name': 'F'}, {'value': ['prop[2]=y'], 'name': 'F'}, {'value': ['pos[2]=_H_'], 'name': 'F'}, {'value': ['post[2]=riik'], 'name': 'F'}, {'value': ['iuoc[0]=y'], 'name': 'F'}, {'value': ['nprop[0]=y'], 'name': 'F'}, {'value': ['pprop[0]=y'], 'name': 'F'}, {'value': ['ngaz[0]=loc'], 'name': 'F'}, {'value': ['pgaz[0]=loc'], 'name': 'F'}, {'value': ['gaz[0]=loc'], 'name': 'F'}, {'value': ['gaz[-1]=loc'], 'name': 'F'}, {'value': ['gaz[1]=loc'], 'name': 'F'}, {'value': ['gaz[2]=loc'], 'name': 'F'}, {'value': ['lem[0]|lem[-1]=laht|Soome'], 'name': 'F'}, {'value': ['lem[0]|lem[1]=laht|Soome'], 'name': 'F'}, {'value': ['pos[0]|pos[-1]=_S_|_H_'], 'name': 'F'}, {'value': ['pos[0]|pos[1]=_S_|_H_'], 'name': 'F'}, {'value': ['gaz[0]|gaz[1]=loc|loc'], 'name': 'F'}, {'value': ['gaz[-1]|gaz[0]=loc|loc'], 'name': 'F'}]), 'name': 'next'}, {'value': 'y', 'name': 'iuoc'}, {'value': 'y', 'name': 'pprop'}, {'value': 'y', 'name': 'nprop'}, {'value': {'loc'}, 'name': 'pgaz'}, {'value': {'loc'}, 'name': 'ngaz'}, {'value': ['iu[0]=y'], 'name': 'F'}, {'value': ['p3[0]=Soo'], 'name': 'F'}, {'value': ['p4[0]=Soom'], 'name': 'F'}, {'value': ['s3[0]=ome'], 'name': 'F'}, {'value': ['s4[0]=oome'], 'name': 'F'}, {'value': [], 'name': 'F'}, {'value': ['aan[0]=y'], 'name': 'F'}, {'value': ['cu[0]=y'], 'name': 'F'}, {'value': ['iu[2]=y'], 'name': 'F'}, {'value': ['cu[2]=y'], 'name': 'F'}, {'value': ['lem[0]=Soome'], 'name': 'F'}, {'value': ['pos[0]=_H_'], 'name': 'F'}, {'value': ['case[0]=g'], 'name': 'F'}, {'value': ['lem[-1]=üle'], 'name': 'F'}, {'value': ['pos[-1]=_D_'], 'name': 'F'}, {'value': ['lem[-2]=põhi'], 'name': 'F'}, {'value': ['pos[-2]=_S_'], 'name': 'F'}, {'value': ['lem[1]=laht'], 'name': 'F'}, {'value': ['pos[1]=_S_'], 'name': 'F'}, {'value': ['lem[2]=Soome'], 'name': 'F'}, {'value': ['prop[2]=y'], 'name': 'F'}, {'value': ['pos[2]=_H_'], 'name': 'F'}, {'value': ['iuoc[0]=y'], 'name': 'F'}, {'value': ['nprop[0]=y'], 'name': 'F'}, {'value': ['pprop[0]=y'], 'name': 'F'}, {'value': ['ngaz[0]=loc'], 'name': 'F'}, {'value': ['pgaz[0]=loc'], 'name': 'F'}, {'value': ['gaz[0]=loc'], 'name': 'F'}, {'value': ['gaz[1]=loc'], 'name': 'F'}, {'value': ['gaz[2]=loc'], 'name': 'F'}, {'value': ['lem[0]|lem[-1]=Soome|üle'], 'name': 'F'}, {'value': ['lem[0]|lem[1]=Soome|laht'], 'name': 'F'}, {'value': ['pos[0]|pos[-1]=_H_|_D_'], 'name': 'F'}, {'value': ['pos[0]|pos[1]=_H_|_S_'], 'name': 'F'}, {'value': ['gaz[0]|gaz[1]=loc|loc'], 'name': 'F'}]), 'name': 'next'}, {'value': 'y', 'name': 'iuoc'}, {'value': 'y', 'name': 'pprop'}, {'value': 'y', 'name': 'nprop'}, {'value': {'loc'}, 'name': 'ngaz'}, {'value': [], 'name': 'F'}, {'value': ['p3[0]=üle'], 'name': 'F'}, {'value': ['s3[0]=üle'], 'name': 'F'}, {'value': ['aan[0]=y'], 'name': 'F'}, {'value': ['iu[1]=y'], 'name': 'F'}, {'value': ['cu[1]=y'], 'name': 'F'}, {'value': ['lem[0]=üle'], 'name': 'F'}, {'value': ['pos[0]=_D_'], 'name': 'F'}, {'value': ['lem[-1]=põhi'], 'name': 'F'}, {'value': ['pos[-1]=_S_'], 'name': 'F'}, {'value': ['lem[-2]=piirnema'], 'name': 'F'}, {'value': ['pos[-2]=_V_'], 'name': 'F'}, {'value': ['lem[1]=Soome'], 'name': 'F'}, {'value': ['prop[1]=y'], 'name': 'F'}, {'value': ['pos[1]=_H_'], 'name': 'F'}, {'value': ['lem[2]=laht'], 'name': 'F'}, {'value': ['pos[2]=_S_'], 'name': 'F'}, {'value': ['iuoc[0]=y'], 'name': 'F'}, {'value': ['nprop[0]=y'], 'name': 'F'}, {'value': ['pprop[0]=y'], 'name': 'F'}, {'value': ['ngaz[0]=loc'], 'name': 'F'}, {'value': ['gaz[1]=loc'], 'name': 'F'}, {'value': ['gaz[2]=loc'], 'name': 'F'}, {'value': ['lem[0]|lem[-1]=üle|põhi'], 'name': 'F'}, {'value': ['lem[0]|lem[1]=üle|Soome'], 'name': 'F'}, {'value': ['pos[0]|pos[-1]=_D_|_S_'], 'name': 'F'}, {'value': ['pos[0]|pos[1]=_D_|_H_'], 'name': 'F'}]), 'name': 'next'}, {'value': 'y', 'name': 'iuoc'}, {'value': 'y', 'name': 'pprop'}, {'value': 'y', 'name': 'nprop'}, {'value': [], 'name': 'F'}, {'value': ['p3[0]=põh'], 'name': 'F'}, {'value': ['p4[0]=põhi'], 'name': 'F'}, {'value': ['s3[0]=õhi'], 'name': 'F'}, {'value': ['s4[0]=põhi'], 'name': 'F'}, {'value': ['aan[0]=y'], 'name': 'F'}, {'value': ['iu[-2]=y'], 'name': 'F'}, {'value': ['cu[-2]=y'], 'name': 'F'}, {'value': ['fsnt[-2]=y'], 'name': 'F'}, {'value': ['iu[2]=y'], 'name': 'F'}, {'value': ['cu[2]=y'], 'name': 'F'}, {'value': ['lem[0]=põhi'], 'name': 'F'}, {'value': ['pos[0]=_S_'], 'name': 'F'}, {'value': ['case[0]=g'], 'name': 'F'}, {'value': ['end[0]=s'], 'name': 'F'}, {'value': ['lem[-1]=piirnema'], 'name': 'F'}, {'value': ['pos[-1]=_V_'], 'name': 'F'}, {'value': ['lem[-2]=Eesti'], 'name': 'F'}, {'value': ['prop[-2]=y'], 'name': 'F'}, {'value': ['pos[-2]=_H_'], 'name': 'F'}, {'value': ['lem[1]=üle'], 'name': 'F'}, {'value': ['pos[1]=_D_'], 'name': 'F'}, {'value': ['lem[2]=Soome'], 'name': 'F'}, {'value': ['prop[2]=y'], 'name': 'F'}, {'value': ['pos[2]=_H_'], 'name': 'F'}, {'value': ['iuoc[0]=y'], 'name': 'F'}, {'value': ['nprop[0]=y'], 'name': 'F'}, {'value': ['pprop[0]=y'], 'name': 'F'}, {'value': ['gaz[-2]=loc'], 'name': 'F'}, {'value': ['gaz[2]=loc'], 'name': 'F'}, {'value': ['lem[0]|lem[-1]=põhi|piirnema'], 'name': 'F'}, {'value': ['lem[0]|lem[1]=põhi|üle'], 'name': 'F'}, {'value': ['pos[0]|pos[-1]=_S_|_V_'], 'name': 'F'}, {'value': ['pos[0]|pos[1]=_S_|_D_'], 'name': 'F'}]), 'name': 'next'}, {'value': 'y', 'name': 'iuoc'}, {'value': 'y', 'name': 'pprop'}, {'value': 'y', 'name': 'nprop'}, {'value': {'loc'}, 'name': 'pgaz'}, {'value': [], 'name': 'F'}, {'value': ['p3[0]=pii'], 'name': 'F'}, {'value': ['p4[0]=piir'], 'name': 'F'}, {'value': ['s3[0]=ema'], 'name': 'F'}, {'value': ['s4[0]=nema'], 'name': 'F'}, {'value': ['aan[0]=y'], 'name': 'F'}, {'value': ['iu[-1]=y'], 'name': 'F'}, {'value': ['cu[-1]=y'], 'name': 'F'}, {'value': ['fsnt[-1]=y'], 'name': 'F'}, {'value': ['cs[-2]=y'], 'name': 'F'}, {'value': ['cdt[-2]=y'], 'name': 'F'}, {'value': ['lem[0]=piirnema'], 'name': 'F'}, {'value': ['pos[0]=_V_'], 'name': 'F'}, {'value': ['end[0]=b'], 'name': 'F'}, {'value': ['lem[-1]=Eesti'], 'name': 'F'}, {'value': ['prop[-1]=y'], 'name': 'F'}, {'value': ['pos[-1]=_H_'], 'name': 'F'}, {'value': ['lem[-2]=.'], 'name': 'F'}, {'value': ['pos[-2]=_Z_'], 'name': 'F'}, {'value': ['pun[-2]=y'], 'name': 'F'}, {'value': ['lem[1]=põhi'], 'name': 'F'}, {'value': ['pos[1]=_S_'], 'name': 'F'}, {'value': ['lem[2]=üle'], 'name': 'F'}, {'value': ['pos[2]=_D_'], 'name': 'F'}, {'value': ['iuoc[0]=y'], 'name': 'F'}, {'value': ['nprop[0]=y'], 'name': 'F'}, {'value': ['pprop[0]=y'], 'name': 'F'}, {'value': ['pgaz[0]=loc'], 'name': 'F'}, {'value': ['gaz[-1]=loc'], 'name': 'F'}, {'value': ['lem[0]|lem[-1]=piirnema|Eesti'], 'name': 'F'}, {'value': ['lem[0]|lem[1]=piirnema|põhi'], 'name': 'F'}, {'value': ['pos[0]|pos[-1]=_V_|_H_'], 'name': 'F'}, {'value': ['pos[0]|pos[1]=_V_|_S_'], 'name': 'F'}", 'iu[0]=y', 'fsnt[0]=y', 'p3[0]=Ees', 'p4[0]=Eest', 's3[0]=sti', 's4[0]=esti', '', 'aan[0]=y', 'cu[0]=y', 'cs[-1]=y', 'cdt[-1]=y', 'iu[-2]=y', 'cu[-2]=y', 'cs[-2]=y', 'cds[-2]=y', 'adash[-2]=Euroobas', 'lem[0]=Eesti', 'pos[0]=_H_', 'case[0]=g', 'lem[-1]=.', 'pos[-1]=_Z_', 'pun[-1]=y', 'lem[-2]=Põhja-Euroobas', 'prop[-2]=y', 'pos[-2]=_H_', 'post[-2]=Eu', 'lem[1]=piirnema', 'pos[1]=_V_', 'lem[2]=põhi', 'pos[2]=_S_', 'gaz[0]=loc', 'iu[0]|fsnt[0]=y|y', 'lem[0]|lem[-1]=Eesti|.', 'lem[0]|lem[1]=Eesti|piirnema', 'pos[0]|pos[-1]=_H_|_Z_', 'pos[0]|pos[1]=_H_|_V_'], ['', '', 'loc', '', 'p3[0]=pii', 'p4[0]=piir', 's3[0]=ema', 's4[0]=nema', 'aan[0]=y', 'iu[-1]=y', 'cu[-1]=y', 'fsnt[-1]=y', 'cs[-2]=y', 'cdt[-2]=y', 'lem[0]=piirnema', 'pos[0]=_V_', 'end[0]=b', 'lem[-1]=Eesti', 'prop[-1]=y', 'pos[-1]=_H_', 'lem[-2]=.', 'pos[-2]=_Z_', 'pun[-2]=y', 'lem[1]=põhi', 'pos[1]=_S_', 'lem[2]=üle', 'pos[2]=_D_', 'iuoc[0]=y', 'nprop[0]=y', 'pprop[0]=y', 'pgaz[0]=loc', 'gaz[-1]=loc', 'lem[0]|lem[-1]=piirnema|Eesti', 'lem[0]|lem[1]=piirnema|põhi', 'pos[0]|pos[-1]=_V_|_H_', 'pos[0]|pos[1]=_V_|_S_'], ['', '', '', 'p3[0]=põh', 'p4[0]=põhi', 's3[0]=õhi', 's4[0]=põhi', 'aan[0]=y', 'iu[-2]=y', 'cu[-2]=y', 'fsnt[-2]=y', 'iu[2]=y', 'cu[2]=y', 'lem[0]=põhi', 'pos[0]=_S_', 'case[0]=g', 'end[0]=s', 'lem[-1]=piirnema', 'pos[-1]=_V_', 'lem[-2]=Eesti', 'prop[-2]=y', 'pos[-2]=_H_', 'lem[1]=üle', 'pos[1]=_D_', 'lem[2]=Soome', 'prop[2]=y', 'pos[2]=_H_', 'iuoc[0]=y', 'nprop[0]=y', 'pprop[0]=y', 'gaz[-2]=loc', 'gaz[2]=loc', 'lem[0]|lem[-1]=põhi|piirnema', 'lem[0]|lem[1]=põhi|üle', 'pos[0]|pos[-1]=_S_|_V_', 'pos[0]|pos[1]=_S_|_D_'], ['', '', 'loc', '', 'p3[0]=üle', 's3[0]=üle', 'aan[0]=y', 'iu[1]=y', 'cu[1]=y', 'lem[0]=üle', 'pos[0]=_D_', 'lem[-1]=põhi', 'pos[-1]=_S_', 'lem[-2]=piirnema', 'pos[-2]=_V_', 'lem[1]=Soome', 'prop[1]=y', 'pos[1]=_H_', 'lem[2]=laht', 'pos[2]=_S_', 'iuoc[0]=y', 'nprop[0]=y', 'pprop[0]=y', 'ngaz[0]=loc', 'gaz[1]=loc', 'gaz[2]=loc', 'lem[0]|lem[-1]=üle|põhi', 'lem[0]|lem[1]=üle|Soome', 'pos[0]|pos[-1]=_D_|_S_', 'pos[0]|pos[1]=_D_|_H_'], ['', '', '', 'loc', 'loc', 'iu[0]=y', 'p3[0]=Soo', 'p4[0]=Soom', 's3[0]=ome', 's4[0]=oome', '', 'aan[0]=y', 'cu[0]=y', 'iu[2]=y', 'cu[2]=y', 'lem[0]=Soome', 'pos[0]=_H_', 'case[0]=g', 'lem[-1]=üle', 'pos[-1]=_D_', 'lem[-2]=põhi', 'pos[-2]=_S_', 'lem[1]=laht', 'pos[1]=_S_', 'lem[2]=Soome', 'prop[2]=y', 'pos[2]=_H_', 'iuoc[0]=y', 'nprop[0]=y', 'pprop[0]=y', 'ngaz[0]=loc', 'pgaz[0]=loc', 'gaz[0]=loc', 'gaz[1]=loc', 'gaz[2]=loc', 'lem[0]|lem[-1]=Soome|üle', 'lem[0]|lem[1]=Soome|laht', 'pos[0]|pos[-1]=_H_|_D_', 'pos[0]|pos[1]=_H_|_S_', 'gaz[0]|gaz[1]=loc|loc'], ['', '', '', 'loc', 'loc', '', 'p3[0]=lah', 'p4[0]=laht', 's3[0]=aht', 's4[0]=laht', 'aan[0]=y', 'iu[-1]=y', 'cu[-1]=y', 'iu[1]=y', 'cu[1]=y', 'iu[2]=y', 'cu[2]=y', 'lem[0]=laht', 'pos[0]=_S_', 'case[0]=g', 'lem[-1]=Soome', 'prop[-1]=y', 'pos[-1]=_H_', 'lem[-2]=üle', 'pos[-2]=_D_', 'lem[1]=Soome', 'prop[1]=y', 'pos[1]=_H_', 'lem[2]=Vabariik', 'prop[2]=y', 'pos[2]=_H_', 'post[2]=riik', 'iuoc[0]=y', 'nprop[0]=y', 'pprop[0]=y', 'ngaz[0]=loc', 'pgaz[0]=loc', 'gaz[0]=loc', 'gaz[-1]=loc', 'gaz[1]=loc', 'gaz[2]=loc', 'lem[0]|lem[-1]=laht|Soome', 'lem[0]|lem[1]=laht|Soome', 'pos[0]|pos[-1]=_S_|_H_', 'pos[0]|pos[1]=_S_|_H_', 'gaz[0]|gaz[1]=loc|loc', 'gaz[-1]|gaz[0]=loc|loc'], ['', '', '', 'loc', 'loc', 'iu[0]=y', 'p3[0]=Soo', 'p4[0]=Soom', 's3[0]=ome', 's4[0]=oome', '', 'aan[0]=y', 'cu[0]=y', 'iu[-2]=y', 'cu[-2]=y', 'iu[1]=y', 'cu[1]=y', 'cs[2]=y', 'cdt[2]=y', 'lem[0]=Soome', 'pos[0]=_H_', 'case[0]=g', 'lem[-1]=laht', 'pos[-1]=_S_', 'lem[-2]=Soome', 'prop[-2]=y', 'pos[-2]=_H_', 'lem[1]=Vabariik', 'prop[1]=y', 'pos[1]=_H_', 'post[1]=riik', 'lem[2]=.', 'pos[2]=_Z_', 'pun[2]=y', 'iuoc[0]=y', 'nprop[0]=y', 'pprop[0]=y', 'ngaz[0]=loc', 'pgaz[0]=loc', 'gaz[0]=loc', 'gaz[-1]=loc', 'gaz[-2]=loc', 'gaz[1]=loc', 'lem[0]|lem[-1]=Soome|laht', 'lem[0]|lem[1]=Soome|Vabariik', 'pos[0]|pos[-1]=_H_|_S_', 'pos[0]|pos[1]=_H_|_H_', 'iu[0]|iu[1]=y|y', 'gaz[0]|gaz[1]=loc|loc', 'gaz[-1]|gaz[0]=loc|loc'], ['', '', '', 'loc', 'iu[0]=y', 'p3[0]=Vab', 'p4[0]=Vaba', 's3[0]=iik', 's4[0]=riik', '', 'aan[0]=y', 'cu[0]=y', 'iu[-1]=y', 'cu[-1]=y', 'cs[1]=y', 'cdt[1]=y', 'lem[0]=Vabariik', 'pos[0]=_H_', 'pref[0]=Vaba', 'post[0]=riik', 'case[0]=g', 'end[0]=ga', 'lem[-1]=Soome', 'prop[-1]=y', 'pos[-1]=_H_', 'lem[-2]=laht', 'pos[-2]=_S_', 'lem[1]=.', 'pos[1]=_Z_', 'pun[1]=y', 'iuoc[0]=y', 'nprop[0]=y', 'pprop[0]=y', 'pgaz[0]=loc', 'gaz[0]=loc', 'gaz[-1]=loc', 'gaz[-2]=loc', 'lem[0]|lem[-1]=Vabariik|Soome', 'lem[0]|lem[1]=Vabariik|.', 'pos[0]|pos[-1]=_H_|_H_', 'pos[0]|pos[1]=_H_|_Z_', 'iu[0]|iu[-1]=y|y', 'gaz[-1]|gaz[0]=loc|loc'], ['', '', 'loc', 'loc', '', 'ao[0]=y', 'cs[0]=y', 'cdt[0]=y', 'iu[-1]=y', 'cu[-1]=y', 'iu[-2]=y', 'cu[-2]=y', 'lem[0]=.', 'pos[0]=_Z_', 'pun[0]=y', 'end[0]=', 'lem[-1]=Vabariik', 'prop[-1]=y', 'pos[-1]=_H_', 'post[-1]=riik', 'lem[-2]=Soome', 'prop[-2]=y', 'pos[-2]=_H_', 'iuoc[0]=y', 'nprop[0]=y', 'pprop[0]=y', 'ngaz[0]=loc', 'pgaz[0]=loc', 'gaz[-1]=loc', 'gaz[-2]=loc', 'lem[0]|lem[-1]=.|Vabariik', 'pos[0]|pos[-1]=_Z_|_H_']]

In [29]:
inputs = [['iu[0]=y', 'fsnt[0]=y', 'p3[0]=ees', 'p4[0]=eest', 's3[0]=sti', 's4[0]=esti', 'aan[0]=y', 'cu[0]=y', 'iu[1]=y', 'cu[1]=y', 'lem[0]=eesti', 'pos[0]=_H_', 'case[0]=g', 'lem[1]=vabariik', 'pos[1]=_S_', 'post[1]=riik', 'lem[2]=ole', 'pos[2]=_V_', 'iuoc[0]=y', 'ngaz[0]=loc', 'gaz[0]=loc', 'gaz[1]=loc', 'iu[0]|fsnt[0]=y|y', 'lem[0]|lem[1]=eesti|vabariik', 'pos[0]|pos[1]=_H_|_S_', 'iu[0]|iu[1]=y|y', 'gaz[0]|gaz[1]=loc|loc'], ['iu[0]=y', 'p3[0]=vab', 'p4[0]=vaba', 's3[0]=iik', 's4[0]=riik', 'aan[0]=y', 'cu[0]=y', 'iu[-1]=y', 'cu[-1]=y', 'fsnt[-1]=y', 'lem[0]=vabariik', 'pos[0]=_S_', 'pref[0]=vaba', 'post[0]=riik', 'case[0]=n', 'lem[-1]=eesti', 'prop[-1]=y', 'pos[-1]=_H_', 'lem[1]=ole', 'pos[1]=_V_', 'lem[2]=riik', 'pos[2]=_S_', 'iuoc[0]=y', 'pprop[0]=y', 'pgaz[0]=loc', 'gaz[0]=loc', 'gaz[-1]=loc', 'lem[0]|lem[-1]=vabariik|eesti', 'lem[0]|lem[1]=vabariik|ole', 'pos[0]|pos[-1]=_S_|_H_', 'pos[0]|pos[1]=_S_|_V_', 'iu[0]|iu[-1]=y|y', 'gaz[-1]|gaz[0]=loc|loc'], ['p3[0]=ole', 's3[0]=ole', 'aan[0]=y', 'iu[-1]=y', 'cu[-1]=y', 'iu[-2]=y', 'cu[-2]=y', 'fsnt[-2]=y', 'iu[2]=y', 'cu[2]=y', 'cs[2]=y', 'cds[2]=y', 'lem[0]=ole', 'pos[0]=_V_', 'lem[-1]=vabariik', 'pos[-1]=_S_', 'post[-1]=riik', 'lem[-2]=eesti', 'prop[-2]=y', 'pos[-2]=_H_', 'lem[1]=riik', 'pos[1]=_S_', 'lem[2]=põhjaeuroobas', 'prop[2]=y', 'pos[2]=_H_', 'post[2]=roobas', 'gaz[-1]=loc', 'gaz[-2]=loc', 'lem[0]|lem[-1]=ole|vabariik', 'lem[0]|lem[1]=ole|riik', 'pos[0]|pos[-1]=_V_|_S_', 'pos[0]|pos[1]=_V_|_S_'], ['p3[0]=rii', 'p4[0]=riik', 's3[0]=iik', 's4[0]=riik', 'aan[0]=y', 'iu[-2]=y', 'cu[-2]=y', 'iu[1]=y', 'cu[1]=y', 'cs[1]=y', 'cds[1]=y', 'cs[2]=y', 'cdt[2]=y', 'lem[0]=riik', 'pos[0]=_S_', 'case[0]=n', 'lem[-1]=ole', 'pos[-1]=_V_', 'lem[-2]=vabariik', 'pos[-2]=_S_', 'post[-2]=riik', 'lem[1]=põhjaeuroobas', 'prop[1]=y', 'pos[1]=_H_', 'post[1]=roobas', 'lem[2]=.', 'pos[2]=_Z_', 'pun[2]=y', 'gaz[-2]=loc', 'lem[0]|lem[-1]=riik|ole', 'lem[0]|lem[1]=riik|põhjaeuroobas', 'pos[0]|pos[-1]=_S_|_V_', 'pos[0]|pos[1]=_S_|_H_'], ['iu[0]=y', 'p3[0]=põh', 'p4[0]=põhj', 's3[0]=bas', 's4[0]=obas', 'cu[0]=y', 'cs[0]=y', 'cds[0]=y', 'cs[1]=y', 'cdt[1]=y', 'iu[2]=y', 'cu[2]=y', 'lem[0]=põhjaeuroobas', 'pos[0]=_H_', 'pref[0]=põhja', 'post[0]=roobas', 'case[0]=in', 'end[0]=s', 'lem[-1]=riik', 'pos[-1]=_S_', 'lem[-2]=ole', 'pos[-2]=_V_', 'lem[1]=.', 'pos[1]=_Z_', 'pun[1]=y', 'lem[2]=eesti', 'prop[2]=y', 'pos[2]=_H_', 'iuoc[0]=y', 'gaz[2]=loc', 'lem[0]|lem[-1]=põhjaeuroobas|riik', 'lem[0]|lem[1]=põhjaeuroobas|.', 'pos[0]|pos[-1]=_H_|_S_', 'pos[0]|pos[1]=_H_|_Z_'], ['ao[0]=y', 'cs[0]=y', 'cdt[0]=y', 'iu[-1]=y', 'cu[-1]=y', 'cs[-1]=y', 'cds[-1]=y', 'iu[1]=y', 'cu[1]=y', 'lem[0]=.', 'pos[0]=_Z_', 'pun[0]=y', 'lem[-1]=põhjaeuroobas', 'prop[-1]=y', 'pos[-1]=_H_', 'post[-1]=roobas', 'lem[-2]=riik', 'pos[-2]=_S_', 'lem[1]=eesti', 'prop[1]=y', 'pos[1]=_H_', 'lem[2]=piirne', 'pos[2]=_V_', 'gaz[1]=loc', 'lem[0]|lem[-1]=.|põhjaeuroobas', 'lem[0]|lem[1]=.|eesti', 'pos[0]|pos[-1]=_Z_|_H_', 'pos[0]|pos[1]=_Z_|_H_'], ['iu[0]=y', 'p3[0]=ees', 'p4[0]=eest', 's3[0]=sti', 's4[0]=esti', 'aan[0]=y', 'cu[0]=y', 'cs[-1]=y', 'cdt[-1]=y', 'iu[-2]=y', 'cu[-2]=y', 'cs[-2]=y', 'cds[-2]=y', 'lem[0]=eesti', 'pos[0]=_H_', 'case[0]=n', 'lem[-1]=.', 'pos[-1]=_Z_', 'pun[-1]=y', 'lem[-2]=põhjaeuroobas', 'prop[-2]=y', 'pos[-2]=_H_', 'post[-2]=roobas', 'lem[1]=piirne', 'pos[1]=_V_', 'lem[2]=põhi', 'pos[2]=_S_', 'iuoc[0]=y', 'ngaz[0]=loc', 'gaz[0]=loc', 'lem[0]|lem[-1]=eesti|.', 'lem[0]|lem[1]=eesti|piirne', 'pos[0]|pos[-1]=_H_|_Z_', 'pos[0]|pos[1]=_H_|_V_'], ['p3[0]=pii', 'p4[0]=piir', 's3[0]=rne', 's4[0]=irne', 'aan[0]=y', 'iu[-1]=y', 'cu[-1]=y', 'cs[-2]=y', 'cdt[-2]=y', 'lem[0]=piirne', 'pos[0]=_V_', 'end[0]=b', 'lem[-1]=eesti', 'prop[-1]=y', 'pos[-1]=_H_', 'lem[-2]=.', 'pos[-2]=_Z_', 'pun[-2]=y', 'lem[1]=põhi', 'pos[1]=_S_', 'lem[2]=üle', 'pos[2]=_D_', 'gaz[-1]=loc', 'lem[0]|lem[-1]=piirne|eesti', 'lem[0]|lem[1]=piirne|põhi', 'pos[0]|pos[-1]=_V_|_H_', 'pos[0]|pos[1]=_V_|_S_'], ['p3[0]=põh', 'p4[0]=põhi', 's3[0]=õhi', 's4[0]=põhi', 'aan[0]=y', 'iu[-2]=y', 'cu[-2]=y', 'iu[2]=y', 'cu[2]=y', 'lem[0]=põhi', 'pos[0]=_S_', 'case[0]=in', 'end[0]=s', 'lem[-1]=piirne', 'pos[-1]=_V_', 'lem[-2]=eesti', 'prop[-2]=y', 'pos[-2]=_H_', 'lem[1]=üle', 'pos[1]=_D_', 'lem[2]=soome', 'prop[2]=y', 'pos[2]=_H_', 'gaz[-2]=loc', 'gaz[2]=loc', 'lem[0]|lem[-1]=põhi|piirne', 'lem[0]|lem[1]=põhi|üle', 'pos[0]|pos[-1]=_S_|_V_', 'pos[0]|pos[1]=_S_|_D_'], ['p3[0]=üle', 's3[0]=üle', 'aan[0]=y', 'iu[1]=y', 'cu[1]=y', 'lem[0]=üle', 'pos[0]=_D_', 'lem[-1]=põhi', 'pos[-1]=_S_', 'lem[-2]=piirne', 'pos[-2]=_V_', 'lem[1]=soome', 'prop[1]=y', 'pos[1]=_H_', 'lem[2]=laht', 'pos[2]=_S_', 'gaz[1]=loc', 'gaz[2]=loc', 'lem[0]|lem[-1]=üle|põhi', 'lem[0]|lem[1]=üle|soome', 'pos[0]|pos[-1]=_D_|_S_', 'pos[0]|pos[1]=_D_|_H_'], ['iu[0]=y', 'p3[0]=soo', 'p4[0]=soom', 's3[0]=ome', 's4[0]=oome', 'aan[0]=y', 'cu[0]=y', 'iu[2]=y', 'cu[2]=y', 'lem[0]=soome', 'pos[0]=_H_', 'case[0]=g', 'lem[-1]=üle', 'pos[-1]=_D_', 'lem[-2]=põhi', 'pos[-2]=_S_', 'lem[1]=laht', 'pos[1]=_S_', 'lem[2]=soome', 'prop[2]=y', 'pos[2]=_H_', 'iuoc[0]=y', 'nprop[0]=y', 'ngaz[0]=loc', 'pgaz[0]=loc', 'gaz[0]=loc', 'gaz[1]=loc', 'gaz[2]=loc', 'lem[0]|lem[-1]=soome|üle', 'lem[0]|lem[1]=soome|laht', 'pos[0]|pos[-1]=_H_|_D_', 'pos[0]|pos[1]=_H_|_S_', 'gaz[0]|gaz[1]=loc|loc'], ['p3[0]=lah', 'p4[0]=laht', 's3[0]=aht', 's4[0]=laht', 'aan[0]=y', 'iu[-1]=y', 'cu[-1]=y', 'iu[1]=y', 'cu[1]=y', 'iu[2]=y', 'cu[2]=y', 'lem[0]=laht', 'pos[0]=_S_', 'case[0]=g', 'lem[-1]=soome', 'prop[-1]=y', 'pos[-1]=_H_', 'lem[-2]=üle', 'pos[-2]=_D_', 'lem[1]=soome', 'prop[1]=y', 'pos[1]=_H_', 'lem[2]=vabariik', 'prop[2]=y', 'pos[2]=_H_', 'post[2]=riik', 'gaz[0]=loc', 'gaz[-1]=loc', 'gaz[1]=loc', 'gaz[2]=loc', 'lem[0]|lem[-1]=laht|soome', 'lem[0]|lem[1]=laht|soome', 'pos[0]|pos[-1]=_S_|_H_', 'pos[0]|pos[1]=_S_|_H_', 'gaz[0]|gaz[1]=loc|loc', 'gaz[-1]|gaz[0]=loc|loc'], ['iu[0]=y', 'p3[0]=soo', 'p4[0]=soom', 's3[0]=ome', 's4[0]=oome', 'aan[0]=y', 'cu[0]=y', 'iu[-2]=y', 'cu[-2]=y', 'iu[1]=y', 'cu[1]=y', 'cs[2]=y', 'cdt[2]=y', 'lem[0]=soome', 'pos[0]=_H_', 'case[0]=n', 'lem[-1]=laht', 'pos[-1]=_S_', 'lem[-2]=soome', 'prop[-2]=y', 'pos[-2]=_H_', 'lem[1]=vabariik', 'prop[1]=y', 'pos[1]=_H_', 'post[1]=riik', 'lem[2]=.', 'pos[2]=_Z_', 'pun[2]=y', 'iuoc[0]=y', 'nprop[0]=y', 'ngaz[0]=loc', 'pgaz[0]=loc', 'gaz[0]=loc', 'gaz[-1]=loc', 'gaz[-2]=loc', 'gaz[1]=loc', 'lem[0]|lem[-1]=soome|laht', 'lem[0]|lem[1]=soome|vabariik', 'pos[0]|pos[-1]=_H_|_S_', 'pos[0]|pos[1]=_H_|_H_', 'iu[0]|iu[1]=y|y', 'gaz[0]|gaz[1]=loc|loc', 'gaz[-1]|gaz[0]=loc|loc'], ['iu[0]=y', 'p3[0]=vab', 'p4[0]=vaba', 's3[0]=iik', 's4[0]=riik', 'aan[0]=y', 'cu[0]=y', 'iu[-1]=y', 'cu[-1]=y', 'cs[1]=y', 'cdt[1]=y', 'lem[0]=vabariik', 'pos[0]=_H_', 'pref[0]=vaba', 'post[0]=riik', 'case[0]=kom', 'end[0]=ga', 'lem[-1]=soome', 'prop[-1]=y', 'pos[-1]=_H_', 'lem[-2]=laht', 'pos[-2]=_S_', 'lem[1]=.', 'pos[1]=_Z_', 'pun[1]=y', 'iuoc[0]=y', 'pprop[0]=y', 'pgaz[0]=loc', 'gaz[0]=loc', 'gaz[-1]=loc', 'gaz[-2]=loc', 'lem[0]|lem[-1]=vabariik|soome', 'lem[0]|lem[1]=vabariik|.', 'pos[0]|pos[-1]=_H_|_H_', 'pos[0]|pos[1]=_H_|_Z_', 'iu[0]|iu[-1]=y|y', 'gaz[-1]|gaz[0]=loc|loc'], ['ao[0]=y', 'cs[0]=y', 'cdt[0]=y', 'iu[-1]=y', 'cu[-1]=y', 'iu[-2]=y', 'cu[-2]=y', 'lem[0]=.', 'pos[0]=_Z_', 'pun[0]=y', 'lem[-1]=vabariik', 'prop[-1]=y', 'pos[-1]=_H_', 'post[-1]=riik', 'lem[-2]=soome', 'prop[-2]=y', 'pos[-2]=_H_', 'gaz[-1]=loc', 'gaz[-2]=loc', 'lem[0]|lem[-1]=.|vabariik', 'pos[0]|pos[-1]=_Z_|_H_']]

In [37]:
loppinput[3]

['',
 '',
 '',
 'p3[0]=rii',
 'p4[0]=riik',
 's3[0]=iik',
 's4[0]=riik',
 'aan[0]=y',
 'iu[-2]=y',
 'cu[-2]=y',
 'iu[1]=y',
 'cu[1]=y',
 'cs[1]=y',
 'cds[1]=y',
 'adash[1]=Euroobas',
 'cs[2]=y',
 'cdt[2]=y',
 'lem[0]=riik',
 'pos[0]=_S_',
 'case[0]=g',
 'lem[-1]=olema',
 'pos[-1]=_V_',
 'lem[-2]=vabariik',
 'pos[-2]=_S_',
 'post[-2]=riik',
 'lem[1]=Põhja-Euroobas',
 'prop[1]=y',
 'pos[1]=_H_',
 'post[1]=Eu',
 'lem[2]=.',
 'pos[2]=_Z_',
 'pun[2]=y',
 'iuoc[0]=y',
 'nprop[0]=y',
 'pprop[0]=y',
 'gaz[-2]=loc',
 'lem[0]|lem[-1]=riik|olema',
 'lem[0]|lem[1]=riik|Põhja-Euroobas',
 'pos[0]|pos[-1]=_S_|_V_',
 'pos[0]|pos[1]=_S_|_H_']

In [5]:
inputs[0]

['iu[0]=y',
 'fsnt[0]=y',
 'p3[0]=ees',
 'p4[0]=eest',
 's3[0]=sti',
 's4[0]=esti',
 'aan[0]=y',
 'cu[0]=y',
 'iu[1]=y',
 'cu[1]=y',
 'lem[0]=eesti',
 'pos[0]=_H_',
 'case[0]=g',
 'lem[1]=vabariik',
 'pos[1]=_S_',
 'post[1]=riik',
 'lem[2]=ole',
 'pos[2]=_V_',
 'iuoc[0]=y',
 'ngaz[0]=loc',
 'gaz[0]=loc',
 'gaz[1]=loc',
 'iu[0]|fsnt[0]=y|y',
 'lem[0]|lem[1]=eesti|vabariik',
 'pos[0]|pos[1]=_H_|_S_',
 'iu[0]|iu[1]=y|y',
 'gaz[0]|gaz[1]=loc|loc']

In [74]:
loppinput2 = [['iu[0]=y', 'fsnt[0]=y', 'p3[0]=Ees', 'p4[0]=Eest', 's3[0]=sti', 's4[0]=esti', 'aan[0]=y', 'cu[0]=y', 'iu[1]=y', 'cu[1]=y', 'lem[0]=Eesti', 'pos[0]=_H_', 'case[0]=g', 'lem[1]=vabariik', 'pos[1]=_S_', 'post[1]=riik', 'lem[2]=olema', 'pos[2]=_V_', 'gaz[0]=loc', 'gaz[1]=loc', 'iu[0]|fsnt[0]=y|y', 'lem[0]|lem[1]=Eesti|vabariik', 'pos[0]|pos[1]=_H_|_S_', 'iu[0]|iu[1]=y|y', 'gaz[0]|gaz[1]=loc|loc'], ['iu[0]=y', 'p3[0]=vab', 'p4[0]=vaba', 's3[0]=iik', 's4[0]=riik', 'aan[0]=y', 'cu[0]=y', 'iu[-1]=y', 'cu[-1]=y', 'fsnt[-1]=y', 'lem[0]=vabariik', 'pos[0]=_S_', 'pref[0]=vaba', 'post[0]=riik', 'case[0]=g', 'lem[-1]=Eesti', 'prop[-1]=y', 'pos[-1]=_H_', 'lem[1]=olema', 'pos[1]=_V_', 'lem[2]=riik', 'pos[2]=_S_', 'iuoc[0]=y', 'nprop[0]=y', 'pprop[0]=y', 'pgaz[0]=loc', 'gaz[0]=loc', 'gaz[-1]=loc', 'lem[0]|lem[-1]=vabariik|Eesti', 'lem[0]|lem[1]=vabariik|olema', 'pos[0]|pos[-1]=_S_|_H_', 'pos[0]|pos[1]=_S_|_V_', 'iu[0]|iu[-1]=y|y', 'gaz[-1]|gaz[0]=loc|loc'], ['p3[0]=ole', 'p4[0]=olem', 's3[0]=ema', 's4[0]=lema', 'aan[0]=y', 'iu[-1]=y', 'cu[-1]=y', 'iu[-2]=y', 'cu[-2]=y', 'fsnt[-2]=y', 'iu[2]=y', 'cu[2]=y', 'cs[2]=y', 'cds[2]=y', 'adash[2]=Euroobas', 'lem[0]=olema', 'pos[0]=_V_', 'lem[-1]=vabariik', 'pos[-1]=_S_', 'post[-1]=riik', 'lem[-2]=Eesti', 'prop[-2]=y', 'pos[-2]=_H_', 'lem[1]=riik', 'pos[1]=_S_', 'lem[2]=Põhja-Euroobas', 'prop[2]=y', 'pos[2]=_H_', 'post[2]=Eu', 'iuoc[0]=y', 'nprop[0]=y', 'pprop[0]=y', 'pgaz[0]=loc', 'gaz[-1]=loc', 'gaz[-2]=loc', 'lem[0]|lem[-1]=olema|vabariik', 'lem[0]|lem[1]=olema|riik', 'pos[0]|pos[-1]=_V_|_S_', 'pos[0]|pos[1]=_V_|_S_'], ['p3[0]=rii', 'p4[0]=riik', 's3[0]=iik', 's4[0]=riik', 'aan[0]=y', 'iu[-2]=y', 'cu[-2]=y', 'iu[1]=y', 'cu[1]=y', 'cs[1]=y', 'cds[1]=y', 'adash[1]=Euroobas', 'cs[2]=y', 'cdt[2]=y', 'lem[0]=riik', 'pos[0]=_S_', 'case[0]=g', 'lem[-1]=olema', 'pos[-1]=_V_', 'lem[-2]=vabariik', 'pos[-2]=_S_', 'post[-2]=riik', 'lem[1]=Põhja-Euroobas', 'prop[1]=y', 'pos[1]=_H_', 'post[1]=Eu', 'lem[2]=.', 'pos[2]=_Z_', 'pun[2]=y', 'iuoc[0]=y', 'nprop[0]=y', 'pprop[0]=y', 'gaz[-2]=loc', 'lem[0]|lem[-1]=riik|olema', 'lem[0]|lem[1]=riik|Põhja-Euroobas', 'pos[0]|pos[-1]=_S_|_V_', 'pos[0]|pos[1]=_S_|_H_'], ['iu[0]=y', 'p3[0]=Põh', 'p4[0]=Põhj', 's3[0]=bas', 's4[0]=obas', 'bdash[0]=Põhja', 'adash[0]=Euroobas', 'cu[0]=y', 'cs[0]=y', 'cds[0]=y', 'cs[1]=y', 'cdt[1]=y', 'iu[2]=y', 'cu[2]=y', 'fsnt[2]=y', 'lem[0]=Põhja-Euroobas', 'pos[0]=_H_', 'pref[0]=Põhja', 'post[0]=Eu', 'case[0]=g', 'end[0]=s', 'lem[-1]=riik', 'pos[-1]=_S_', 'lem[-2]=olema', 'pos[-2]=_V_', 'lem[1]=.', 'pos[1]=_Z_', 'pun[1]=y', 'lem[2]=Eesti', 'prop[2]=y', 'pos[2]=_H_', 'iuoc[0]=y', 'nprop[0]=y', 'pprop[0]=y', 'gaz[2]=loc', 'lem[0]|lem[-1]=Põhja-Euroobas|riik', 'lem[0]|lem[1]=Põhja-Euroobas|.', 'pos[0]|pos[-1]=_H_|_S_', 'pos[0]|pos[1]=_H_|_Z_'], ['ao[0]=y', 'cs[0]=y', 'cdt[0]=y', 'iu[-1]=y', 'cu[-1]=y', 'cs[-1]=y', 'cds[-1]=y', 'adash[-1]=Euroobas', 'iu[1]=y', 'cu[1]=y', 'fsnt[1]=y', 'lem[0]=.', 'pos[0]=_Z_', 'pun[0]=y', 'end[0]=', 'lem[-1]=Põhja-Euroobas', 'prop[-1]=y', 'pos[-1]=_H_', 'post[-1]=Eu', 'lem[-2]=riik', 'pos[-2]=_S_', 'lem[1]=Eesti', 'prop[1]=y', 'pos[1]=_H_', 'lem[2]=piirnema', 'pos[2]=_V_', 'iuoc[0]=y', 'nprop[0]=y', 'pprop[0]=y', 'ngaz[0]=loc', 'pgaz[0]=loc', 'gaz[1]=loc', 'lem[0]|lem[-1]=.|Põhja-Euroobas', 'lem[0]|lem[1]=.|Eesti', 'pos[0]|pos[-1]=_Z_|_H_', 'pos[0]|pos[1]=_Z_|_H_'], ['iu[0]=y', 'fsnt[0]=y', 'p3[0]=Ees', 'p4[0]=Eest', 's3[0]=sti', 's4[0]=esti', 'aan[0]=y', 'cu[0]=y', 'cs[-1]=y', 'cdt[-1]=y', 'iu[-2]=y', 'cu[-2]=y', 'cs[-2]=y', 'cds[-2]=y', 'adash[-2]=Euroobas', 'lem[0]=Eesti', 'pos[0]=_H_', 'case[0]=g', 'lem[-1]=.', 'pos[-1]=_Z_', 'pun[-1]=y', 'lem[-2]=Põhja-Euroobas', 'prop[-2]=y', 'pos[-2]=_H_', 'post[-2]=Eu', 'lem[1]=piirnema', 'pos[1]=_V_', 'lem[2]=põhi', 'pos[2]=_S_', 'gaz[0]=loc', 'iu[0]|fsnt[0]=y|y', 'lem[0]|lem[-1]=Eesti|.', 'lem[0]|lem[1]=Eesti|piirnema', 'pos[0]|pos[-1]=_H_|_Z_', 'pos[0]|pos[1]=_H_|_V_'], ['p3[0]=pii', 'p4[0]=piir', 's3[0]=ema', 's4[0]=nema', 'aan[0]=y', 'iu[-1]=y', 'cu[-1]=y', 'fsnt[-1]=y', 'cs[-2]=y', 'cdt[-2]=y', 'lem[0]=piirnema', 'pos[0]=_V_', 'end[0]=b', 'lem[-1]=Eesti', 'prop[-1]=y', 'pos[-1]=_H_', 'lem[-2]=.', 'pos[-2]=_Z_', 'pun[-2]=y', 'lem[1]=põhi', 'pos[1]=_S_', 'lem[2]=üle', 'pos[2]=_D_', 'iuoc[0]=y', 'nprop[0]=y', 'pprop[0]=y', 'pgaz[0]=loc', 'gaz[-1]=loc', 'lem[0]|lem[-1]=piirnema|Eesti', 'lem[0]|lem[1]=piirnema|põhi', 'pos[0]|pos[-1]=_V_|_H_', 'pos[0]|pos[1]=_V_|_S_'], ['p3[0]=põh', 'p4[0]=põhi', 's3[0]=õhi', 's4[0]=põhi', 'aan[0]=y', 'iu[-2]=y', 'cu[-2]=y', 'fsnt[-2]=y', 'iu[2]=y', 'cu[2]=y', 'lem[0]=põhi', 'pos[0]=_S_', 'case[0]=g', 'end[0]=s', 'lem[-1]=piirnema', 'pos[-1]=_V_', 'lem[-2]=Eesti', 'prop[-2]=y', 'pos[-2]=_H_', 'lem[1]=üle', 'pos[1]=_D_', 'lem[2]=Soome', 'prop[2]=y', 'pos[2]=_H_', 'iuoc[0]=y', 'nprop[0]=y', 'pprop[0]=y', 'gaz[-2]=loc', 'gaz[2]=loc', 'lem[0]|lem[-1]=põhi|piirnema', 'lem[0]|lem[1]=põhi|üle', 'pos[0]|pos[-1]=_S_|_V_', 'pos[0]|pos[1]=_S_|_D_'], ['p3[0]=üle', 's3[0]=üle', 'aan[0]=y', 'iu[1]=y', 'cu[1]=y', 'lem[0]=üle', 'pos[0]=_D_', 'lem[-1]=põhi', 'pos[-1]=_S_', 'lem[-2]=piirnema', 'pos[-2]=_V_', 'lem[1]=Soome', 'prop[1]=y', 'pos[1]=_H_', 'lem[2]=laht', 'pos[2]=_S_', 'iuoc[0]=y', 'nprop[0]=y', 'pprop[0]=y', 'ngaz[0]=loc', 'gaz[1]=loc', 'gaz[2]=loc', 'lem[0]|lem[-1]=üle|põhi', 'lem[0]|lem[1]=üle|Soome', 'pos[0]|pos[-1]=_D_|_S_', 'pos[0]|pos[1]=_D_|_H_'], ['iu[0]=y', 'p3[0]=Soo', 'p4[0]=Soom', 's3[0]=ome', 's4[0]=oome', 'aan[0]=y', 'cu[0]=y', 'iu[2]=y', 'cu[2]=y', 'lem[0]=Soome', 'pos[0]=_H_', 'case[0]=g', 'lem[-1]=üle', 'pos[-1]=_D_', 'lem[-2]=põhi', 'pos[-2]=_S_', 'lem[1]=laht', 'pos[1]=_S_', 'lem[2]=Soome', 'prop[2]=y', 'pos[2]=_H_', 'iuoc[0]=y', 'nprop[0]=y', 'pprop[0]=y', 'ngaz[0]=loc', 'pgaz[0]=loc', 'gaz[0]=loc', 'gaz[1]=loc', 'gaz[2]=loc', 'lem[0]|lem[-1]=Soome|üle', 'lem[0]|lem[1]=Soome|laht', 'pos[0]|pos[-1]=_H_|_D_', 'pos[0]|pos[1]=_H_|_S_', 'gaz[0]|gaz[1]=loc|loc'], ['p3[0]=lah', 'p4[0]=laht', 's3[0]=aht', 's4[0]=laht', 'aan[0]=y', 'iu[-1]=y', 'cu[-1]=y', 'iu[1]=y', 'cu[1]=y', 'iu[2]=y', 'cu[2]=y', 'lem[0]=laht', 'pos[0]=_S_', 'case[0]=g', 'lem[-1]=Soome', 'prop[-1]=y', 'pos[-1]=_H_', 'lem[-2]=üle', 'pos[-2]=_D_', 'lem[1]=Soome', 'prop[1]=y', 'pos[1]=_H_', 'lem[2]=Vabariik', 'prop[2]=y', 'pos[2]=_H_', 'post[2]=riik', 'iuoc[0]=y', 'nprop[0]=y', 'pprop[0]=y', 'ngaz[0]=loc', 'pgaz[0]=loc', 'gaz[0]=loc', 'gaz[-1]=loc', 'gaz[1]=loc', 'gaz[2]=loc', 'lem[0]|lem[-1]=laht|Soome', 'lem[0]|lem[1]=laht|Soome', 'pos[0]|pos[-1]=_S_|_H_', 'pos[0]|pos[1]=_S_|_H_', 'gaz[0]|gaz[1]=loc|loc', 'gaz[-1]|gaz[0]=loc|loc'], ['iu[0]=y', 'p3[0]=Soo', 'p4[0]=Soom', 's3[0]=ome', 's4[0]=oome', 'aan[0]=y', 'cu[0]=y', 'iu[-2]=y', 'cu[-2]=y', 'iu[1]=y', 'cu[1]=y', 'cs[2]=y', 'cdt[2]=y', 'lem[0]=Soome', 'pos[0]=_H_', 'case[0]=g', 'lem[-1]=laht', 'pos[-1]=_S_', 'lem[-2]=Soome', 'prop[-2]=y', 'pos[-2]=_H_', 'lem[1]=Vabariik', 'prop[1]=y', 'pos[1]=_H_', 'post[1]=riik', 'lem[2]=.', 'pos[2]=_Z_', 'pun[2]=y', 'iuoc[0]=y', 'nprop[0]=y', 'pprop[0]=y', 'ngaz[0]=loc', 'pgaz[0]=loc', 'gaz[0]=loc', 'gaz[-1]=loc', 'gaz[-2]=loc', 'gaz[1]=loc', 'lem[0]|lem[-1]=Soome|laht', 'lem[0]|lem[1]=Soome|Vabariik', 'pos[0]|pos[-1]=_H_|_S_', 'pos[0]|pos[1]=_H_|_H_', 'iu[0]|iu[1]=y|y', 'gaz[0]|gaz[1]=loc|loc', 'gaz[-1]|gaz[0]=loc|loc'], ['iu[0]=y', 'p3[0]=Vab', 'p4[0]=Vaba', 's3[0]=iik', 's4[0]=riik', 'aan[0]=y', 'cu[0]=y', 'iu[-1]=y', 'cu[-1]=y', 'cs[1]=y', 'cdt[1]=y', 'lem[0]=Vabariik', 'pos[0]=_H_', 'pref[0]=Vaba', 'post[0]=riik', 'case[0]=g', 'end[0]=ga', 'lem[-1]=Soome', 'prop[-1]=y', 'pos[-1]=_H_', 'lem[-2]=laht', 'pos[-2]=_S_', 'lem[1]=.', 'pos[1]=_Z_', 'pun[1]=y', 'iuoc[0]=y', 'nprop[0]=y', 'pprop[0]=y', 'pgaz[0]=loc', 'gaz[0]=loc', 'gaz[-1]=loc', 'gaz[-2]=loc', 'lem[0]|lem[-1]=Vabariik|Soome', 'lem[0]|lem[1]=Vabariik|.', 'pos[0]|pos[-1]=_H_|_H_', 'pos[0]|pos[1]=_H_|_Z_', 'iu[0]|iu[-1]=y|y', 'gaz[-1]|gaz[0]=loc|loc'], ['ao[0]=y', 'cs[0]=y', 'cdt[0]=y', 'iu[-1]=y', 'cu[-1]=y', 'iu[-2]=y', 'cu[-2]=y', 'lem[0]=.', 'pos[0]=_Z_', 'pun[0]=y', 'end[0]=', 'lem[-1]=Vabariik', 'prop[-1]=y', 'pos[-1]=_H_', 'post[-1]=riik', 'lem[-2]=Soome', 'prop[-2]=y', 'pos[-2]=_H_', 'iuoc[0]=y', 'nprop[0]=y', 'pprop[0]=y', 'ngaz[0]=loc', 'pgaz[0]=loc', 'gaz[-1]=loc', 'gaz[-2]=loc', 'lem[0]|lem[-1]=.|Vabariik', 'pos[0]|pos[-1]=_Z_|_H_']]

In [1]:
inputs[0]

NameError: name 'inputs' is not defined

In [81]:
loppinput2[1]

['iu[0]=y',
 'p3[0]=vab',
 'p4[0]=vaba',
 's3[0]=iik',
 's4[0]=riik',
 'aan[0]=y',
 'cu[0]=y',
 'iu[-1]=y',
 'cu[-1]=y',
 'fsnt[-1]=y',
 'lem[0]=vabariik',
 'pos[0]=_S_',
 'pref[0]=vaba',
 'post[0]=riik',
 'case[0]=g',
 'lem[-1]=Eesti',
 'prop[-1]=y',
 'pos[-1]=_H_',
 'lem[1]=olema',
 'pos[1]=_V_',
 'lem[2]=riik',
 'pos[2]=_S_',
 'iuoc[0]=y',
 'nprop[0]=y',
 'pprop[0]=y',
 'pgaz[0]=loc',
 'gaz[0]=loc',
 'gaz[-1]=loc',
 'lem[0]|lem[-1]=vabariik|Eesti',
 'lem[0]|lem[1]=vabariik|olema',
 'pos[0]|pos[-1]=_S_|_H_',
 'pos[0]|pos[1]=_S_|_V_',
 'iu[0]|iu[-1]=y|y',
 'gaz[-1]|gaz[0]=loc|loc']